# Import

In [ ]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Percorso alla cartella desiderata
# Cambia la directory alla cartella desiderata
os.chdir('/content/drive/My Drive/Thesis/ANALISI_TESI')  # Modifica il percorso con il nome corretto


# Conferma che la directory è stata impostata
print("Directory corrente:", os.getcwd())

Directory corrente: /content/drive/My Drive/Thesis/ANALISI_TESI


In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
#per aumentare la dimensione massima vidibile di colonne e righe
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

pd.reset_option('display.max_rows')

## Read the raw data

In [ ]:
root_directory = "Extraction ALL 20122024"

# Columns to exclude (ATTENZIONE A QUESTO, POTREBBE ESSERE UTILE RIMETTERNE DENTRO ALCUNI)
columns_to_exclude = [
    # 'Mean PSD Ch1 Delta (dB)', 'Mean PSD Ch1 Theta (dB)'
    # , 'Mean PSD Ch1 Alpha (dB)', 'Mean PSD Ch1 Beta (dB)'
    # , 'Mean PSD Ch1 Gamma (dB)',
    # 'Mean PSD Ch2 Delta (dB)', 'Mean PSD Ch2 Theta (dB)', 'Mean PSD Ch2 Alpha (dB)', 'Mean PSD Ch2 Beta (dB)', 'Mean PSD Ch2 Gamma (dB)',
    # 'Mean PSD Ch3 Delta (dB)', 'Mean PSD Ch3 Theta (dB)', 'Mean PSD Ch3 Alpha (dB)', 'Mean PSD Ch3 Beta (dB)', 'Mean PSD Ch3 Gamma (dB)',
    # 'Mean PSD Ch4 Delta (dB)', 'Mean PSD Ch4 Theta (dB)', 'Mean PSD Ch4 Alpha (dB)', 'Mean PSD Ch4 Beta (dB)', 'Mean PSD Ch4 Gamma (dB)',
    # 'Mean PSD Ch5 Delta (dB)', 'Mean PSD Ch5 Theta (dB)', 'Mean PSD Ch5 Alpha (dB)', 'Mean PSD Ch5 Beta (dB)', 'Mean PSD Ch5 Gamma (dB)',
    # 'Mean PSD Ch6 Delta (dB)', 'Mean PSD Ch6 Theta (dB)', 'Mean PSD Ch6 Alpha (dB)', 'Mean PSD Ch6 Beta (dB)', 'Mean PSD Ch6 Gamma (dB)',
    # 'Mean PSD Ch7 Delta (dB)', 'Mean PSD Ch7 Theta (dB)'
    # , 'Mean PSD Ch7 Alpha (dB)', 'Mean PSD Ch7 Beta (dB)'
    # , 'Mean PSD Ch7 Gamma (dB)',
    # 'Mean PSD Ch8 Delta (dB)', 'Mean PSD Ch8 Theta (dB)', 'Mean PSD Ch8 Alpha (dB)', 'Mean PSD Ch8 Beta (dB)', 'Mean PSD Ch8 Gamma (dB)',
    'Gender',
    'Age',
    'Group',
    'Has Peaks'
]

# Initialize an empty list to store dataframes
dfs = []

def traverse_directory(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)

                # Find the row index where 'Study Name' appears
                with open(file_path, 'r') as f:
                    study_name_index = next((i for i, line in enumerate(f) if line.startswith('Study Name')), None)

                if study_name_index is not None:
                    # Read the CSV file with the 'Study Name' row as the header
                    delimiter = detect_delimiter(file_path)
                    df = pd.read_csv(file_path, header=study_name_index, delimiter=delimiter)

                    # Reset index
                    df.reset_index(drop=True, inplace=True)

                    # Merge columns with the same name
                    df = merge_duplicate_columns(df)
                    # Exclude columns to be excluded
                    df = exclude_columns(df, columns_to_exclude)
                    # Append the dataframe to the list
                    dfs.append(df)
                else:
                    print(f"'Study Name' not found in {file_path}")

# Function to detect delimiter
def detect_delimiter(file_path):
    with open(file_path, 'r') as f:
        first_line = f.readline()
        if ';' in first_line:
            return ';'
        elif ',' in first_line:
            return ','
        else:
            raise ValueError("Delimiter not found in the first line of the file.")

# Function to merge duplicate columns
def merge_duplicate_columns(df):
    # Find all duplicate columns (ending with '.1', '.2', etc.)
    duplicates = set([col.rsplit('.', 1)[0] for col in df.columns if '.' in col])

    for dup_col in duplicates:
        # Find all columns with the same name (ignoring the suffix)
        same_cols = [c for c in df.columns if c.startswith(dup_col)]

        # For each row, find the first non-blank value from these columns
        combined_values = df[same_cols].apply(lambda row: next((val for val in row if pd.notna(val) and val != ''), None), axis=1)

        # Update the original column with combined values
        df[dup_col] = combined_values

        # Drop duplicate columns
        df.drop(columns=[c for c in same_cols if c != dup_col], inplace=True)

    return df


# Function to exclude columns if they exist in the dataframe
def exclude_columns(df, columns_to_exclude):
    columns_to_drop = [col for col in columns_to_exclude if col in df.columns]
    if columns_to_drop:
        df.drop(columns=columns_to_drop, inplace=True)
    return df

# Call the function to traverse the directory
traverse_directory(root_directory)

# Concatenate all dataframes in the list
sensors_df = pd.concat(dfs, ignore_index=True)

# Rename 'Respondent Name' to 'ID'
sensors_df.rename(columns={'Respondent Name': 'ID'}, inplace=True)

sensors_df.head()

,Study Name,ID,Type,Label,Start (ms),Duration (ms),Parent Stimulus,Comment,Signal Duration,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Mean Frontal Asymmetry Delta,Mean Frontal Asymmetry Theta,Mean Frontal Asymmetry Alpha,Mean Frontal Asymmetry Beta,Mean Frontal Asymmetry Gamma
0,FCP_lineare_180_new_smartphone,s141,Stimulus,Kilimangiaro_A,1070.0,383788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FCP_lineare_180_new_smartphone,s141,Annotation,binning_05s,1070.0,383788,Kilimangiaro_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FCP_lineare_180_new_smartphone,s141,Annotation (Instance),binning_05s (instance 001),1070.0,500,Kilimangiaro_A,Subdivision 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FCP_lineare_180_new_smartphone,s141,Annotation (Instance),binning_05s (instance 002),1570.0,500,Kilimangiaro_A,Subdivision 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FCP_lineare_180_new_smartphone,s141,Annotation (Instance),binning_05s (instance 003),2070.0,500,Kilimangiaro_A,Subdivision 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Drop a vision that was repeated because data were not recorded
sensors_df = sensors_df.loc[(sensors_df['ID'] != 's303') | (sensors_df['Study Name'] != 'FCP_lineare_180_Smartphone')]

In [ ]:
len(sensors_df['ID'].unique())

619

Study Name: partendo dalla colonna Studyname, otteniamo le colonne Device, Protocol e Length

In [ ]:

# Create columns Device, Protocol and Length
def transform_study_name(df):
    # Define functions to extract the new column values based on conditions
    def get_device(study_name):
        if 'TV' in study_name:
            return 'TV'
        elif 'Smartphone' in study_name or 'smartphone' in study_name:
            return 'Smartphone'
        elif 'Laptop' in study_name or 'laptop' in study_name:
            return 'Laptop'
        else:
            return np.nan

    def get_protocol(study_name):
        if 'lineare' in study_name:
            return 'Linear_' + str(get_length(study_name))
        elif 'VOD' in study_name:
            return 'VOD'
        elif 'YouTube' in study_name or 'YT' in study_name or 'Youtube' in study_name:
            return 'YouTube'
        else:
            return np.nan

    def get_length(study_name):
        if '60' in study_name:
            return 60
        elif '120' in study_name:
            return 120
        elif '180' in study_name:
            return 180
        else:
            return np.nan

    # Apply the functions to the 'Study Name' column to create new columns
    df['Device'] = df['Study Name'].apply(get_device)
    df['Protocol'] = df['Study Name'].apply(get_protocol)

    return df

# Transform the DataFrame
sensors_df = transform_study_name(sensors_df)

# Drop column Study Name
sensors_df.drop(columns=['Study Name'], inplace=True)

sensors_df

,ID,Type,Label,Start (ms),Duration (ms),Parent Stimulus,Comment,Signal Duration,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Mean Frontal Asymmetry Delta,Mean Frontal Asymmetry Theta,Mean Frontal Asymmetry Alpha,Mean Frontal Asymmetry Beta,Mean Frontal Asymmetry Gamma,Device,Protocol
0,s141,Stimulus,Kilimangiaro_A,1070.0,383788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Smartphone,Linear_180
1,s141,Annotation,binning_05s,1070.0,383788,Kilimangiaro_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Smartphone,Linear_180
2,s141,Annotation (Instance),binning_05s (instance 001),1070.0,500,Kilimangiaro_A,Subdivision 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Smartphone,Linear_180
3,s141,Annotation (Instance),binning_05s (instance 002),1570.0,500,Kilimangiaro_A,Subdivision 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Smartphone,Linear_180
4,s141,Annotation (Instance),binning_05s (instance 003),2070.0,500,Kilimangiaro_A,Subdivision 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Smartphone,Linear_180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1537583,s1338,Annotation (Instance),binning_05s (instance 1095),581328.0,500,SceneRecording-1,Subdivision 111,500.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.479,-0.855,-0.436,-0.079,-0.063,Smartphone,VOD
1537584,s1338,Annotation (Instance),binning_05s (instance 1096),581828.0,500,SceneRecording-1,Subdivision 112,500.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

### Standardizza l'ID

In [ ]:
# Convert 'ID' column to lowercase
sensors_df['ID'] = sensors_df['ID'].str.lower()

# Remove spaces from 'ID' column
sensors_df['ID'] = sensors_df['ID'].str.replace("\\s+", "", regex=True)

# Add an 's' in front of IDs that don't have it
sensors_df['ID'] = sensors_df['ID'].apply(lambda x: 's' + x if not x.startswith('s') else x)

sensors_df = sensors_df.iloc[sensors_df['ID'].str.extract(r'(\d+)').astype(float).fillna(-1).squeeze().argsort(kind='mergesort')]

# Drop some incomplete visions that have been replicated, and adjust some IDs
sensors_df = sensors_df[~((sensors_df['ID'] == "s168") & (sensors_df['Protocol'] == 'Linear_120'))]
sensors_df.loc[sensors_df['ID'] == 's168b', 'ID'] = 's168'
sensors_df = sensors_df[~((sensors_df['ID'] == "s257") & (sensors_df['Protocol'] == 'Linear_180'))]
sensors_df.loc[sensors_df['ID'] == 's257b', 'ID'] = 's257'
sensors_df = sensors_df[~((sensors_df['ID'] == "s263") & (sensors_df['Protocol'] == 'Linear_120'))]
sensors_df.loc[sensors_df['ID'] == 's263b', 'ID'] = 's263'
sensors_df.loc[sensors_df['ID'] == 's319b', 'ID'] = 's319'
sensors_df.loc[sensors_df['ID'] == 's1222_yt', 'Protocol'] = 'YouTube'
sensors_df.loc[sensors_df['ID'] == 's1222_yt', 'ID'] = 's1222'

sensors_df

,ID,Type,Label,Start (ms),Duration (ms),Parent Stimulus,Comment,Signal Duration,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Mean Frontal Asymmetry Delta,Mean Frontal Asymmetry Theta,Mean Frontal Asymmetry Alpha,Mean Frontal Asymmetry Beta,Mean Frontal Asymmetry Gamma,Device,Protocol
76238,s01,Stimulus,Segreti Foreste Incantate_A,854.0,400377,NaN,NaN,400377.0,0.0,0.0,NaN,14.873,6.393,2.156,-3.100,-9.086,14.637,6.746,1.956,-2.866,-8.934,14.023,6.285,1.443,-3.264,-9.636,13.601,5.990,0.896,-3.523,-9.990,13.402,5.980,0.600,-3.620,-10.037,12.910,5.183,-0.269,-4.157,-10.002,12.246,3.301,-2.134,-5.372,-10.149,NaN,NaN,NaN,NaN,NaN,14.397,6.235,0.904,-3.620,-9.526,-0.313,-0.333,-0.571,-0.306,-0.221,Laptop,Linear_60
76239,s01,Annotation,binning_05s,854.0,400377,Segreti Foreste Incantate_A,NaN,400377.0,0.0,0.0,NaN,14.873,6.393,2.156,-3.100,-9.086,14.637,6.746,1.956,-2.866,-8.934,14.023,6.285,1.443,-3.264,-9.636,13.601,5.990,0.896,-3.523,-9.990,13.402,5.980,0.600,-3.620,-10.037,12.910,5.183,-0.269,-4.157,-10.002,12.246,3.301,-2.134,-5.372,-10.149,NaN,NaN,NaN,NaN,NaN,14.397,6.235,0.904,-3.620,-9.526,-0.313,-0.333,-0.571,-0.306,-0.221,Laptop,Linear_60
76240,s01,Annotation (Instance),binning_05s (instance 001),854.0,500,Segreti Foreste Incantate_A,Subdivision 1,500.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_60
76241,s01,Annotation (Instance),binning_05s (instance 002),1354.0,500,Segreti Foreste Incantate_A,Subdivision 2,500.0,0.0,0.0,NaN,11.171,1.726,-0.187,-4.271,-9.734,13.018,3.181,0.602,-5.677,-12.241,11.904,1.483,-0.091,-6.043,-11.921,10.602,-0.240,-1.396,-5.189,-11.169,7.771,1.799,-1.449,-5.790,-10.848,10.409,0.351,-0.048,-5.597,-10.628,5.318,-0.342,-4.000,-6.309,-10.696,NaN,NaN,NaN,NaN,NaN,10.606,1.300,-0.732,-5.508,-10.964,-0.870,-0.346,-0.383,-0.144,0.098,Laptop,Linear_60
76242,s01,Annotation (Instance),binning_05s (instance 003),1854.0,500,Segreti Foreste Incantate_A,Subdivision 3,500.0,0.0,0.0,NaN,21.750,16.252,3.654,-0.417,-8.015,21.682,17.814,5.205,-0.576,-9.630,20.558,17.242,3.734,-1.504,-10.299,19.492,16.742,1.470,-2.139,-10.646,17.640,15.276,-0.194,-2.938,-11.124,14.539,13.120,-4.813,-3.149,-12.703,12.622,9.587,-6.426,-2.741,-12.220,NaN,NaN,NaN,NaN,NaN,19.362,15.800,1.929,-1.798,-10.408,-1.366,-0.900,-1.664,-0.491,-0.635,Laptop,Linear_60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1537583,s1338,Annotation (Instance),binning_05s (instance 1095),581328.0,500,SceneRecording-1,S

### Alcune osservazioni hanno solo YT o VOD, mettiamo la giusta suddivisione di Protocol

In [ ]:

sensors_df.loc[sensors_df['ID'] == 's1004', 'Protocol'] = 'YouTube'
sensors_df.loc[sensors_df['ID'] == 's1007', 'Protocol'] = 'YouTube'
sensors_df.loc[sensors_df['ID'] == 's1010', 'Protocol'] = 'YouTube'


# Trova le righe con ID 's1001' e Label 'SceneRecording-1'
filtered = sensors_df[(sensors_df['ID'] == 's1001') & (sensors_df['Label'] == "SceneRecording-1")]

# Identifica la seconda occorrenza
if len(filtered) > 1:  # Controlla che ci sia almeno una seconda occorrenza
    second_occurrence_index = filtered.index[1]  # Ottieni l'indice della seconda occorrenza

    # Trova tutte le righe successive alla seconda occorrenza e con ID 's1001'
    sensors_df.loc[
        (sensors_df.index >= second_occurrence_index) & (sensors_df['ID'] == 's1001'),
        'Protocol'
    ] = 'YouTube'




# Lista degli ID per cui applicare la modifica
ids_to_update = [ 's1007', 's1004', 's1010']

# Itera sugli ID e aggiorna la colonna 'Protocol' a partire dalla seconda occorrenza
for id_value in ids_to_update:
    # Filtra le righe con l'ID e Label 'SceneRecording-1'
    filtered = sensors_df[(sensors_df['ID'] == id_value) & (sensors_df['Label'] == "SceneRecording-1")]

    # Verifica che ci sia almeno una seconda occorrenza
    if len(filtered) > 1:
        # Ottieni l'indice della seconda occorrenza
        second_occurrence_index = filtered.index[1]

        # Modifica la colonna 'Protocol' a partire dalla seconda occorrenza
        sensors_df.loc[
            (sensors_df.index >= second_occurrence_index) & (sensors_df['ID'] == id_value),
            'Protocol'
        ] = 'VOD'

sensors_df[(sensors_df['ID'] == 's1001') & (sensors_df['Label'] == "SceneRecording-1")]

,ID,Type,Label,Start (ms),Duration (ms),Parent Stimulus,Comment,Signal Duration,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Mean Frontal Asymmetry Delta,Mean Frontal Asymmetry Theta,Mean Frontal Asymmetry Alpha,Mean Frontal Asymmetry Beta,Mean Frontal Asymmetry Gamma,Device,Protocol
643439,s1001,Stimulus,SceneRecording-1,168.0,795645,NaN,NaN,795645.0,6.0,0.45,0.0119,18.634,10.455,3.454,0.316,-2.595,17.319,7.855,2.729,-1.097,-4.892,15.936,7.267,2.258,-1.828,-5.696,15.253,7.120,1.906,-2.934,-7.446,15.690,7.207,1.890,-3.133,-7.87,16.337,7.251,1.911,-2.511,-6.260,19.085,11.166,2.091,-2.000,-4.210,NaN,NaN,NaN,NaN,NaN,19.574,11.746,2.884,-1.788,-4.921,0.158,0.238,-0.208,-0.416,-0.331,Laptop,VOD
649386,s1001,Stimulus,SceneRecording-1,224.0,775214,NaN,NaN,775214.0,6.0,0.46,0.0087,19.579,10.988,4.492,2.121,-0.335,19.683,9.668,4.224,0.649,-2.323,17.898,8.855,3.400,0.090,-3.603,17.046,8.533,2.821,-1.922,-5.818,16.971,8.237,2.505,-2.338,-6.60,17.515,8.303,2.155,-1.630,-5.036,18.375,11.001,2.331,-1.171,-3.017,NaN,NaN,NaN,NaN,NaN,19.577,11.966,3.743,-0.288,-3.021,-0.229,0.061,-0.370,-0.589,-0.520,Laptop,YouTube


In [ ]:
# Import file containing whether it was first or second vision
vision_df = pd.read_csv("Vision2.csv")
mask = vision_df['ID'].str.extract(r's(\d+)').astype(int)[0].between(400, 424) #serve per cambiare gli indici in Vision2
vision_df.loc[mask, 'ID'] = "s" + (vision_df['ID'].str.extract(r's(\d+)').astype(int)[0][mask] - 400 + 1000).astype(str)
sensors_df = sensors_df.merge(vision_df, on=['ID', 'Protocol'], how='left')
sensors_df

,ID,Type,Label,Start (ms),Duration (ms),Parent Stimulus,Comment,Signal Duration,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Mean Frontal Asymmetry Delta,Mean Frontal Asymmetry Theta,Mean Frontal Asymmetry Alpha,Mean Frontal Asymmetry Beta,Mean Frontal Asymmetry Gamma,Device,Protocol,Vision
0,s01,Stimulus,Segreti Foreste Incantate_A,854.0,400377,NaN,NaN,400377.0,0.0,0.0,NaN,14.873,6.393,2.156,-3.100,-9.086,14.637,6.746,1.956,-2.866,-8.934,14.023,6.285,1.443,-3.264,-9.636,13.601,5.990,0.896,-3.523,-9.990,13.402,5.980,0.600,-3.620,-10.037,12.910,5.183,-0.269,-4.157,-10.002,12.246,3.301,-2.134,-5.372,-10.149,NaN,NaN,NaN,NaN,NaN,14.397,6.235,0.904,-3.620,-9.526,-0.313,-0.333,-0.571,-0.306,-0.221,Laptop,Linear_60,NaN
1,s01,Annotation,binning_05s,854.0,400377,Segreti Foreste Incantate_A,NaN,400377.0,0.0,0.0,NaN,14.873,6.393,2.156,-3.100,-9.086,14.637,6.746,1.956,-2.866,-8.934,14.023,6.285,1.443,-3.264,-9.636,13.601,5.990,0.896,-3.523,-9.990,13.402,5.980,0.600,-3.620,-10.037,12.910,5.183,-0.269,-4.157,-10.002,12.246,3.301,-2.134,-5.372,-10.149,NaN,NaN,NaN,NaN,NaN,14.397,6.235,0.904,-3.620,-9.526,-0.313,-0.333,-0.571,-0.306,-0.221,Laptop,Linear_60,NaN
2,s01,Annotation (Instance),binning_05s (instance 001),854.0,500,Segreti Foreste Incantate_A,Subdivision 1,500.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_60,NaN
3,s01,Annotation (Instance),binning_05s (instance 002),1354.0,500,Segreti Foreste Incantate_A,Subdivision 2,500.0,0.0,0.0,NaN,11.171,1.726,-0.187,-4.271,-9.734,13.018,3.181,0.602,-5.677,-12.241,11.904,1.483,-0.091,-6.043,-11.921,10.602,-0.240,-1.396,-5.189,-11.169,7.771,1.799,-1.449,-5.790,-10.848,10.409,0.351,-0.048,-5.597,-10.628,5.318,-0.342,-4.000,-6.309,-10.696,NaN,NaN,NaN,NaN,NaN,10.606,1.300,-0.732,-5.508,-10.964,-0.870,-0.346,-0.383,-0.144,0.098,Laptop,Linear_60,NaN
4,s01,Annotation (Instance),binning_05s (instance 003),1854.0,500,Segreti Foreste Incantate_A,Subdivision 3,500.0,0.0,0.0,NaN,21.750,16.252,3.654,-0.417,-8.015,21.682,17.814,5.205,-0.576,-9.630,20.558,17.242,3.734,-1.504,-10.299,19.492,16.742,1.470,-2.139,-10.646,17.640,15.276,-0.194,-2.938,-11.124,14.539,13.120,-4.813,-3.149,-12.703,12.622,9.587,-6.426,-2.741,-12.220,NaN,NaN,NaN,NaN,NaN,19.362,15.800,1.929,-1.798,-10.408,-1.366,-0.900,-1.664,-0.491,-0.635,Laptop,Linear_60,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533097,s1338,Annotation (Instance),binning_05s (instance 1095),581328.0,500,SceneRe

# Change IDs of specific tests (FACCIAMOLO PER TUTTI I DEVICES)

## 1. Verifico TV

In [ ]:
sensors_df_TV = sensors_df[sensors_df["Device"] == "TV"]
sensors_df_TV['ID'].unique()

array(['s25', 's26', 's27', 's28', 's29', 's30', 's31', 's32', 's33',
       's34', 's35', 's36', 's38', 's40', 's42', 's44', 's46', 's48',
       's50', 's52', 's54', 's56', 's58', 's60', 's62', 's64', 's66',
       's68', 's70', 's72', 's74', 's76', 's78', 's80', 's82', 's84',
       's86', 's88', 's90', 's92', 's94', 's96', 's98', 's100', 's102',
       's104', 's106', 's108', 's110', 's112', 's114', 's116', 's118',
       's120', 's122', 's125', 's128', 's131', 's134', 's137', 's140',
       's143', 's146', 's149', 's152', 's155', 's158', 's161', 's164',
       's167', 's170', 's173', 's176', 's179', 's182', 's187', 's188',
       's191', 's194', 's197', 's200', 's203', 's206', 's216', 's218',
       's221', 's223', 's226', 's229', 's232', 's235', 's238', 's241',
       's244', 's247', 's250', 's253', 's256', 's260', 's264', 's1000',
       's1003', 's1006', 's1009', 's1012', 's1015', 's1021', 's1024',
       's1027', 's1030', 's1035', 's1039', 's1042', 's1045', 's1048',
       's1

## 2. Verifico Laptop

In [ ]:
sensors_df_Laptop = sensors_df[sensors_df["Device"] == "TV"]
sensors_df_Laptop['ID'].unique()

array(['s25', 's26', 's27', 's28', 's29', 's30', 's31', 's32', 's33',
       's34', 's35', 's36', 's38', 's40', 's42', 's44', 's46', 's48',
       's50', 's52', 's54', 's56', 's58', 's60', 's62', 's64', 's66',
       's68', 's70', 's72', 's74', 's76', 's78', 's80', 's82', 's84',
       's86', 's88', 's90', 's92', 's94', 's96', 's98', 's100', 's102',
       's104', 's106', 's108', 's110', 's112', 's114', 's116', 's118',
       's120', 's122', 's125', 's128', 's131', 's134', 's137', 's140',
       's143', 's146', 's149', 's152', 's155', 's158', 's161', 's164',
       's167', 's170', 's173', 's176', 's179', 's182', 's187', 's188',
       's191', 's194', 's197', 's200', 's203', 's206', 's216', 's218',
       's221', 's223', 's226', 's229', 's232', 's235', 's238', 's241',
       's244', 's247', 's250', 's253', 's256', 's260', 's264', 's1000',
       's1003', 's1006', 's1009', 's1012', 's1015', 's1021', 's1024',
       's1027', 's1030', 's1035', 's1039', 's1042', 's1045', 's1048',
       's1

## 3. verifico smartphone


In [ ]:
# Now you can change the names of some IDs
sensors_df.loc[sensors_df['ID'] == 's213b', 'ID'] = 's213'
sensors_df.loc[sensors_df['ID'] == 's219b', 'ID'] = 's219'
sensors_df.loc[sensors_df['ID'] == 's275_60', 'ID'] = 's275'
sensors_df.loc[sensors_df['ID'] == 's280_60', 'ID'] = 's280'
sensors_df.loc[sensors_df['ID'] == 's290b', 'ID'] = 's290'
sensors_df.loc[sensors_df['ID'] == 's294b', 'ID'] = 's294'

vision_df.loc[vision_df['ID'] == 's213b', 'ID'] = 's213'
vision_df.loc[vision_df['ID'] == 's219b', 'ID'] = 's219'
vision_df.loc[vision_df['ID'] == 's275_60', 'ID'] = 's275'
vision_df.loc[vision_df['ID'] == 's280_60', 'ID'] = 's280'
vision_df.loc[vision_df['ID'] == 's290b', 'ID'] = 's290'
vision_df.loc[vision_df['ID'] == 's294b', 'ID'] = 's294'

### Some visions have only one row and not all the observations, so we eliminate them

#### Laptop

In [ ]:
# There are some visions that only have one aggregate row and not all the bins
grouped_counts_Laptop = sensors_df_Laptop.groupby(['ID', 'Protocol']).size().reset_index(name='Count')

filtered_counts_Laptop = grouped_counts_Laptop[grouped_counts_Laptop['Count'] < 10]


# Get rid of them
sensors_df = sensors_df[~((sensors_df['ID'] == "s1328	") & (sensors_df['Protocol'] == 'VOD'))]
sensors_df = sensors_df[~((sensors_df['ID'] == "s1332") & (sensors_df['Protocol'] == 'VOD'))]


#### Smartphone

In [ ]:
sensors_df

,ID,Type,Label,Start (ms),Duration (ms),Parent Stimulus,Comment,Signal Duration,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Mean Frontal Asymmetry Delta,Mean Frontal Asymmetry Theta,Mean Frontal Asymmetry Alpha,Mean Frontal Asymmetry Beta,Mean Frontal Asymmetry Gamma,Device,Protocol,Vision
0,s01,Stimulus,Segreti Foreste Incantate_A,854.0,400377,NaN,NaN,400377.0,0.0,0.0,NaN,14.873,6.393,2.156,-3.100,-9.086,14.637,6.746,1.956,-2.866,-8.934,14.023,6.285,1.443,-3.264,-9.636,13.601,5.990,0.896,-3.523,-9.990,13.402,5.980,0.600,-3.620,-10.037,12.910,5.183,-0.269,-4.157,-10.002,12.246,3.301,-2.134,-5.372,-10.149,NaN,NaN,NaN,NaN,NaN,14.397,6.235,0.904,-3.620,-9.526,-0.313,-0.333,-0.571,-0.306,-0.221,Laptop,Linear_60,NaN
1,s01,Annotation,binning_05s,854.0,400377,Segreti Foreste Incantate_A,NaN,400377.0,0.0,0.0,NaN,14.873,6.393,2.156,-3.100,-9.086,14.637,6.746,1.956,-2.866,-8.934,14.023,6.285,1.443,-3.264,-9.636,13.601,5.990,0.896,-3.523,-9.990,13.402,5.980,0.600,-3.620,-10.037,12.910,5.183,-0.269,-4.157,-10.002,12.246,3.301,-2.134,-5.372,-10.149,NaN,NaN,NaN,NaN,NaN,14.397,6.235,0.904,-3.620,-9.526,-0.313,-0.333,-0.571,-0.306,-0.221,Laptop,Linear_60,NaN
2,s01,Annotation (Instance),binning_05s (instance 001),854.0,500,Segreti Foreste Incantate_A,Subdivision 1,500.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_60,NaN
3,s01,Annotation (Instance),binning_05s (instance 002),1354.0,500,Segreti Foreste Incantate_A,Subdivision 2,500.0,0.0,0.0,NaN,11.171,1.726,-0.187,-4.271,-9.734,13.018,3.181,0.602,-5.677,-12.241,11.904,1.483,-0.091,-6.043,-11.921,10.602,-0.240,-1.396,-5.189,-11.169,7.771,1.799,-1.449,-5.790,-10.848,10.409,0.351,-0.048,-5.597,-10.628,5.318,-0.342,-4.000,-6.309,-10.696,NaN,NaN,NaN,NaN,NaN,10.606,1.300,-0.732,-5.508,-10.964,-0.870,-0.346,-0.383,-0.144,0.098,Laptop,Linear_60,NaN
4,s01,Annotation (Instance),binning_05s (instance 003),1854.0,500,Segreti Foreste Incantate_A,Subdivision 3,500.0,0.0,0.0,NaN,21.750,16.252,3.654,-0.417,-8.015,21.682,17.814,5.205,-0.576,-9.630,20.558,17.242,3.734,-1.504,-10.299,19.492,16.742,1.470,-2.139,-10.646,17.640,15.276,-0.194,-2.938,-11.124,14.539,13.120,-4.813,-3.149,-12.703,12.622,9.587,-6.426,-2.741,-12.220,NaN,NaN,NaN,NaN,NaN,19.362,15.800,1.929,-1.798,-10.408,-1.366,-0.900,-1.664,-0.491,-0.635,Laptop,Linear_60,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533097,s1338,Annotation (Instance),binning_05s (instance 1095),581328.0,500,SceneRe

In [ ]:
# There are some visions that only have one aggregate row and not all the bins
grouped_counts = sensors_df.groupby(['ID', 'Protocol']).size().reset_index(name='Count')

filtered_counts = grouped_counts[grouped_counts['Count'] < 10]

# Get rid of them
sensors_df = sensors_df[~((sensors_df['ID'] == "s1222") & (sensors_df['Protocol'] == 'VOD'))]
sensors_df = sensors_df[~((sensors_df['ID'] == "s1225") & (sensors_df['Protocol'] == 'YouTube'))]
sensors_df = sensors_df[~((sensors_df['ID'] == "s1256") & (sensors_df['Protocol'] == 'YouTube'))]
sensors_df = sensors_df[~((sensors_df['ID'] == "s1285") & (sensors_df['Protocol'] == 'VOD'))]
sensors_df = sensors_df[~((sensors_df['ID'] == "s1285") & (sensors_df['Protocol'] == 'YouTube'))]
sensors_df = sensors_df[~((sensors_df['ID'] == "s1294") & (sensors_df['Protocol'] == 'YouTube'))]

# There are some visions with extremely weird data. Let's remove them
sensors_df = sensors_df[~((sensors_df['ID'] == "s227") & (sensors_df['Vision'] == 1))]
sensors_df = sensors_df[~((sensors_df['ID'] == "s299") & (sensors_df['Vision'] == 2))]

filtered_counts

,ID,Protocol,Count
471,s1222,VOD,1
478,s1225,YouTube,1
542,s1256,YouTube,1
599,s1285,VOD,1
600,s1285,YouTube,1
618,s1294,YouTube,1


### Import scheduling_df to know which video each participant watched

In [ ]:

scheduling_df = pd.read_excel("FCP_esperimenti_20240416.xlsx")
scheduling_df.loc[scheduling_df['ID partecipante'].between(400, 424), 'ID partecipante'] = scheduling_df['ID partecipante'] - 400 + 1000
scheduling_df = scheduling_df.dropna(subset=['ID partecipante'])
scheduling_df['ID'] = 's' + scheduling_df['ID partecipante'].astype(int).astype(str)


vision_df = vision_df.merge(scheduling_df[["ID", "Video VOD", "Video YT"]], on = "ID", how = "left")
vision_df['Contenuto'] = np.where(vision_df['Protocol'] == 'VOD', vision_df['Video VOD'], vision_df['Video YT'])
vision_df = vision_df.drop(columns=['Video VOD', 'Video YT'])

# Import the ads_scheme to know which ads each participant watched
ads_scheme_df = pd.read_excel("Schema adv YT e VOD.xlsx")
vision_df = vision_df.merge(ads_scheme_df, on = "Contenuto", how = "left")
vision_df

,ID,Protocol,Vision,Contenuto,Emittente/Canale,ADV_1,ADV_2,ADV_3,ADV_4
0,s1,Linear_60,1,NaN,NaN,NaN,NaN,NaN,NaN
1,s2,Linear_120,1,NaN,NaN,NaN,NaN,NaN,NaN
2,s3,Linear_60,1,NaN,NaN,NaN,NaN,NaN,NaN
3,s4,Linear_120,1,NaN,NaN,NaN,NaN,NaN,NaN
4,s5,Linear_180,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1181,s1321,VOD,2,Schindler - La vera stoeia,SKY,Fregola,Amazon,Chanel,Fastweb
1182,s1322,Youtube,2,Da operaio a IMPRENDITORE partendo da zero (come ho fatto),Michele Molteni,Ikea,Fregola,Mutti,Chanel
1183,s1323,VOD,2,Freedom,MEDIASET,Fregola,Chanel,Fastweb,Coca cola
1184,s1324,VOD,2,Di Martedì (Luca e Paolo),LA7,Barilla,Alfa Romeo,Fregola,Mutti


In [ ]:
sensors_df=sensors_df[~(sensors_df['ID']=='s1273')]

### Eliminate the rows that summarize the whole vision (NB: HERE WE ELIMINATE ALSO THE ROWS THAT LAST LESS THAN 5sec. THIS REPRESENT THE LAST 5 SEC OF THE CONTENT PEOPLE WATCHED, SO IT SHOULD NOT INFLUENCE THE ANALYSIS OF THE ADV. MODIFY IT IF YOU THINK IT'S NEEDED)

In [ ]:
# Clean the data, only keeping the timestamps

# Only keep the 5secs bins
sensors_df = sensors_df[sensors_df['Duration (ms)'] == 500]

# Rename column
sensors_df.rename(columns={'Parent Stimulus': 'Content'}, inplace=True)

# Get rid of unnecessary columns
columns_to_drop = ['Type', 'Label', 'Signal Duration']
sensors_df.drop(columns=columns_to_drop, inplace=True)

sensors_df.head(20)

<ipython-input-21-e609ec633f48>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sensors_df.rename(columns={'Parent Stimulus': 'Content'}, inplace=True)


,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Mean Frontal Asymmetry Delta,Mean Frontal Asymmetry Theta,Mean Frontal Asymmetry Alpha,Mean Frontal Asymmetry Beta,Mean Frontal Asymmetry Gamma,Device,Protocol,Vision
2,s01,854.0,500,Segreti Foreste Incantate_A,Subdivision 1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_60,NaN
3,s01,1354.0,500,Segreti Foreste Incantate_A,Subdivision 2,0.0,0.0,NaN,11.171,1.726,-0.187,-4.271,-9.734,13.018,3.181,0.602,-5.677,-12.241,11.904,1.483,-0.091,-6.043,-11.921,10.602,-0.240,-1.396,-5.189,-11.169,7.771,1.799,-1.449,-5.790,-10.848,10.409,0.351,-0.048,-5.597,-10.628,5.318,-0.342,-4.000,-6.309,-10.696,NaN,NaN,NaN,NaN,NaN,10.606,1.300,-0.732,-5.508,-10.964,-0.870,-0.346,-0.383,-0.144,0.098,Laptop,Linear_60,NaN
4,s01,1854.0,500,Segreti Foreste Incantate_A,Subdivision 3,0.0,0.0,NaN,21.750,16.252,3.654,-0.417,-8.015,21.682,17.814,5.205,-0.576,-9.630,20.558,17.242,3.734,-1.504,-10.299,19.492,16.742,1.470,-2.139,-10.646,17.640,15.276,-0.194,-2.938,-11.124,14.539,13.120,-4.813,-3.149,-12.703,12.622,9.587,-6.426,-2.741,-12.220,NaN,NaN,NaN,NaN,NaN,19.362,15.800,1.929,-1.798,-10.408,-1.366,-0.900,-1.664,-0.491,-0.635,Laptop,Linear_60,NaN
5,s01,2354.0,500,Segreti Foreste Incantate_A,Subdivision 4,0.0,0.0,NaN,17.443,12.548,0.244,1.492,-1.880,20.064,14.474,2.829,-0.091,-5.419,20.318,13.933,2.895,-0.542,-7.154,19.629,13.667,0.738,-0.601,-6.423,19.177,12.677,-0.041,-1.038,-6.129,17.301,10.419,-1.302,-1.846,-5.359,15.009,6.399,-4.418,-2.036,-3.566,NaN,NaN,NaN,NaN,NaN,18.747,12.617,0.688,-0.520,-4.775,-0.453,-0.735,-0.855,-0.459,-0.146,Laptop,Linear_60,NaN
6,s01,2854.0,500,Segreti Foreste Incantate_A,Subdivision 5,0.0,0.0,NaN,20.454,7.625,4.933,1.457,3.544,22.409,4.042,3.174,-3.161,-3.209,20.544,2.119,1.969,-3.669,-6.575,20.575,0.663,-0.023,-4.256,-8.514,19.028,4.141,-3.579,-3.249,-7.721,14.202,4.471,-2.328,-0.227,-5.113,13.952,4.146,-3.179,0.700,-2.206,NaN,NaN,NaN,NaN,NaN,19.636,4.362,1.217,-1.222,-2.240,-1.112,-0.219,-1.503,0.117,-1.009,Laptop,Linear_60,NaN
7,s01,3354.0,500,Segreti Foreste Incantate_A,Subdivision 6,0.0,0.0,NaN,17.162,6.270,-1.920,-0.218,-3.679,15.779,11.008,3.449,0.756,-6.931,15.167,10.105,3.973,0.745,-8.443,15.027,10.663,3.460,1.111,-8.699,16.589,11.394,4.458,2.151,-8.049,17.151,10.768,3.665,1.250,-7.711,12.299,6.424,-0.350,-0.957,-6.730,NaN,NaN,NaN,NaN,NaN,15.846,9.928,2.887,0.790,-6.844,-0.070,0.106,0.121,0.126,-0.363,Laptop,Linear_60,NaN
8,s01,3854.0,500,Segreti Foreste Incantate_A,Subdivision 7,0.0,0.0,NaN,20.913,9.622,5.244,1.342,-4.784,21.410,12.178,7.526,3.083,-7.819,20.602,11.682,6.909,2.966,-7.915,20.22

### La colonna Content dovrebbe indicare se l'oservazione riguarda l'adv o il contenuto. Per il lineare questo è correttamente riportato, ma per il non-lineare no, quindi qui lo aggiorniamo in base a : ad1, ad2, content, ad3, ad4, content


Qui avviene un merge ma non è left (da mettere il left is null)

In [ ]:
# Add a column telling which position the rows belongs to (positions from 1 to 6) inside the vision. It will be removed later
position_numbers = (sensors_df['Comment'] == 'Subdivision 1').groupby([sensors_df['ID'], sensors_df['Vision']]).cumsum()
sensors_df['Position'] = position_numbers

# Merge with vision_df to know what content and ads the participant watched
sensors_df = pd.merge(sensors_df, vision_df.drop('Protocol', axis = 1), on=['ID', 'Vision'])

# Define a function to get 'Content' based on the block
def get_content(row):

    # For the linear, do not change the content
    if row["Protocol"].startswith("Linear"):
        return row["Content"]

    # For the non linear, change the content based on the position
    position_num = row['Position']
    if position_num == 1:
        return row['ADV_1']
    elif position_num == 2:
        return row['ADV_2']
    elif position_num == 3:
        return row['Contenuto']
    elif position_num == 4:
        return row['ADV_3']
    elif position_num == 5:
        return row['ADV_4']
    elif position_num == 6:
        return row['Contenuto']

# Apply the function to create 'Content' column
sensors_df['Content'] = sensors_df.apply(get_content, axis=1)

# Drop unnecessary columns
sensors_df.drop(['Contenuto', 'Emittente/Canale', 'ADV_1', 'ADV_2', 'ADV_3', 'ADV_4'], axis=1, inplace=True)

sensors_df.head(500)

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Mean Frontal Asymmetry Delta,Mean Frontal Asymmetry Theta,Mean Frontal Asymmetry Alpha,Mean Frontal Asymmetry Beta,Mean Frontal Asymmetry Gamma,Device,Protocol,Vision,Position
0,s10,876.0,500,Succession_A,Subdivision 1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_120,2.0,1.0
1,s10,1376.0,500,Succession_A,Subdivision 2,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_120,2.0,1.0
2,s10,1876.0,500,Succession_A,Subdivision 3,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.286,12.343,6.090,-2.374,-5.569,21.562,11.696,5.716,-1.677,-4.655,21.971,11.083,4.989,0.585,-3.712,21.697,9.474,1.475,2.274,-1.991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.636,11.273,4.897,0.098,-3.774,0.127,-0.458,-0.578,0.895,0.645,Laptop,Linear_120,2.0,1.0
3,s10,2376.0,500,Succession_A,Subdivision 4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.233,14.107,8.446,0.301,-2.905,25.491,14.118,8.082,0.321,-3.433,25.422,12.405,7.244,3.230,0.037,23.575,12.439,5.061,4.270,2.038,28.524,-3.740,-3.058,-6.990,-11.456,NaN,NaN,NaN,NaN,NaN,26.154,12.401,6.518,1.538,-1.373,0.022,-0.781,-0.846,0.437,0.556,Laptop,Linear_120,2.0,1.0
4,s10,2876.0,500,Succession_A,Subdivision 5,0.0,0.0,NaN,11.957,7.213,2.870,-6.401,-9.379,17.539,12.394,8.701,-3.467,-8.990,21.439,15.421,10.545,-2.395,-9.617,22.348,16.538,11.707,-1.191,-10.052,21.985,16.750,12.854,-0.348,-4.477,20.567,15.426,10.937,1.428,-4.521,19.777,3.603,-4.179,-6.942,-9.250,NaN,NaN,NaN,NaN,NaN,20.302,14.268,9.848,-1.890,-7.385,0.550,0.386,0.439,0.695,0.860,Laptop,Linear_120,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,s10,248376.0,500,Succession_A,Subdivision 496,0.0,0.0,NaN,18.074,13.804,8.531,0.906,-9.912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.225,19.981,11.973,3.539,-9.659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.372,17.909,10.584,2.419,-9.784,1.186,1.422,0.792,0.606,0.058,Laptop,Linear_120,2.0,1.0
496,s10,248876.0,500,Succession_A,Subdivision 497,0.0,0.0,NaN,7.403,2.587,-5.317,-4.131,-9.012,14.353,5.463,-1.994,-3.004,-6.163,13.166,5.318,-2.744,-2.530,-4.861,14.494,4.336,-0.261,-2.089,-7.842,13.891,4.042,-3.605,-1.871,-8.883,13.475,3.749,-4.613,-3

### Make the names of the ads uniform (att: questo non riguarda i questionari, ma il dataset sui sensori)

In [ ]:
sensors_df['Content'] = sensors_df['Content'].str.replace('00_', '') #Rimuove il prefisso "00_" da tutte le stringhe presenti nella colonna Content
sensors_df.loc[sensors_df["Content"] == "MUTTI1", "Content"] = "MUTTI"
sensors_df.loc[sensors_df["Content"] == "IKEA ITALIA", "Content"] = "IKEA"
sensors_df.loc[sensors_df["Content"] == "ALFA TONALE", "Content"] = "ALFA ROMEO"
sensors_df.loc[sensors_df["Content"] == "Ikea", "Content"] = "IKEA"
sensors_df.loc[sensors_df["Content"] == "Fregola", "Content"] = "FREGOLA"
sensors_df.loc[sensors_df["Content"] == "Chanel", "Content"] = "CHANEL"
sensors_df.loc[sensors_df["Content"] == "Coca cola", "Content"] = "COCA COLA"
sensors_df.loc[sensors_df["Content"] == "Barilla", "Content"] = "BARILLA"
sensors_df.loc[sensors_df["Content"] == "Alfa Romeo", "Content"] = "ALFA ROMEO"
sensors_df.loc[sensors_df["Content"] == "Orasì", "Content"] = "ORASI"
sensors_df.loc[sensors_df["Content"] == "Fastweb", "Content"] = "FASTWEB"
sensors_df.loc[sensors_df["Content"] == "Coop", "Content"] = "COOP"
sensors_df.loc[sensors_df["Content"] == "Lete", "Content"] = "LETE"
sensors_df.loc[sensors_df["Content"] == "Multicentrum", "Content"] = "MULTICENTRUM"
sensors_df.loc[sensors_df["Content"] == "Mutti", "Content"] = "MUTTI"
sensors_df.loc[sensors_df["Content"] == "Amazon", "Content"] = "AMAZON"

In [ ]:
len(sensors_df['ID'].unique())

577

### Qua creiamo la colonna Type che distingue tra content_A, content_B e ad

In [ ]:
# Add the Type of content (content_A, ad, content_B)
def determine_type(row):
    if row['Content'] in ['FREGOLA', 'LETE', 'COOP', 'FASTWEB', 'MULTICENTRUM', 'ORASI', 'MUTTI', 'BARILLA',
                          'COCA COLA', 'IKEA', 'AMAZON', 'CHANEL', 'ALFA ROMEO']:
        return 'ad'
    elif row['Protocol'].startswith('Linear'):
        if row['Position'] == 1:
            return 'content_A'
        else:
            return 'content_B'
    else:
        if row['Position'] == 3:
            return 'content_A'
        elif row['Position'] == 6:
            return 'content_B'

# Assuming 'Position' column exists in your DataFrame
sensors_df['Type'] = sensors_df.apply(determine_type, axis=1)

sensors_df.head()

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Mean Frontal Asymmetry Delta,Mean Frontal Asymmetry Theta,Mean Frontal Asymmetry Alpha,Mean Frontal Asymmetry Beta,Mean Frontal Asymmetry Gamma,Device,Protocol,Vision,Position,Type
0,s10,876.0,500,Succession_A,Subdivision 1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_120,2.0,1.0,content_A
1,s10,1376.0,500,Succession_A,Subdivision 2,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_120,2.0,1.0,content_A
2,s10,1876.0,500,Succession_A,Subdivision 3,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.286,12.343,6.090,-2.374,-5.569,21.562,11.696,5.716,-1.677,-4.655,21.971,11.083,4.989,0.585,-3.712,21.697,9.474,1.475,2.274,-1.991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.636,11.273,4.897,0.098,-3.774,0.127,-0.458,-0.578,0.895,0.645,Laptop,Linear_120,2.0,1.0,content_A
3,s10,2376.0,500,Succession_A,Subdivision 4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.233,14.107,8.446,0.301,-2.905,25.491,14.118,8.082,0.321,-3.433,25.422,12.405,7.244,3.230,0.037,23.575,12.439,5.061,4.270,2.038,28.524,-3.740,-3.058,-6.990,-11.456,NaN,NaN,NaN,NaN,NaN,26.154,12.401,6.518,1.538,-1.373,0.022,-0.781,-0.846,0.437,0.556,Laptop,Linear_120,2.0,1.0,content_A
4,s10,2876.0,500,Succession_A,Subdivision 5,0.0,0.0,NaN,11.957,7.213,2.87,-6.401,-9.379,17.539,12.394,8.701,-3.467,-8.99,21.439,15.421,10.545,-2.395,-9.617,22.348,16.538,11.707,-1.191,-10.052,21.985,16.750,12.854,-0.348,-4.477,20.567,15.426,10.937,1.428,-4.521,19.777,3.603,-4.179,-6.942,-9.250,NaN,NaN,NaN,NaN,NaN,20.302,14.268,9.848,-1.890,-7.385,0.550,0.386,0.439,0.695,0.860,Laptop,Linear_120,2.0,1.0,content_A


### For those visions that got blocked in the middle of the ads, get rid of the experiment

In [ ]:
# For those visions that got blocked in the middle of the ads, get rid of the experiment
groups_with_content_b = sensors_df.groupby(['ID', 'Vision']).filter(lambda x: (x['Type'] == 'content_B').sum() > 0) #quelli che non hanno obs per il content_B vengono eliminate

# Filter the original DataFrame to include only these groups
sensors_df = sensors_df[sensors_df.set_index(['ID', 'Vision']).index.isin(groups_with_content_b.set_index(['ID', 'Vision']).index)]

sensors_df

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Mean Frontal Asymmetry Delta,Mean Frontal Asymmetry Theta,Mean Frontal Asymmetry Alpha,Mean Frontal Asymmetry Beta,Mean Frontal Asymmetry Gamma,Device,Protocol,Vision,Position,Type
0,s10,876.0,500,Succession_A,Subdivision 1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_120,2.0,1.0,content_A
1,s10,1376.0,500,Succession_A,Subdivision 2,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_120,2.0,1.0,content_A
2,s10,1876.0,500,Succession_A,Subdivision 3,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.286,12.343,6.090,-2.374,-5.569,21.562,11.696,5.716,-1.677,-4.655,21.971,11.083,4.989,0.585,-3.712,21.697,9.474,1.475,2.274,-1.991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.636,11.273,4.897,0.098,-3.774,0.127,-0.458,-0.578,0.895,0.645,Laptop,Linear_120,2.0,1.0,content_A
3,s10,2376.0,500,Succession_A,Subdivision 4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.233,14.107,8.446,0.301,-2.905,25.491,14.118,8.082,0.321,-3.433,25.422,12.405,7.244,3.230,0.037,23.575,12.439,5.061,4.270,2.038,28.524,-3.740,-3.058,-6.990,-11.456,NaN,NaN,NaN,NaN,NaN,26.154,12.401,6.518,1.538,-1.373,0.022,-0.781,-0.846,0.437,0.556,Laptop,Linear_120,2.0,1.0,content_A
4,s10,2876.0,500,Succession_A,Subdivision 5,0.0,0.0,NaN,11.957,7.213,2.87,-6.401,-9.379,17.539,12.394,8.701,-3.467,-8.99,21.439,15.421,10.545,-2.395,-9.617,22.348,16.538,11.707,-1.191,-10.052,21.985,16.750,12.854,-0.348,-4.477,20.567,15.426,10.937,1.428,-4.521,19.777,3.603,-4.179,-6.942,-9.250,NaN,NaN,NaN,NaN,NaN,20.302,14.268,9.848,-1.890,-7.385,0.550,0.386,0.439,0.695,0.860,Laptop,Linear_120,2.0,1.0,content_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1273232,s1338,580828.0,500,Sky calcio club,Subdivision 110,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.106,-2.715,-0.204,-0.201,-0.292,Smartphone,VOD,2.0,6.0,content_B
1273233,s1338,581328.0,500,Sky calcio club,Subdivision 111,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

# Calculate indicators

In [ ]:
sensors_df2=sensors_df.copy()

In [ ]:
# Transform columns from decibel to linear
def convert_to_float(value):
    try:
        # If value contains comma, replace with dot
        if ',' in str(value):
            return float(value.replace(',', '.'))
        else:
            return float(value)
    except (ValueError, TypeError):
        return float('nan')

# Compute Engagement
sensors_df['Engagement'] = 10**(sensors_df['Mean PSD Electrode Cluster Average Beta (dB)'].apply(convert_to_float) / 10) / (10**(sensors_df['Mean PSD Electrode Cluster Average Alpha (dB)'].apply(convert_to_float) / 10) + 10**(sensors_df['Mean PSD Electrode Cluster Average Theta (dB)'].apply(convert_to_float) / 10))
sensors_df['Engagement(1/alpha)'] = 10*np.log10(   (1/    (10**(sensors_df['Mean PSD Electrode Cluster Average Alpha (dB)'].apply(convert_to_float)     / 10)))      )



# Calculate Frontal Alpha Asymmetry (FAA)
sensors_df['FAA'] = sensors_df['Mean PSD Ch7 Alpha (dB)'] - sensors_df['Mean PSD Ch1 Alpha (dB)']

# Compute Pleasantness Index (weighted bands approach)
w_alpha = 0.7  # Weight for alpha band    --> TROVA ARTICOLI CHE NE PARLANO
w_beta = 0.3   # Weight for beta band

sensors_df['Pleasantness Index'] = (
    w_alpha * (sensors_df['Mean PSD Ch7 Alpha (dB)'] - sensors_df['Mean PSD Ch1 Alpha (dB)']) +
    w_beta * (sensors_df['Mean PSD Ch7 Beta (dB)'] - sensors_df['Mean PSD Ch1 Beta (dB)'])
)
# sensors_df.drop(['Mean PSD Ch1 Alpha (dB)','Mean PSD Ch7 Alpha (dB)', 'Mean PSD Ch7 Beta (dB)','Mean PSD Ch1 Beta (dB)'], inplace=True)
sensors_df[['FAA', 'Pleasantness Index']]



# Compute Pleasanteness
sensors_df['Pleasantness'] = sensors_df['Mean Frontal Asymmetry Alpha'].apply(convert_to_float)

#change name to the asymmetries
sensors_df.rename(columns={
    'Mean Frontal Asymmetry Delta': 'Asymmetry Delta',
    'Mean Frontal Asymmetry Theta': 'Asymmetry Theta',
    'Mean Frontal Asymmetry Beta': 'Asymmetry Beta',
    'Mean Frontal Asymmetry Gamma': 'Asymmetry Gamma'
}, inplace=True)



# Compute Attention
sensors_df['Attention'] = sensors_df['Mean PSD Electrode Cluster Average Theta (dB)'].apply(convert_to_float)
sensors_df['Beta'] = sensors_df['Mean PSD Electrode Cluster Average Beta (dB)'].apply(convert_to_float)
sensors_df['Alpha'] = sensors_df['Mean PSD Electrode Cluster Average Alpha (dB)'].apply(convert_to_float)
sensors_df['Gamma'] = sensors_df['Mean PSD Electrode Cluster Average Gamma (dB)'].apply(convert_to_float)
sensors_df['Delta'] = sensors_df['Mean PSD Electrode Cluster Average Delta (dB)'].apply(convert_to_float)


# Drop useless columns
#sensors_df.drop(columns=sensors_df.filter(like='Mean').columns, inplace=True)

sensors_df

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Asymmetry Delta,Asymmetry Theta,Mean Frontal Asymmetry Alpha,Asymmetry Beta,Asymmetry Gamma,Device,Protocol,Vision,Position,Type,Engagement,Engagement(1/alpha),FAA,Pleasantness Index,Pleasantness,Attention,Beta,Alpha,Gamma,Delta
0,s10,876.0,500,Succession_A,Subdivision 1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_120,2.0,1.0,content_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,s10,1376.0,500,Succession_A,Subdivision 2,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_120,2.0,1.0,content_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,s10,1876.0,500,Succession_A,Subdivision 3,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.286,12.343,6.090,-2.374,-5.569,21.562,11.696,5.716,-1.677,-4.655,21.971,11.083,4.989,0.585,-3.712,21.697,9.474,1.475,2.274,-1.991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.636,11.273,4.897,0.098,-3.774,0.127,-0.458,-0.578,0.895,0.645,Laptop,Linear_120,2.0,1.0,content_A,0.062011,-4.897,NaN,NaN,-0.578,11.273,0.098,4.897,-3.774,21.636
3,s10,2376.0,500,Succession_A,Subdivision 4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.233,14.107,8.446,0.301,-2.905,25.491,14.118,8.082,0.321,-3.433,25.422,12.405,7.244,3.230,0.037,23.575,12.439,5.061,4.270,2.038,28.524,-3.740,-3.058,-6.990,-11.456,NaN,NaN,NaN,NaN,NaN,26.154,12.401,6.518,1.538,-1.373,0.022,-0.781,-0.846,0.437,0.556,Laptop,Linear_120,2.0,1.0,content_A,0.065163,-6.518,NaN,NaN,-0.846,12.401,1.538,6.518,-1.373,26.154
4,s10,2876.0,500,Succession_A,Subdivision 5,0.0,0.0,NaN,11.957,7.213,2.87,-6.401,-9.379,17.539,12.394,8.701,-3.467,-8.99,21.439,15.421,10.545,-2.395,-9.617,22.348,16.538,11.707,-1.191,-10.052,21.985,16.750,12.854,-0.348,-4.477,20.567,15.426,10.937,1.428,-4.521,19.777,3.603,-4.179,-6.942,-9.250,NaN,NaN,NaN,NaN,NaN,20.302,14.268,9.848,-1.890,-7.385,0.550,0.386,0.439,0.695,0.860,Laptop,Linear_120,2.0,1.0,content_A,0.017791,-9.848,-7.049,-5.0966,0.439,14.268,-1.890,9.848,-7.385,20.302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1273232,s1338,580828.0,500,Sky calcio club,Subdivision 110,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [ ]:

# Funzione per convertire decibel a lineare
def db_to_linear(df, columns):
    for col in columns:
        df[col] = 10 ** (df[col] / 10)
    return df

# Elenco delle colonne da convertire
columns_to_convert = [
    'Mean PSD Ch1 Delta (dB)', 'Mean PSD Ch1 Theta (dB)', 'Mean PSD Ch1 Alpha (dB)', 'Mean PSD Ch1 Beta (dB)',
    'Mean PSD Ch1 Gamma (dB)', 'Mean PSD Ch2 Delta (dB)', 'Mean PSD Ch2 Theta (dB)', 'Mean PSD Ch2 Alpha (dB)',
    'Mean PSD Ch2 Beta (dB)', 'Mean PSD Ch2 Gamma (dB)', 'Mean PSD Ch3 Delta (dB)', 'Mean PSD Ch3 Theta (dB)',
    'Mean PSD Ch3 Alpha (dB)', 'Mean PSD Ch3 Beta (dB)', 'Mean PSD Ch3 Gamma (dB)', 'Mean PSD Ch4 Delta (dB)',
    'Mean PSD Ch4 Theta (dB)', 'Mean PSD Ch4 Alpha (dB)', 'Mean PSD Ch4 Beta (dB)', 'Mean PSD Ch4 Gamma (dB)',
    'Mean PSD Ch5 Delta (dB)', 'Mean PSD Ch5 Theta (dB)', 'Mean PSD Ch5 Alpha (dB)', 'Mean PSD Ch5 Beta (dB)',
    'Mean PSD Ch5 Gamma (dB)', 'Mean PSD Ch6 Delta (dB)', 'Mean PSD Ch6 Theta (dB)', 'Mean PSD Ch6 Alpha (dB)',
    'Mean PSD Ch6 Beta (dB)', 'Mean PSD Ch6 Gamma (dB)', 'Mean PSD Ch7 Delta (dB)', 'Mean PSD Ch7 Theta (dB)',
    'Mean PSD Ch7 Alpha (dB)', 'Mean PSD Ch7 Beta (dB)', 'Mean PSD Ch7 Gamma (dB)','Mean PSD Electrode Cluster Average Theta (dB)',
    'Mean PSD Electrode Cluster Average Beta (dB)', 'Mean PSD Electrode Cluster Average Alpha (dB)',
    'Mean PSD Electrode Cluster Average Gamma (dB)', 'Mean PSD Electrode Cluster Average Delta (dB)'
]

# Converte i valori da dB a lineare
df = db_to_linear(sensors_df, columns_to_convert)

# Funzione per calcolare gli indicatori
def calculate_indicators(df):
    # Approach/Withdrawal (AW) Index
    df['AW Index'] = (df['Mean PSD Ch7 Alpha (dB)'] - df['Mean PSD Ch1 Alpha (dB)']) / (df['Mean PSD Ch7 Alpha (dB)'] + df['Mean PSD Ch1 Alpha (dB)'])

    # Vamv Valence
    df['Vamv Valence'] = (df['Mean PSD Ch1 Beta (dB)'] / df['Mean PSD Ch1 Alpha (dB)']) - (df['Mean PSD Ch7 Beta (dB)'] / df['Mean PSD Ch7 Alpha (dB)'])

    # Kirk Valence
    df['Kirk Valence'] = np.log(df['Mean PSD Ch1 Alpha (dB)']+df['Mean PSD Ch2 Alpha (dB)']) - np.log(df['Mean PSD Ch7 Alpha (dB)']+df['Mean PSD Ch6 Alpha (dB)']) #prima era solo ch1-ch7

    # Ram12 Valence
    df['Ram12 Valence'] = df['Mean PSD Ch7 Alpha (dB)'] - df['Mean PSD Ch1 Beta (dB)']

    # Ram15 Valence
    df['Ram15 Valence'] = (df['Mean PSD Ch7 Alpha (dB)'] / df['Mean PSD Ch7 Beta (dB)']) - (df['Mean PSD Ch1 Alpha (dB)'] / df['Mean PSD Ch1 Beta (dB)'])

    # Choice Index (Gamma)
    df['Choice Index (Gamma)'] = (np.log(df['Mean PSD Ch2 Gamma (dB)']) - np.log(df['Mean PSD Ch6 Gamma (dB)'])) / (np.log(df['Mean PSD Ch2 Gamma (dB)']) + np.log(df['Mean PSD Ch6 Gamma (dB)']))

    # Choice Index (Beta)
    df['Choice Index (Beta)'] = (np.log(df['Mean PSD Ch2 Beta (dB)']) - np.log(df['Mean PSD Ch6 Beta (dB)'])) / (np.log(df['Mean PSD Ch2 Beta (dB)']) + np.log(df['Mean PSD Ch6 Beta (dB)']))

    # Effort Index 1
    df['Effort Index 1'] = (df['Mean PSD Ch7 Theta (dB)'] - df['Mean PSD Ch1 Theta (dB)']) / (df['Mean PSD Ch7 Theta (dB)'] + df['Mean PSD Ch1 Theta (dB)'])

    # Effort Index 2
    df['Effort Index 2'] = (df['Mean PSD Ch6 Theta (dB)'] + df['Mean PSD Ch7 Theta (dB)']) - (df['Mean PSD Ch1 Theta (dB)'] + df['Mean PSD Ch2 Theta (dB)'])


    return df

# Calcola gli indicatori
sensors_df = calculate_indicators(sensors_df)

#WORKLOAD
for ch in range(1, 8):
    sensors_df[f"WorkloadCh{ch}"] = (sensors_df[f"Mean PSD Ch{ch} Beta (dB)"] + sensors_df[f"Mean PSD Ch{ch} Gamma (dB)"]) / sensors_df[f"Mean PSD Ch{ch} Alpha (dB)"]

# 4. (Facoltativo) Calcola un indice medio di Workload su tutti i 7 canali
workload_cols = [f"WorkloadCh{ch}" for ch in range(1, 8)]
sensors_df["Workload_mean"] = sensors_df[workload_cols].mean(axis=1)
sensors_df.drop(columns=workload_cols, inplace=True)

#workload 2

sensors_df['Workload_mean2'] = (sensors_df['Mean PSD Electrode Cluster Average Beta (dB)'] + sensors_df['Mean PSD Electrode Cluster Average Gamma (dB)']) / sensors_df['Mean PSD Electrode Cluster Average Alpha (dB)']
sensors_df['SASI'] = (sensors_df['Mean PSD Electrode Cluster Average Beta (dB)'] - sensors_df['Mean PSD Electrode Cluster Average Theta (dB)']) / (sensors_df['Mean PSD Electrode Cluster Average Beta (dB)'] + sensors_df['Mean PSD Electrode Cluster Average Theta (dB)'])

print(sensors_df.head())  # Mostra i primi valori per confermare il calcolo

    ID  Start (ms)  Duration (ms)       Content        Comment  Peak Count  \
0  s10       876.0            500  Succession_A  Subdivision 1         0.0   
1  s10      1376.0            500  Succession_A  Subdivision 2         0.0   
2  s10      1876.0            500  Succession_A  Subdivision 3         0.0   
3  s10      2376.0            500  Succession_A  Subdivision 4         0.0   
4  s10      2876.0            500  Succession_A  Subdivision 5         0.0   

   Peaks Per Minute  Average Peak Amplitude  Mean PSD Ch1 Delta (dB)  \
0               0.0                     NaN                      NaN   
1               0.0                     NaN                      NaN   
2               0.0                     NaN                      NaN   
3               0.0                     NaN                      NaN   
4               0.0                     NaN                15.692784   

   Mean PSD Ch1 Theta (dB)  Mean PSD Ch1 Alpha (dB)  Mean PSD Ch1 Beta (dB)  \
0                  

In [ ]:
print(sensors_df.columns)


Index(['ID', 'Start (ms)', 'Duration (ms)', 'Content', 'Comment', 'Peak Count',
       'Peaks Per Minute', 'Average Peak Amplitude', 'Mean PSD Ch1 Delta (dB)',
       'Mean PSD Ch1 Theta (dB)', 'Mean PSD Ch1 Alpha (dB)',
       'Mean PSD Ch1 Beta (dB)', 'Mean PSD Ch1 Gamma (dB)',
       'Mean PSD Ch2 Delta (dB)', 'Mean PSD Ch2 Theta (dB)',
       'Mean PSD Ch2 Alpha (dB)', 'Mean PSD Ch2 Beta (dB)',
       'Mean PSD Ch2 Gamma (dB)', 'Mean PSD Ch3 Delta (dB)',
       'Mean PSD Ch3 Theta (dB)', 'Mean PSD Ch3 Alpha (dB)',
       'Mean PSD Ch3 Beta (dB)', 'Mean PSD Ch3 Gamma (dB)',
       'Mean PSD Ch4 Delta (dB)', 'Mean PSD Ch4 Theta (dB)',
       'Mean PSD Ch4 Alpha (dB)', 'Mean PSD Ch4 Beta (dB)',
       'Mean PSD Ch4 Gamma (dB)', 'Mean PSD Ch5 Delta (dB)',
       'Mean PSD Ch5 Theta (dB)', 'Mean PSD Ch5 Alpha (dB)',
       'Mean PSD Ch5 Beta (dB)', 'Mean PSD Ch5 Gamma (dB)',
       'Mean PSD Ch6 Delta (dB)', 'Mean PSD Ch6 Theta (dB)',
       'Mean PSD Ch6 Alpha (dB)', 'Mean PSD Ch6 Beta

# RIMOZIONE OUTLIERS PROVA

In [ ]:
import pandas as pd
import numpy as np

# Funzione per sostituire outlier con NaN
def replace_outliers_with_nan(df):
    # Seleziona solo le colonne numeriche
    numeric_columns = df.select_dtypes(include=['number']).columns
    df_replaced = df.copy()

    for col in numeric_columns:
        Q1 = df[col].quantile(0.25)  # 25° percentile (primo quartile)
        Q3 = df[col].quantile(0.75)  # 75° percentile (terzo quartile)
        IQR = Q3 - Q1  # Intervallo interquartile

        # Calcola i limiti per individuare gli outlier
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Sostituisci gli outlier con NaN
        df_replaced[col] = df[col].apply(
            lambda x: np.nan if x < lower_bound or x > upper_bound else x
        )

    return df_replaced



# Apply the function to your dataset
sensors_df = replace_outliers_with_nan(sensors_df)

# Standardize values

In [ ]:
# Extend standardization to additional columns for content_A
columns_to_standardize = [
    'Engagement', 'Pleasantness', 'Attention', 'FAA', 'Engagement(1/alpha)',
    'Asymmetry Delta',
    'Asymmetry Theta',
    'Asymmetry Beta',
    'Asymmetry Gamma',
    'Beta', 'Alpha', 'Gamma', 'Delta', 'AW Index', 'Vamv Valence', 'Kirk Valence',
    'Ram12 Valence', 'Ram15 Valence', 'Choice Index (Gamma)',
    'Choice Index (Beta)', 'Effort Index 1', 'Effort Index 2',"Workload_mean",'Workload_mean2','SASI'
]

# Calculate mean and std for content_A
standardizing_values = (
    sensors_df[sensors_df['Type'] == 'content_A']
    .groupby(['ID', 'Vision'])
    .agg({col: ['mean', 'std'] for col in columns_to_standardize})
    .reset_index()
)

# Rename columns for content_A
new_columns = ['ID', 'Vision'] + [
    f"{col}_{stat}" for col in columns_to_standardize for stat in ['mean', 'std']
]
standardizing_values.columns = new_columns

sensors_df = pd.merge(sensors_df, standardizing_values, on=['ID', 'Vision'])

# Apply standardization for content_A
for col in columns_to_standardize:
    sensors_df[f'{col} standardized'] = (
        (sensors_df[col] - sensors_df[f'{col}_mean']) / sensors_df[f'{col}_std']
    )
    sensors_df = sensors_df.drop([f'{col}_mean', f'{col}_std'], axis=1)

# Calculate mean and std for ads
standardizing_values = (
    sensors_df[sensors_df['Type'] == 'ad']
    .groupby(['ID', 'Vision'])
    .agg({col: ['mean', 'std'] for col in columns_to_standardize})
    .reset_index()
)

# # Rename columns for ads
# new_columns = ['ID', 'Vision'] + [
#     f"{col}_{stat}" for col in columns_to_standardize for stat in ['mean', 'std']
# ]
# standardizing_values.columns = new_columns

# sensors_df = pd.merge(sensors_df, standardizing_values, on=['ID', 'Vision'])

# # Apply standardization for ads
# for col in columns_to_standardize:
#     sensors_df[f'{col} standardized_ad'] = (
#         (sensors_df[col] - sensors_df[f'{col}_mean']) / sensors_df[f'{col}_std']
#     )
#     sensors_df = sensors_df.drop([f'{col}_mean', f'{col}_std'], axis=1)

# Drop columns that are no longer needed
columns_to_drop = ['Alpha', 'Beta', 'Gamma', 'Delta']
sensors_df = sensors_df.drop(columns_to_drop, axis=1)


In [ ]:
sensors_df

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Asymmetry Delta,Asymmetry Theta,Mean Frontal Asymmetry Alpha,Asymmetry Beta,Asymmetry Gamma,Device,Protocol,Vision,Position,Type,Engagement,Engagement(1/alpha),FAA,Pleasantness Index,Pleasantness,Attention,AW Index,Vamv Valence,Kirk Valence,Ram12 Valence,Ram15 Valence,Choice Index (Gamma),Choice Index (Beta),Effort Index 1,Effort Index 2,Workload_mean,Workload_mean2,SASI,Engagement standardized,Pleasantness standardized,Attention standardized,FAA standardized,Engagement(1/alpha) standardized,Asymmetry Delta standardized,Asymmetry Theta standardized,Asymmetry Beta standardized,Asymmetry Gamma standardized,Beta standardized,Alpha standardized,Gamma standardized,Delta standardized,AW Index standardized,Vamv Valence standardized,Kirk Valence standardized,Ram12 Valence standardized,Ram15 Valence standardized,Choice Index (Gamma) standardized,Choice Index (Beta) standardized,Effort Index 1 standardized,Effort Index 2 standardized,Workload_mean standardized,Workload_mean2 standardized,SASI standardized
0,s10,876.0,500,Succession_A,Subdivision 1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_120,2.0,1.0,content_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,s10,1376.0,500,Succession_A,Subdivision 2,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_120,2.0,1.0,content_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,s10,1876.0,500,Succession_A,Subdivision 3,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.064433,0.578895,0.277396,NaN,NaN,3.729065,0.679673,0.342373,NaN,NaN,3.154278,1.144195,0.425402,NaN,8.859312,1.404430,1.688107,0.632266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.747126,13.406024,3.088161,1.022822,0.419373,0.127,-0.458,-0.578,0.895,0.645,Laptop,Linear_120,2.0,1.0,content_A,0.062011,-4.897,NaN,NaN,-0.578,11.273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.658636,0.467007,-0.858225,-0.882392,-0.420272,1.340928,NaN,-1.373869,0.198127,-0.287123,2.456267,0.845844,0.900460,1.373869,1.322586,1.417281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.474636,-0.706337,-0.905985
3,s10,2376.0,500,Succession_A,Subdivision 4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:

# standardizing_values = (
#     sensors_df[sensors_df['Type'] == 'content_A']
#     .groupby(['ID', 'Vision'])
#     .agg({'Engagement': ['mean', 'std'], 'Pleasantness': ['mean', 'std'],'Attention': ['mean', 'std'],'FAA':['mean','std']})
#     .reset_index()
# )

# # Rename columns
# standardizing_values.columns = ['ID', 'Vision','Engagement_mean', 'Engagement_std', 'Pleasantness_mean', 'Pleasantness_std', 'Attention_mean', 'Attention_std', 'FAA_mean', 'FAA_std']

# sensors_df = pd.merge(sensors_df, standardizing_values, on=['ID', 'Vision'])

# for col in ['Engagement', 'Pleasantness', 'Attention','FAA']:
#     sensors_df[f'{col} standardized'] = (
#         (sensors_df[col] - sensors_df[f'{col}_mean']) / sensors_df[f'{col}_std']
#     )
#     sensors_df = sensors_df.drop(f'{col}_mean', axis = 1)
#     sensors_df = sensors_df.drop(f'{col}_std', axis = 1)

# Import surveys


In [ ]:
directory = r"Surveys"

def import_dataset_with_prefix(directory, prefix):
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file name starts with the specified prefix
            if file.startswith(prefix) and file.endswith(('.xlsx', '.xls')):
                # Read the Excel file and return the DataFrame
                return pd.read_excel(os.path.join(root, file))
    # If no file with the specified prefix is found, return None
    return None

### Import dataset of pre vision survey

In [ ]:

df_pre = import_dataset_with_prefix(directory, 'FCP_Pre_visione')
df_pre = df_pre.iloc[1:,:]
df_pre

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Q1,Q2,Q3
1,2024-04-16 20:22:43,2024-04-16 20:25:58,IP Address,212.162.98.170,100,195,True,2024-04-16 20:25:59.295000,R_24O9LExwYPjPwgn,NaN,NaN,NaN,NaN,41.8904,12.5126,anonymous,EN,test,NaN,Altro/Preferisco non rispondere
2,2024-04-16 20:42:29,2024-04-16 20:43:00,IP Address,87.13.79.7,100,30,True,2024-04-16 20:43:01.110000,R_6rPQ8xDWFit6ybf,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,deee,1850,Femmina
3,2024-04-16 20:51:57,2024-04-16 20:52:27,IP Address,93.51.247.100,100,29,True,2024-04-16 20:52:27.489000,R_2xZXv9U0QqaJDc5,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,cazzituoi?,1901,Femmina
4,2024-04-16 20:56:06,2024-04-16 20:56:46,IP Address,188.216.114.233,100,40,True,2024-04-16 20:56:47.483000,R_6eUuX7lDdPbobLA,NaN,NaN,NaN,NaN,45.5819,9.2683,anonymous,EN,Test sonia Campari,1971,Femmina
5,2024-04-16 23:39:34,2024-04-16 23:39:58,IP Address,2.233.164.46,100,23,True,2024-04-16 23:39:58.716000,R_214XeqNN4yYaIc4,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,oh3,NaN,Maschio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,2024-06-03 19:02:33,2024-06-03 19:14:00,IP Address,93.51.247.100,100,686,True,2024-06-03 19:14:00.778000,R_8CTp7Fd8bDlP24b,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,s1324,1980,Maschio
614,2024-06-03 19:14:08,2024-06-03 19:14:29,IP Address,93.51.247.100,100,20,True,2024-06-03 19:14:29.746000,R_82W55nW2cKkaDMm,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,s1324,1980,Maschio
615,2024-06-03 20:10:09,2024-06-03 20:16:05,IP Address,93.51.247.100,100,356,True,2024-06-03 20:16:06.821000,R_2hxM9GTMvzh8spu,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,S1336,1969,Maschio
616,2024-05-29 10:28:52,2024-05-29 10:50:47,IP Address,93.51.247.100,75,1315,False,2024-06-05 10:50:52.329000,R_2BYrRzRJEZfBmEZ,NaN,NaN,NaN,NaN,NaN,NaN,anonymous,EN,s1227,1967,Maschio


In [ ]:

df_pre = df_pre.rename(columns={'Q1': 'ID',
                                'Q2': 'DOB',
                                'Q3': 'Gender'})
df_pre

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,ID,DOB,Gender
1,2024-04-16 20:22:43,2024-04-16 20:25:58,IP Address,212.162.98.170,100,195,True,2024-04-16 20:25:59.295000,R_24O9LExwYPjPwgn,NaN,NaN,NaN,NaN,41.8904,12.5126,anonymous,EN,test,NaN,Altro/Preferisco non rispondere
2,2024-04-16 20:42:29,2024-04-16 20:43:00,IP Address,87.13.79.7,100,30,True,2024-04-16 20:43:01.110000,R_6rPQ8xDWFit6ybf,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,deee,1850,Femmina
3,2024-04-16 20:51:57,2024-04-16 20:52:27,IP Address,93.51.247.100,100,29,True,2024-04-16 20:52:27.489000,R_2xZXv9U0QqaJDc5,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,cazzituoi?,1901,Femmina
4,2024-04-16 20:56:06,2024-04-16 20:56:46,IP Address,188.216.114.233,100,40,True,2024-04-16 20:56:47.483000,R_6eUuX7lDdPbobLA,NaN,NaN,NaN,NaN,45.5819,9.2683,anonymous,EN,Test sonia Campari,1971,Femmina
5,2024-04-16 23:39:34,2024-04-16 23:39:58,IP Address,2.233.164.46,100,23,True,2024-04-16 23:39:58.716000,R_214XeqNN4yYaIc4,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,oh3,NaN,Maschio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,2024-06-03 19:02:33,2024-06-03 19:14:00,IP Address,93.51.247.100,100,686,True,2024-06-03 19:14:00.778000,R_8CTp7Fd8bDlP24b,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,s1324,1980,Maschio
614,2024-06-03 19:14:08,2024-06-03 19:14:29,IP Address,93.51.247.100,100,20,True,2024-06-03 19:14:29.746000,R_82W55nW2cKkaDMm,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,s1324,1980,Maschio
615,2024-06-03 20:10:09,2024-06-03 20:16:05,IP Address,93.51.247.100,100,356,True,2024-06-03 20:16:06.821000,R_2hxM9GTMvzh8spu,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,S1336,1969,Maschio
616,2024-05-29 10:28:52,2024-05-29 10:50:47,IP Address,93.51.247.100,75,1315,False,2024-06-05 10:50:52.329000,R_2BYrRzRJEZfBmEZ,NaN,NaN,NaN,NaN,NaN,NaN,anonymous,EN,s1227,1967,Maschio


In [ ]:
len(df_pre['ID'].unique())

610

In [ ]:
df_pre.columns

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'ID', 'DOB', 'Gender'],
      dtype='object')

In [ ]:
# Convert 'ID' column to lowercase
df_pre['ID'] = df_pre['ID'].str.lower()

# Remove spaces from 'ID' column
df_pre['ID'] = df_pre['ID'].str.replace("\\s+", "", regex=True)

# Add an 's' in front of IDs that don't have it
df_pre['ID'] = df_pre['ID'].apply(lambda x: 's' + x if not x.startswith('s') else x)

df_pre = df_pre.iloc[df_pre['ID'].str.extract(r'(\d+)').astype(float).fillna(-1).squeeze().argsort(kind='mergesort')]

In [ ]:
df_pre['ID'].unique()

array(['stest', 'sdeee', 'scazzituoi?', 'stestsoniacampari', 's', 's01',
       's02', 'soh3', 's03', 's04', 's05', 's06', 's07', 's08', 's09',
       's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18',
       's19', 's20', 's21', 's22', 's23', 's24', 's25', 's26', 's27',
       's28', 's29', 's30', 's31', 's031', 's32', 's33', 's34', 's35',
       's36', 's37', 's38', 's39', 's40', 's41', 's42', 's43', 's44',
       's45', 's46', 's47', 's48', 's49', 's50', 's51', 's52', 's53',
       's54', 's55', 's56', 's57', 's58', 's59', 's60', 's61', 's62',
       's63', 's64', 's65', 's66', 's67', 's68', 's69', 's70', 's71',
       's72', 's73', 's74', 's75', 's76', 's77', 's78', 's79', 's80',
       's81', 's82', 's83', 's84', 's85', 's86', 's87', 's88', 's89',
       's90', 's91', 's92', 's93', 's94', 's95', 's96', 's97', 's98',
       's99', 's100', 's101', 's102', 's103', 's104', 's105', 's106',
       's107', 's108', 's109', 's110', 's111', 's112', 's113', 's114',
       's115',

In [ ]:

# Merge df_pre with sensors_df
sensors_df = sensors_df.merge(df_pre, on = 'ID',how='left')

sensors_df

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Asymmetry Delta,Asymmetry Theta,Mean Frontal Asymmetry Alpha,Asymmetry Beta,Asymmetry Gamma,Device,Protocol,Vision,Position,Type,Engagement,Engagement(1/alpha),FAA,Pleasantness Index,Pleasantness,Attention,AW Index,Vamv Valence,Kirk Valence,Ram12 Valence,Ram15 Valence,Choice Index (Gamma),Choice Index (Beta),Effort Index 1,Effort Index 2,Workload_mean,Workload_mean2,SASI,Engagement standardized,Pleasantness standardized,Attention standardized,FAA standardized,Engagement(1/alpha) standardized,Asymmetry Delta standardized,Asymmetry Theta standardized,Asymmetry Beta standardized,Asymmetry Gamma standardized,Beta standardized,Alpha standardized,Gamma standardized,Delta standardized,AW Index standardized,Vamv Valence standardized,Kirk Valence standardized,Ram12 Valence standardized,Ram15 Valence standardized,Choice Index (Gamma) standardized,Choice Index (Beta) standardized,Effort Index 1 standardized,Effort Index 2 standardized,Workload_mean standardized,Workload_mean2 standardized,SASI standardized,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,DOB,Gender
0,s10,876.0,500,Succession_A,Subdivision 1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_120,2.0,1.0,content_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-17 15:50:34,2024-04-17 15:57:46,IP Address,93.51.247.100,100,431,True,2024-04-17 15:57:46.450000,R_5dMsXsUPvJePDAo,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,1974,Femmina
1,s10,1376.0,500,Succession_A,Subdivision 2,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_120,2.0,1.0,content_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-17 15:50:34,2024-04-17 15:57:46,IP Address,93.51.247.100,100,431,True,2024-04-17 15:57:46.450000,R_5dMsXsUPvJePDAo,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,1974,Femmina
2,s10,1876.0,500,Succession_A,Subdivision 3,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.064433,0.578895,0.277396,NaN,NaN,3.729065,0.679673,0.342373,NaN,NaN,3.1

In [ ]:
sensors_df[sensors_df['DOB'].isna()]['ID'].unique()

array(['s105', 's114', 's154', 's308', 's318', 's1009', 's1017', 's1162'],
      dtype=object)

In [ ]:
#sostituiamo quelli che non avevano corrispondenza su df_pre
sensors_df.loc[sensors_df['ID'] == 's154', ['Gender','DOB']]=['Maschio',1998]
sensors_df.loc[sensors_df['ID'] == 's308', ['Gender','DOB']]=['Maschio',1998]
sensors_df.loc[sensors_df['ID'] == 's1009', ['Gender','DOB']]=['Maschio',1998]
sensors_df.loc[sensors_df['ID'] == 's1162', ['Gender','DOB']]=['Femmina',1964]

#eliminiamo le righe con i na in gender e dob che tanto sono dei duplicati
sensors_df = sensors_df.dropna(subset=['Gender', 'DOB'])

In [ ]:
sensors_df[sensors_df['ID']=='s318']

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Asymmetry Delta,Asymmetry Theta,Mean Frontal Asymmetry Alpha,Asymmetry Beta,Asymmetry Gamma,Device,Protocol,Vision,Position,Type,Engagement,Engagement(1/alpha),FAA,Pleasantness Index,Pleasantness,Attention,AW Index,Vamv Valence,Kirk Valence,Ram12 Valence,Ram15 Valence,Choice Index (Gamma),Choice Index (Beta),Effort Index 1,Effort Index 2,Workload_mean,Workload_mean2,SASI,Engagement standardized,Pleasantness standardized,Attention standardized,FAA standardized,Engagement(1/alpha) standardized,Asymmetry Delta standardized,Asymmetry Theta standardized,Asymmetry Beta standardized,Asymmetry Gamma standardized,Beta standardized,Alpha standardized,Gamma standardized,Delta standardized,AW Index standardized,Vamv Valence standardized,Kirk Valence standardized,Ram12 Valence standardized,Ram15 Valence standardized,Choice Index (Gamma) standardized,Choice Index (Beta) standardized,Effort Index 1 standardized,Effort Index 2 standardized,Workload_mean standardized,Workload_mean2 standardized,SASI standardized,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,DOB,Gender
690930,s318,811.0,500,Il cavallo e la torre_A,Subdivision 1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.926838,4.312211,0.681240,0.285496,0.060076,13.185603,3.989330,0.453315,0.371792,0.173820,13.246465,4.258925,0.536785,0.327341,0.193286,11.199535,4.382279,0.468813,0.330826,0.150938,NaN,NaN,NaN,NaN,NaN,11.389378,4.235454,0.535057,0.328852,0.144544,NaN,NaN,-0.033,NaN,NaN,Smartphone,Linear_60,2.0,1.0,content_A,0.068934,2.716,NaN,NaN,-0.033,6.269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.927100,0.884757,-0.855903,-0.536256,0.435171,0.041160,NaN,1.023010,NaN,NaN,NaN,NaN,-0.633683,-1.023010,-0.154211,-0.393646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.406507,0.476938,-0.639992,2024-05-13 19:28:25,2024-05-13 19:29:01,IP Address,93.51.247.100,100,36,True,2024-05-13 19:29:02.545000,R_2uqEM5w3nfCQnSH,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,1969,Maschio
690932,s318,1311.0,500,Il cavallo e la torre_A,Subdivision 2,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.441876,0.114683,NaN,NaN,NaN,0.268534,0.155597,NaN,NaN,NaN,0.387347,0.193865,NaN,NaN,NaN,0.414859,0.157253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.378181,0.155346,NaN,NaN,-0.032,NaN,NaN,Smartphone,Linear_60,2.0,1.0,content_A,0.004815,-7.697,NaN,NaN,-0.032,18.613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.090855,0.090669,-0.989644,-1.261131,0.436992,2.448212,NaN,-2.692782,NaN,NaN,NaN,NaN,-0.321863,2.692782,-0.019639,1.543424,NaN,NaN,NaN,NaN,Na

In [ ]:
len(sensors_df['ID'].unique())

565

### Import datasets post vision survey

In [ ]:

df_post_list = []
for i in range(1, 3):  # Loop from 1 to 2 for FCP_Post_visione_1 and FCP_Post_visione_2

    df_post_i = import_dataset_with_prefix(directory, f'FCP_Post_visione_{i}')

    # Add a column telling if it was the first or second vision
    df_post_i["Source"] = i

    df_post_i = df_post_i.iloc[1:,:]

    df_post_list.append(df_post_i)

# Import VOD surveys
df_post_VOD = import_dataset_with_prefix(directory, f'FCP_Post_visione_VOD')

df_post_VOD["Source"] = "VOD"

df_post_VOD = df_post_VOD.iloc[1:,:]

df_post_list.append(df_post_VOD)

# Import YT surveys
df_post_YT = import_dataset_with_prefix(directory, f'FCP_Post_visione_YouTube')

df_post_YT["Source"] = "YouTube"

df_post_YT = df_post_YT.iloc[1:,:]

df_post_list.append(df_post_YT)


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:

#questions_df contains the questions of the survey
questions_df = pd.DataFrame({'Column_Name': import_dataset_with_prefix(directory, 'FCP_Post_visione_1').columns.tolist(), 'Question': import_dataset_with_prefix(directory, 'FCP_Post_visione_1').iloc[0].tolist()})


#Create a single dataframe for both post_vision surveys
df_post = pd.concat(df_post_list, ignore_index= True)
df_post


,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Q2,Q1_1,Q1_2,Q1_3,Q1_4,Q1_5,Q3,Q5,Q5_8_TEXT,Alim1,Alim1_6_TEXT,Alim2_5,Alim2_6,Alim2_7,Alim2_8,Alim3,Alim4,Bev1,Bev1_7_TEXT,Bev2_2,Bev2_3,Bev2_4,Bev2_5,Bev3,Bev4,Integr1_5,Integr1_6,Integr1_9,Integr1_10,Integr2,Integr3,Auto1_1,Auto1_2,Auto1_3,Auto1_4,Auto2,Auto3,Super1_5,Super1_6,Super1_7,Super1_8,Super2,Super3,Tel1_5,Tel1_8,Tel1_11,Tel1_12,Tel2,Tel3,Arred1_5,Arred1_6,Arred1_7,Arred1_8,Arred2,Arred3,eComm1_5,eComm1_6,eComm1_9,eComm1_10,eComm2,eComm3,cosm1_5,cosm1_6,cosm1_7,cosm1_8,Cosm2,Cosm3,Valutaz1_1,Valutaz1_2,Valutaz1_3,Valutaz1_4,Valutaz1_5,Source
0,2024-04-16 20:29:30,2024-04-16 20:31:59,IP Address,5.77.66.30,100,149,True,2024-04-16 20:32:00.698000,R_4hmxpmQTtI65WmR,NaN,NaN,NaN,NaN,43.1479,12.1097,anonymous,EN,Test,NaN,NaN,NaN,NaN,NaN,Sì,"Prodotti alimentari,Bevande",NaN,Prodotti da frigo,NaN,Fdgg,NaN,NaN,NaN,sì,NaN,NaN,NaN,Frrtf,NaN,NaN,NaN,Sì,NaN,NaN,NaN,NaN,NaN,NaN,Sustenium,NaN,NaN,NaN,NaN,NaN,"Audi,Alfa Romeo",NaN,NaN,NaN,NaN,Sì,"Coop,Conad",NaN,NaN,NaN,NaN,Sì,"Fastweb,Wind/Tre",NaN,NaN,NaN,NaN,No,"Ikea,Leroy Merlin",NaN,NaN,NaN,NaN,NaN,Amazon,NaN,NaN,NaN,NaN,No,"Dolce e Gabbana,Armani Perfume",4,4,3,3,3,1
1,2024-04-16 20:43:07,2024-04-16 20:51:22,IP Address,87.13.79.7,100,494,True,2024-04-16 20:51:22.799000,R_2zZbbaKpyNCdMRj,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,vvv,NaN,NaN,NaN,NaN,NaN,Sì,"Prodotti alimentari,Supermercati",NaN,Pasta,NaN,NaN,NaN,NaN,NaN,NaN,"Barilla,Cirio",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BMW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Vodafone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,4,4,1
2,2024-04-16 20:51:30,2024-04-16 20:56:15,IP Address,87.13.79.7,100,284,True,2024-04-16 20:56:15.934000,R_2MQY4nKLInCEux5,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,rrrrr,NaN,NaN,NaN,NaN,NaN,Sì,"Bevande,Supermercati",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cirio,"Bibite gassate,Vino",NaN,ccc,NaN,NaN,NaN,Sì,NaN,NaN,NaN,NaN,NaN,NaN,Aspirina,NaN,NaN,NaN,NaN,NaN,Audi,NaN,NaN,NaN,NaN,No,Coop,NaN,NaN,NaN,NaN,No,Wind/Tre,NaN,NaN,NaN,NaN,No,Leroy Merlin,NaN,NaN,NaN,NaN,NaN,Ebay,NaN,NaN,NaN,NaN,No,Chanel,1,5,3,3,5,1
3,2024-04-16 23:02:17,2024-04-16 23:06:55,IP Address,2.233.164.46,100,277,True,2024-04-16 23:06:55.879000,R_3DTfCcKYyGQuLlb,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,1r2,NaN,NaN,NaN,NaN,NaN,Sì,"Prodotti alimentari,Automobili,Cosmetica e profumeria",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3,4,5,4,1
4,2024-04-16 23:07:01,2024-04-16 23:07:21,IP Address,2.233.164.46,100,20,True,2024-04-16 23:07:22.143000,R_8qpx9l737tJXHza,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,12e,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,2024-06-03 19:05:14,2024-06-03 19:12:19,IP Address,93.51.247.100,100,425,True,2024-06-03 19:12:20.274000,R_2Rz7P8eB6ZoCEao,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,S1323,7,6,5,8,10,Sì,"Prodotti alimentari,Integratori e farmaci,Automobili,Altro (specificare)",PASTA SARDA,"Biscotti/merendine,Pasta",N

In [ ]:
vision_df

,ID,Protocol,Vision,Contenuto,Emittente/Canale,ADV_1,ADV_2,ADV_3,ADV_4
0,s1,Linear_60,1,NaN,NaN,NaN,NaN,NaN,NaN
1,s2,Linear_120,1,NaN,NaN,NaN,NaN,NaN,NaN
2,s3,Linear_60,1,NaN,NaN,NaN,NaN,NaN,NaN
3,s4,Linear_120,1,NaN,NaN,NaN,NaN,NaN,NaN
4,s5,Linear_180,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1181,s1321,VOD,2,Schindler - La vera stoeia,SKY,Fregola,Amazon,Chanel,Fastweb
1182,s1322,Youtube,2,Da operaio a IMPRENDITORE partendo da zero (come ho fatto),Michele Molteni,Ikea,Fregola,Mutti,Chanel
1183,s1323,VOD,2,Freedom,MEDIASET,Fregola,Chanel,Fastweb,Coca cola
1184,s1324,VOD,2,Di Martedì (Luca e Paolo),LA7,Barilla,Alfa Romeo,Fregola,Mutti


In [ ]:
df_post.rename(columns={'Q2': 'ID'}, inplace=True)
df_post

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,ID,Q1_1,Q1_2,Q1_3,Q1_4,Q1_5,Q3,Q5,Q5_8_TEXT,Alim1,Alim1_6_TEXT,Alim2_5,Alim2_6,Alim2_7,Alim2_8,Alim3,Alim4,Bev1,Bev1_7_TEXT,Bev2_2,Bev2_3,Bev2_4,Bev2_5,Bev3,Bev4,Integr1_5,Integr1_6,Integr1_9,Integr1_10,Integr2,Integr3,Auto1_1,Auto1_2,Auto1_3,Auto1_4,Auto2,Auto3,Super1_5,Super1_6,Super1_7,Super1_8,Super2,Super3,Tel1_5,Tel1_8,Tel1_11,Tel1_12,Tel2,Tel3,Arred1_5,Arred1_6,Arred1_7,Arred1_8,Arred2,Arred3,eComm1_5,eComm1_6,eComm1_9,eComm1_10,eComm2,eComm3,cosm1_5,cosm1_6,cosm1_7,cosm1_8,Cosm2,Cosm3,Valutaz1_1,Valutaz1_2,Valutaz1_3,Valutaz1_4,Valutaz1_5,Source
0,2024-04-16 20:29:30,2024-04-16 20:31:59,IP Address,5.77.66.30,100,149,True,2024-04-16 20:32:00.698000,R_4hmxpmQTtI65WmR,NaN,NaN,NaN,NaN,43.1479,12.1097,anonymous,EN,Test,NaN,NaN,NaN,NaN,NaN,Sì,"Prodotti alimentari,Bevande",NaN,Prodotti da frigo,NaN,Fdgg,NaN,NaN,NaN,sì,NaN,NaN,NaN,Frrtf,NaN,NaN,NaN,Sì,NaN,NaN,NaN,NaN,NaN,NaN,Sustenium,NaN,NaN,NaN,NaN,NaN,"Audi,Alfa Romeo",NaN,NaN,NaN,NaN,Sì,"Coop,Conad",NaN,NaN,NaN,NaN,Sì,"Fastweb,Wind/Tre",NaN,NaN,NaN,NaN,No,"Ikea,Leroy Merlin",NaN,NaN,NaN,NaN,NaN,Amazon,NaN,NaN,NaN,NaN,No,"Dolce e Gabbana,Armani Perfume",4,4,3,3,3,1
1,2024-04-16 20:43:07,2024-04-16 20:51:22,IP Address,87.13.79.7,100,494,True,2024-04-16 20:51:22.799000,R_2zZbbaKpyNCdMRj,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,vvv,NaN,NaN,NaN,NaN,NaN,Sì,"Prodotti alimentari,Supermercati",NaN,Pasta,NaN,NaN,NaN,NaN,NaN,NaN,"Barilla,Cirio",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BMW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Vodafone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,4,4,1
2,2024-04-16 20:51:30,2024-04-16 20:56:15,IP Address,87.13.79.7,100,284,True,2024-04-16 20:56:15.934000,R_2MQY4nKLInCEux5,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,rrrrr,NaN,NaN,NaN,NaN,NaN,Sì,"Bevande,Supermercati",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cirio,"Bibite gassate,Vino",NaN,ccc,NaN,NaN,NaN,Sì,NaN,NaN,NaN,NaN,NaN,NaN,Aspirina,NaN,NaN,NaN,NaN,NaN,Audi,NaN,NaN,NaN,NaN,No,Coop,NaN,NaN,NaN,NaN,No,Wind/Tre,NaN,NaN,NaN,NaN,No,Leroy Merlin,NaN,NaN,NaN,NaN,NaN,Ebay,NaN,NaN,NaN,NaN,No,Chanel,1,5,3,3,5,1
3,2024-04-16 23:02:17,2024-04-16 23:06:55,IP Address,2.233.164.46,100,277,True,2024-04-16 23:06:55.879000,R_3DTfCcKYyGQuLlb,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,1r2,NaN,NaN,NaN,NaN,NaN,Sì,"Prodotti alimentari,Automobili,Cosmetica e profumeria",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3,4,5,4,1
4,2024-04-16 23:07:01,2024-04-16 23:07:21,IP Address,2.233.164.46,100,20,True,2024-04-16 23:07:22.143000,R_8qpx9l737tJXHza,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,12e,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,2024-06-03 19:05:14,2024-06-03 19:12:19,IP Address,93.51.247.100,100,425,True,2024-06-03 19:12:20.274000,R_2Rz7P8eB6ZoCEao,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,S1323,7,6,5,8,10,Sì,"Prodotti alimentari,Integratori e farmaci,Automobili,Altro (specificare)",PASTA SARDA,"Biscotti/merendine,Pasta",N

## Merge Vision2 and df_post to create the Vision Column in df_post

In [ ]:
import re

def normalize_id(value):
    # Extract numbers from the value
    match = re.search(r'\d+', str(value))
    if match:
        number = int(match.group(0))  # Convert to integer to remove leading zeros
        # Format as 's01', 's02', ... for 1-9, and 's10', 's11', ... otherwise
        if 1 <= number <= 9:
            return f's0{number}'
        else:
            return f's{number}'
    return value  # If no match, return the original value

# Apply the normalization function to the "ID" column
df_post['ID'] = df_post['ID'].apply(normalize_id)
df_post['ID'].unique()

array(['Test', 'vvv', 'rrrrr', 's01', 's12', 's07', 'j', 's02', 'zzz',
       'bbb', 'ww', 'ccc', 's666', 'uuu', 'nnn', 'mr', 'Ccc', 'Hb', 'max',
       'test', 'aaa', 's88', 's0', 's111', 's03', 's04', 's06', 's05',
       's08', 's09', 's10', 's11', 's13', 's14', 's15', 's16', 's17',
       's18', 's19', 's20', 's21', 's22', 's23', 's24', 's25', 's26',
       's27', 's28', 's29', 's30', 's31', 's32', 's33', 's34', 's36',
       's35', 's37', 's38', 's40', 's39', 's41', 's42', 's44', 's43',
       's46', 's45', 's47', 's48', 's49', 's50', 's51', 's52', 's54',
       's53', 's56', 's55', 's58', 's57', 's59', 's60', 's61', 's62',
       's63', 's64', 'Test Sonia Campari ', 'me', 's65', 's66', 'UU',
       's67', 's68', 's70', 's69', 's71', 's72', 's73', 's74', 's75',
       's76', 's78', 's77', 'FF', 's79', 's80', 's81', 's82', 's83',
       's84', 's85', 's86', 's87', 's90', 's89', 's92', 's91', 's93',
       's94', 's95', 's96', 's97', 's98', 's99', 's100', 's101', 's102',
       's10

In [ ]:
# import re

# # Function to normalize IDs
# def normalize_id(value):
#     # Extract numbers from the value
#     match = re.search(r'\d+', str(value))
#     if match:
#         number = match.group(0)
#         return f's{int(number)}'  # Convert to integer to remove leading zeros, then format as s{number}
#     return value  # If no match, return the original value

# # Apply the normalization function to the "ID" column
# df_post['ID'] = df_post['ID'].apply(normalize_id)
# df_post['ID'].unique()

In [ ]:
vision_df['Vision']

,Vision
0,1
1,1
2,1
3,1
4,1
...,...
1181,2
1182,2
1183,2
1184,2


In [ ]:
# Rename 'Source' to 'Protocol' in df_post to match vision_df
df_post = df_post.rename(columns={'Source': 'Protocol'})

# Ensure consistent capitalization in vision_df['Protocol']
vision_df['Protocol'] = vision_df['Protocol'].replace({'Youtube': 'YouTube'})

# Create the Vision column in df_post by mapping values from vision_df
def assign_vision(row):
    # If Protocol is numeric (1 or 2), assign it directly
    if row['Protocol'] in [1, 2]:
        return row['Protocol']

    # For other protocols (e.g., Linear_60, VOD, YouTube), match using vision_df
    matched_row = vision_df.loc[
        (vision_df['ID'] == row['ID']) & (vision_df['Protocol'] == row['Protocol'])
    ]

    # If a match is found, return its Vision value
    if not matched_row.empty:
        return matched_row.iloc[0]['Vision']

    # If no match, return NaN
    return None

# Apply the function to populate the Vision column
df_post['Vision'] = df_post.apply(assign_vision, axis=1)

# Display the updated DataFrame
df_post



,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,ID,Q1_1,Q1_2,Q1_3,Q1_4,Q1_5,Q3,Q5,Q5_8_TEXT,Alim1,Alim1_6_TEXT,Alim2_5,Alim2_6,Alim2_7,Alim2_8,Alim3,Alim4,Bev1,Bev1_7_TEXT,Bev2_2,Bev2_3,Bev2_4,Bev2_5,Bev3,Bev4,Integr1_5,Integr1_6,Integr1_9,Integr1_10,Integr2,Integr3,Auto1_1,Auto1_2,Auto1_3,Auto1_4,Auto2,Auto3,Super1_5,Super1_6,Super1_7,Super1_8,Super2,Super3,Tel1_5,Tel1_8,Tel1_11,Tel1_12,Tel2,Tel3,Arred1_5,Arred1_6,Arred1_7,Arred1_8,Arred2,Arred3,eComm1_5,eComm1_6,eComm1_9,eComm1_10,eComm2,eComm3,cosm1_5,cosm1_6,cosm1_7,cosm1_8,Cosm2,Cosm3,Valutaz1_1,Valutaz1_2,Valutaz1_3,Valutaz1_4,Valutaz1_5,Protocol,Vision
0,2024-04-16 20:29:30,2024-04-16 20:31:59,IP Address,5.77.66.30,100,149,True,2024-04-16 20:32:00.698000,R_4hmxpmQTtI65WmR,NaN,NaN,NaN,NaN,43.1479,12.1097,anonymous,EN,Test,NaN,NaN,NaN,NaN,NaN,Sì,"Prodotti alimentari,Bevande",NaN,Prodotti da frigo,NaN,Fdgg,NaN,NaN,NaN,sì,NaN,NaN,NaN,Frrtf,NaN,NaN,NaN,Sì,NaN,NaN,NaN,NaN,NaN,NaN,Sustenium,NaN,NaN,NaN,NaN,NaN,"Audi,Alfa Romeo",NaN,NaN,NaN,NaN,Sì,"Coop,Conad",NaN,NaN,NaN,NaN,Sì,"Fastweb,Wind/Tre",NaN,NaN,NaN,NaN,No,"Ikea,Leroy Merlin",NaN,NaN,NaN,NaN,NaN,Amazon,NaN,NaN,NaN,NaN,No,"Dolce e Gabbana,Armani Perfume",4,4,3,3,3,1,1.0
1,2024-04-16 20:43:07,2024-04-16 20:51:22,IP Address,87.13.79.7,100,494,True,2024-04-16 20:51:22.799000,R_2zZbbaKpyNCdMRj,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,vvv,NaN,NaN,NaN,NaN,NaN,Sì,"Prodotti alimentari,Supermercati",NaN,Pasta,NaN,NaN,NaN,NaN,NaN,NaN,"Barilla,Cirio",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BMW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Vodafone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,4,4,1,1.0
2,2024-04-16 20:51:30,2024-04-16 20:56:15,IP Address,87.13.79.7,100,284,True,2024-04-16 20:56:15.934000,R_2MQY4nKLInCEux5,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,rrrrr,NaN,NaN,NaN,NaN,NaN,Sì,"Bevande,Supermercati",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cirio,"Bibite gassate,Vino",NaN,ccc,NaN,NaN,NaN,Sì,NaN,NaN,NaN,NaN,NaN,NaN,Aspirina,NaN,NaN,NaN,NaN,NaN,Audi,NaN,NaN,NaN,NaN,No,Coop,NaN,NaN,NaN,NaN,No,Wind/Tre,NaN,NaN,NaN,NaN,No,Leroy Merlin,NaN,NaN,NaN,NaN,NaN,Ebay,NaN,NaN,NaN,NaN,No,Chanel,1,5,3,3,5,1,1.0
3,2024-04-16 23:02:17,2024-04-16 23:06:55,IP Address,2.233.164.46,100,277,True,2024-04-16 23:06:55.879000,R_3DTfCcKYyGQuLlb,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,s01,NaN,NaN,NaN,NaN,NaN,Sì,"Prodotti alimentari,Automobili,Cosmetica e profumeria",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3,4,5,4,1,1.0
4,2024-04-16 23:07:01,2024-04-16 23:07:21,IP Address,2.233.164.46,100,20,True,2024-04-16 23:07:22.143000,R_8qpx9l737tJXHza,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,s12,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,2024-06-03 19:05:14,2024-06-03 19:12:19,IP Address,93.51.247.100,100,425,True,2024-06-03 19:12:20.274000,R_2Rz7P8eB6ZoCEao,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,s1323,7,6,5,8,10,Sì,"Prodotti alimentari,Integratori e farmaci,Automobili,Altro (specificare)",PASTA S

### Calculate the values of the Unaided Brand Recall

In [ ]:
# Columns to check in df_post
columns_to_check = ['Alim2_5', 'Alim2_6', 'Alim2_7', 'Alim2_8', 'Bev2_2', 'Bev2_3', 'Bev2_4', 'Bev2_5',
                    'Integr1_5', 'Integr1_6', 'Integr1_9', 'Integr1_10', 'Auto1_1', 'Auto1_2', 'Auto1_3', 'Auto1_4',
                    'Super1_5', 'Super1_6', 'Super1_7', 'Super1_8', 'Tel1_5', 'Tel1_8', 'Tel1_11', 'Tel1_12',
                    'Arred1_5', 'Arred1_6', 'Arred1_7', 'Arred1_8', 'eComm1_5', 'eComm1_6', 'eComm1_9', 'eComm1_10',
                    'cosm1_5', 'cosm1_6', 'cosm1_7', 'cosm1_8']

for i in columns_to_check:
    print(df_post[f'{i}'].unique())

['Fdgg' nan 'Barilla' 'xxxx' 'cc' 'bistefani' 'Ciao' 'rwerwe' 'coca-cola'
 'ere' 'pastaio di nuoro' 'Mutti' 'òlkjhlk' 'fff' 'Fregola Sarda?'
 'Il Pastaio di Nuoro' 'coop' 'mulino bianco' 'coca cola' 'no' 'mutti'
 'MUTTI' 'PASTAIO NUORO' 'mulino bianco le macine' 'orasi'
 'il pastaio di nuoro' 'ora si' 'Fregola sarda' 'pastaio ci nuoro'
 'Mulino Bianco' 'rummo' 'Macine' 'pasta di nuoro' 'granoro bio'
 'pasta nuoro' 'pastificio di Nuoro' 'OraSì' 'noto' 'coca' 'MULINO BIANCO'
 'mulino' 'LETE' 'Pippo' 'eee' 'PIPPO' 'acqua lete' 'lete' 'FECOLA SARDA'
 'COOP' 'ALPRO' 'PASTA DI SASSARI' 'nuoro' 'Pastaio di Nuoro'
 'molticentrum' 'fregole' 'mitti' 'PASTA DELLA SARDEGNA' 'DI nuoro'
 'caseificio Nuoro' 'pastai di nuoro' 'fregole orto'
 'pasta nuoro (fregole)' 'PASTA DI NUORO' 'barilla' 'pastificio di nuoro'
 'il pastaio della nonna' 'Coca Cola' 'Pasta di Nuoro'
 "i sapori d'occidente" 'macine mulino bianco' 'copp' 'coop cibo per cani'
 'cocacola' 'programma gerry scotti' 'Pasta Nuoro' 'mulinp bi

In [ ]:
# Filter sensors_df to only include rows where Type == 'ad'
sensors_df = sensors_df[sensors_df['Type'] == 'ad'].copy()

# Now proceed with the rest of the script

# Preprocess df_post columns once outside the loop
df_post_processed = df_post.copy()

# List of columns to check
columns_to_check = [
    'Alim2_5', 'Alim2_6', 'Alim2_7', 'Alim2_8',
    'Bev2_2', 'Bev2_3', 'Bev2_4', 'Bev2_5',
    'Integr1_5', 'Integr1_6', 'Integr1_9', 'Integr1_10',
    'Auto1_1', 'Auto1_2', 'Auto1_3', 'Auto1_4',
    'Super1_5', 'Super1_6', 'Super1_7', 'Super1_8',
    'Tel1_5', 'Tel1_8', 'Tel1_11', 'Tel1_12',
    'Arred1_5', 'Arred1_6', 'Arred1_7', 'Arred1_8',
    'eComm1_5', 'eComm1_6', 'eComm1_9', 'eComm1_10',
    'cosm1_5', 'cosm1_6', 'cosm1_7', 'cosm1_8'
]

# Convert to lowercase and strip once
for col in columns_to_check:
    df_post_processed[col] = df_post_processed[col].fillna('').str.lower().str.strip()

# Combine all columns into a single text field for easier searching
df_post_processed['combined_text'] = df_post_processed[columns_to_check].apply(lambda x: ' '.join(x), axis=1)

# Create a dictionary keyed by (ID, Vision) for fast lookup
post_dict = {}
for (pid, vision), group in df_post_processed.groupby(['ID', 'Vision']):
    # Concatenate all rows for this (ID, Vision)
    combined_text_for_key = ' '.join(group['combined_text'].tolist())
    post_dict[(pid, vision)] = combined_text_for_key

# Define keywords for each label
keywords = {
    'FREGOLA': ['nuoro'],
    'LETE': ['lete'],
    'COOP': ['coop', 'cop'],
    'FASTWEB': ['fastweb'],
    'MULTICENTRUM': ['multi'],
    'ORASI': ['oras', 'ora s', 'orosi'],
    'MUTTI': ['mutti', 'muti', 'mitti'],
    'BARILLA': ['barilla', 'mulino', 'bianco', 'barila'],
    'COCA COLA': ['coca', 'cola'],
    'IKEA': ['ikea'],
    'AMAZON': ['amazon', 'amason'],
    'CHANEL': ['chanel'],
    'ALFA ROMEO': ['alfa', 'tonale'],
}

# Initialize Unaided Recall with 0 for all (remaining) ads
sensors_df['Unaided Recall'] = 0

# Iterate over each ad row in sensors_df and check for keywords in the combined text
for index, row in sensors_df.iterrows():
    key = (row['ID'], row['Vision'])
    if key in post_dict:
        text = post_dict[key]
        label_keywords = [kw.strip().lower() for kw in keywords.get(row['Content'], [])]
        # Check if any keyword is present in the combined text
        if any(kw in text for kw in label_keywords):
            sensors_df.at[index, 'Unaided Recall'] = 1


In [ ]:
unique_values = sensors_df['Unaided Recall'].unique()
print(unique_values)



[1 0]


In [ ]:
# Filter the rows where Unaided Recall is NaN
nan_rows = sensors_df[sensors_df['Unaided Recall'].isna()]

# Extract unique IDs from those rows
ids_with_nan = nan_rows['ID'].unique()

print(ids_with_nan)


[]


In [ ]:
# ### Vecchio per Unaided Recall

# # Columns to check in df_post
# columns_to_check = ['Alim2_5', 'Alim2_6', 'Alim2_7', 'Alim2_8', 'Bev2_2', 'Bev2_3', 'Bev2_4', 'Bev2_5',
#                     'Integr1_5', 'Integr1_6', 'Integr1_9', 'Integr1_10', 'Auto1_1', 'Auto1_2', 'Auto1_3', 'Auto1_4',
#                     'Super1_5', 'Super1_6', 'Super1_7', 'Super1_8', 'Tel1_5', 'Tel1_8', 'Tel1_11', 'Tel1_12',
#                     'Arred1_5', 'Arred1_6', 'Arred1_7', 'Arred1_8', 'eComm1_5', 'eComm1_6', 'eComm1_9', 'eComm1_10',
#                     'cosm1_5', 'cosm1_6', 'cosm1_7', 'cosm1_8']

# # Define keywords for each label
# keywords = {
#     'FREGOLA': ['nuoro'],
#     'LETE': ['lete'],
#     'COOP': ['coop', 'cop'],
#     'FASTWEB': ['fastweb'],
#     'MULTICENTRUM': ['multi'],
#     'ORASI': ['oras', 'ora s', 'orosi'],
#     'MUTTI': ['mutti', 'muti', 'mitti'],
#     'BARILLA': ['barilla', 'mulino', 'bianco', 'barila'],
#     'COCA COLA': ['coca', 'cola'],
#     'IKEA': ['ikea'],
#     'AMAZON': ['amazon', 'amason'],
#     'CHANEL': ['chanel'],
#     'ALFA ROMEO': ['alfa', 'tonale'],
# }

# # Iterate over each row in sensors_df
# for index, row in sensors_df[sensors_df["Type"] == "ad"].iterrows():

#     # Find the corresponding row in df_post based on 'ID' and 'Vision'
#     match_row = df_post[(df_post['ID'] == row['ID']) & (df_post['Protocol'] == row['Vision'])]

#     if not match_row.empty:
#         # Initialize the column
#         sensors_df.at[index, 'Unaided Recall'] = 0
#         # Fill NaN values with empty strings
#         match_row = match_row.fillna('')
#         # Extract keywords for the label, trim spaces, and convert to lowercase
#         label_keywords = [keyword.strip().lower() for keyword in keywords.get(row['Content'], [])]
#         # Check if any keyword of the label is contained in any of the specified columns of df_post
#         for keyword in label_keywords:

#             for col in columns_to_check:
#                 # Trim spaces and convert to lowercase before performing substring matching
#                 match_row[col] = match_row[col].str.strip().str.lower()
#                 if match_row[col].str.contains(keyword).any():
#                     # If any keyword is found, set 'Unaided Recall' to 1
#                     sensors_df.at[index, 'Unaided Recall'] = 1
#                     # Extract the matched string from the Series and print it
#                     matched_strings = match_row[col][match_row[col].str.contains(keyword)]
#                     break
#             else:
#                 continue  # Continue to the next keyword if no match is found
#             break  # Break the loop if a match is found for any keyword

# sensors_df[sensors_df["Type"] == "ad"].sample(5)

In [ ]:
# #### Nuovo piu veloce per Unaided Recall

# # Columns to check in df_post
# columns_to_check = ['Alim2_5', 'Alim2_6', 'Alim2_7', 'Alim2_8', 'Bev2_2', 'Bev2_3', 'Bev2_4', 'Bev2_5',
#                     'Integr1_5', 'Integr1_6', 'Integr1_9', 'Integr1_10', 'Auto1_1', 'Auto1_2', 'Auto1_3', 'Auto1_4',
#                     'Super1_5', 'Super1_6', 'Super1_7', 'Super1_8', 'Tel1_5', 'Tel1_8', 'Tel1_11', 'Tel1_12',
#                     'Arred1_5', 'Arred1_6', 'Arred1_7', 'Arred1_8', 'eComm1_5', 'eComm1_6', 'eComm1_9', 'eComm1_10',
#                     'cosm1_5', 'cosm1_6', 'cosm1_7', 'cosm1_8']

# # Define keywords for each label
# keywords = {
#     'FREGOLA': ['nuoro'],
#     'LETE': ['lete'],
#     'COOP': ['coop', 'cop'],
#     'FASTWEB': ['fastweb'],
#     'MULTICENTRUM': ['multi'],
#     'ORASI': ['oras', 'ora s', 'orosi'],
#     'MUTTI': ['mutti', 'muti', 'mitti'],
#     'BARILLA': ['barilla', 'mulino', 'bianco', 'barila'],
#     'COCA COLA': ['coca', 'cola'],
#     'IKEA': ['ikea'],
#     'AMAZON': ['amazon', 'amason'],
#     'CHANEL': ['chanel'],
#     'ALFA ROMEO': ['alfa', 'tonale'],
# }

# # Pre-elabora df_post: rendi tutte le colonne da controllare in minuscolo e senza spazi
# for col in columns_to_check:
#     df_post[col] = df_post[col].fillna('').str.strip().str.lower()

# # Crea una colonna concatenata per facilitare la ricerca
# df_post['all_columns'] = df_post[columns_to_check].apply(lambda x: ' '.join(x), axis=1)

# # Funzione per calcolare 'Unaided Recall' su un singolo contenuto
# def calculate_unaided_recall(row):
#     # Trova le righe di df_post che corrispondono a 'ID' e 'Vision'
#     match_rows = df_post[(df_post['ID'] == row['ID']) & (df_post['Protocol'] == row['Vision'])]

#     if not match_rows.empty:
#         # Recupera parole chiave del contenuto
#         label_keywords = [kw.strip().lower() for kw in keywords.get(row['Content'], [])]
#         # Verifica se uno dei keyword è presente in 'all_columns'
#         for keyword in label_keywords:
#             if match_rows['all_columns'].str.contains(keyword).any():
#                 return 1  # Unaided Recall trovato
#     return 0  # Nessun match trovato

# # Applica la funzione a tutte le righe di interesse
# sensors_df.loc[sensors_df['Type'] == 'ad', 'Unaided Recall'] = sensors_df[sensors_df['Type'] == 'ad'].apply(
#     calculate_unaided_recall, axis=1
# )

### Calculate the values of the Aided Brand Recall

In [ ]:
# #### Nuovo piu veloce per Aided Recall

# # Columns to check in df_post
# columns_to_check = ['Alim4','Bev4','Integr3','Auto3','Super3','Tel3','Arred3','eComm3','Cosm3']

# # Define keywords for each label
# keywords = {
#     'FREGOLA': ['Il Pastaio di Nuoro'],
#     'LETE': ['Acqua Lete'],
#     'COOP': ['Coop'],
#     'FASTWEB': ['Fastweb'],
#     'MULTICENTRUM': ['Multicentrum'],
#     'ORASI': ['Orasì'],
#     'MUTTI': ['Mutti'],
#     'BARILLA': ['Barilla'],
#     'COCA COLA': ['Coca Cola'],
#     'IKEA': ['Ikea'],
#     'AMAZON': ['Amazon'],
#     'CHANEL': ['Chanel'],
#     'ALFA ROMEO': ['Alfa Romeo'],
# }

# # Pre-processa le colonne specificate: rimuove spazi e porta tutto in minuscolo
# for col in columns_to_check:
#     df_post[col] = df_post[col].fillna('').str.strip().str.lower()

# # Crea una colonna concatenata con tutte le colonne da controllare
# df_post['all_columns'] = df_post[columns_to_check].apply(lambda x: ' '.join(x), axis=1)

# # Funzione per calcolare 'Aided Recall' su una riga specifica
# def calculate_aided_recall(row):
#     # Filtra le righe corrispondenti in df_post basate su 'ID' e 'Vision'
#     match_rows = df_post[(df_post['ID'] == row['ID']) & (df_post['Vision'] == row['Vision'])]

#     if not match_rows.empty:
#         # Parole chiave associate al 'Content' della riga corrente
#         label_keywords = [kw.strip().lower() for kw in keywords.get(row['Content'], [])]

#         # Controlla se almeno una parola chiave è contenuta nella colonna concatenata
#         for keyword in label_keywords:
#             if match_rows['all_columns'].str.contains(keyword).any():
#                 return 1  # Match trovato, restituisce 1
#     return 0  # Nessun match trovato

# # Applica la funzione alle righe di interesse in sensors_df
# sensors_df.loc[sensors_df['Type'] == 'ad', 'Aided Recall'] = sensors_df[sensors_df['Type'] == 'ad'].apply(
#     calculate_aided_recall, axis=1
# )


In [ ]:
# First, ensure sensors_df is filtered to Type == 'ad' if you want to run only on ads
# sensors_df = sensors_df[sensors_df['Type'] == 'ad'].copy()

# List of columns to check for Aided Recall
columns_to_check = ['Alim4', 'Bev4', 'Integr3', 'Auto3', 'Super3', 'Tel3', 'Arred3', 'eComm3', 'Cosm3']

# Make a copy of df_post to avoid modifying the original
df_post_processed = df_post.copy()

# Convert to lowercase and strip once
for col in columns_to_check:
    df_post_processed[col] = df_post_processed[col].fillna('').str.lower().str.strip()

# Combine all columns into a single text field for easier searching
df_post_processed['combined_text'] = df_post_processed[columns_to_check].apply(lambda x: ' '.join(x), axis=1)

# Create a dictionary keyed by (ID, Vision) for fast lookup
post_dict_aided = {}
for (pid, vision), group in df_post_processed.groupby(['ID', 'Vision']):
    # Concatenate all rows for this (ID, Vision)
    combined_text_for_key = ' '.join(group['combined_text'].tolist())
    post_dict_aided[(pid, vision)] = combined_text_for_key

# Define keywords for each label (lowercase them to match processed text)
keywords = {
    'FREGOLA': ['il pastaio di nuoro'],
    'LETE': ['acqua lete'],
    'COOP': ['coop'],
    'FASTWEB': ['fastweb'],
    'MULTICENTRUM': ['multicentrum'],
    'ORASI': ['orasì'],
    'MUTTI': ['mutti'],
    'BARILLA': ['barilla'],
    'COCA COLA': ['coca cola'],
    'IKEA': ['ikea'],
    'AMAZON': ['amazon'],
    'CHANEL': ['chanel'],
    'ALFA ROMEO': ['alfa romeo'],
}

# Initialize Aided Recall with 0 for all ads (if not already set)
sensors_df['Aided Recall'] = 0

# Iterate over each ad row and check for Aided Recall keywords
for index, row in sensors_df.iterrows():
    key = (row['ID'], row['Vision'])
    if key in post_dict_aided:
        text = post_dict_aided[key]
        label_keywords = [kw.strip().lower() for kw in keywords.get(row['Content'], [])]

        # Check if any keyword is present in the combined text
        if any(kw in text for kw in label_keywords):
            sensors_df.at[index, 'Aided Recall'] = 1


In [ ]:
unique_values = sensors_df['Aided Recall'].unique()
print(unique_values)



[1 0]


In [ ]:
# ### Vecchio per Aided Recall

# # Columns to check in df_post
# columns_to_check = ['Alim4','Bev4','Integr3','Auto3','Super3','Tel3','Arred3','eComm3','Cosm3']

# # Define keywords for each label
# keywords = {
#     'FREGOLA': ['Il Pastaio di Nuoro'],
#     'LETE': ['Acqua Lete'],
#     'COOP': ['Coop'],
#     'FASTWEB': ['Fastweb'],
#     'MULTICENTRUM': ['Multicentrum'],
#     'ORASI': ['Orasì'],
#     'MUTTI': ['Mutti'],
#     'BARILLA': ['Barilla'],
#     'COCA COLA': ['Coca Cola'],
#     'IKEA': ['Ikea'],
#     'AMAZON': ['Amazon'],
#     'CHANEL': ['Chanel'],
#     'ALFA ROMEO': ['Alfa Romeo'],
# }

# # Iterate over each row in sensors_df
# for index, row in sensors_df[sensors_df["Type"] == "ad"].iterrows():

#     # Find the corresponding row in df_post based on 'ID' and 'Vision'
#     match_row = df_post[(df_post['ID'] == row['ID']) & (df_post['Vision'] == row['Vision'])]

#     if not match_row.empty:
#         # Initialize the column
#         sensors_df.at[index, 'Aided Recall'] = 0
#         # Fill NaN values with empty strings
#         match_row = match_row.fillna('')
#         # Extract keywords for the label, trim spaces, and convert to lowercase
#         label_keywords = [keyword.strip().lower() for keyword in keywords.get(row['Content'], [])]
#         # Check if any keyword of the label is contained in any of the specified columns of df_post
#         for keyword in label_keywords:
#             for col in columns_to_check:
#                 # Trim spaces and convert to lowercase before performing substring matching
#                 match_row[col] = match_row[col].str.strip().str.lower()
#                 if match_row[col].str.contains(keyword).any():
#                     # If any keyword is found, set 'Unaided Recall' to 1
#                     sensors_df.at[index, 'Aided Recall'] = 1
#                     # Extract the matched string from the Series and print it
#                     matched_strings = match_row[col][match_row[col].str.contains(keyword)]
#                     break
#             else:
#                 continue  # Continue to the next keyword if no match is found
#             break  # Break the loop if a match is found for any keyword

# sensors_df[sensors_df["Type"] == "ad"].sample(5)

In [ ]:
# Get rid of observations for which the survey is missing
sensors_df = sensors_df[~((sensors_df['ID'] == "s171") & (sensors_df['Vision'] == 2))]
sensors_df = sensors_df[~((sensors_df['ID'] == "s257") & (sensors_df['Vision'] == 2))]
sensors_df = sensors_df[~((sensors_df['ID'] == "s303") & (sensors_df['Vision'] == 2))]

### Content Appreciation (sta cagata si può fare meglio secondo me)

In [ ]:
# Add column Content Appreciation to df_post, which is the first principal component calculated on the columns regarding the content
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Ensure columns 'Q1_1' to 'Q1_5' are numeric, converting non-numeric values to NaN
df_post.loc[:, 'Q1_1':'Q1_5'] = df_post.loc[:, 'Q1_1':'Q1_5'].apply(pd.to_numeric, errors='coerce')

# Fill NaN values with the mean of each column
df_post.loc[:, 'Q1_1':'Q1_5'] = df_post.loc[:, 'Q1_1':'Q1_5'].apply(lambda col: col.fillna(col.mean()), axis=0)

df_post



<ipython-input-66-5911e29aba66>:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_post.loc[:, 'Q1_1':'Q1_5'] = df_post.loc[:, 'Q1_1':'Q1_5'].apply(lambda col: col.fillna(col.mean()), axis=0)


,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,ID,Q1_1,Q1_2,Q1_3,Q1_4,Q1_5,Q3,Q5,Q5_8_TEXT,Alim1,Alim1_6_TEXT,Alim2_5,Alim2_6,Alim2_7,Alim2_8,Alim3,Alim4,Bev1,Bev1_7_TEXT,Bev2_2,Bev2_3,Bev2_4,Bev2_5,Bev3,Bev4,Integr1_5,Integr1_6,Integr1_9,Integr1_10,Integr2,Integr3,Auto1_1,Auto1_2,Auto1_3,Auto1_4,Auto2,Auto3,Super1_5,Super1_6,Super1_7,Super1_8,Super2,Super3,Tel1_5,Tel1_8,Tel1_11,Tel1_12,Tel2,Tel3,Arred1_5,Arred1_6,Arred1_7,Arred1_8,Arred2,Arred3,eComm1_5,eComm1_6,eComm1_9,eComm1_10,eComm2,eComm3,cosm1_5,cosm1_6,cosm1_7,cosm1_8,Cosm2,Cosm3,Valutaz1_1,Valutaz1_2,Valutaz1_3,Valutaz1_4,Valutaz1_5,Protocol,Vision
0,2024-04-16 20:29:30,2024-04-16 20:31:59,IP Address,5.77.66.30,100,149,True,2024-04-16 20:32:00.698000,R_4hmxpmQTtI65WmR,NaN,NaN,NaN,NaN,43.1479,12.1097,anonymous,EN,Test,7.660668,7.663519,7.717839,7.66179,7.944158,Sì,"Prodotti alimentari,Bevande",NaN,Prodotti da frigo,NaN,Fdgg,NaN,NaN,NaN,sì,NaN,NaN,NaN,Frrtf,NaN,NaN,NaN,Sì,NaN,NaN,NaN,NaN,NaN,NaN,Sustenium,NaN,NaN,NaN,NaN,NaN,"Audi,Alfa Romeo",NaN,NaN,NaN,NaN,Sì,"Coop,Conad",NaN,NaN,NaN,NaN,Sì,"Fastweb,Wind/Tre",NaN,NaN,NaN,NaN,No,"Ikea,Leroy Merlin",NaN,NaN,NaN,NaN,NaN,Amazon,NaN,NaN,NaN,NaN,No,"Dolce e Gabbana,Armani Perfume",4,4,3,3,3,1,1.0
1,2024-04-16 20:43:07,2024-04-16 20:51:22,IP Address,87.13.79.7,100,494,True,2024-04-16 20:51:22.799000,R_2zZbbaKpyNCdMRj,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,vvv,7.660668,7.663519,7.717839,7.66179,7.944158,Sì,"Prodotti alimentari,Supermercati",NaN,Pasta,NaN,NaN,NaN,NaN,NaN,NaN,"Barilla,Cirio",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BMW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Vodafone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,4,4,1,1.0
2,2024-04-16 20:51:30,2024-04-16 20:56:15,IP Address,87.13.79.7,100,284,True,2024-04-16 20:56:15.934000,R_2MQY4nKLInCEux5,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,rrrrr,7.660668,7.663519,7.717839,7.66179,7.944158,Sì,"Bevande,Supermercati",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cirio,"Bibite gassate,Vino",NaN,ccc,NaN,NaN,NaN,Sì,NaN,NaN,NaN,NaN,NaN,NaN,Aspirina,NaN,NaN,NaN,NaN,NaN,Audi,NaN,NaN,NaN,NaN,No,Coop,NaN,NaN,NaN,NaN,No,Wind/Tre,NaN,NaN,NaN,NaN,No,Leroy Merlin,NaN,NaN,NaN,NaN,NaN,Ebay,NaN,NaN,NaN,NaN,No,Chanel,1,5,3,3,5,1,1.0
3,2024-04-16 23:02:17,2024-04-16 23:06:55,IP Address,2.233.164.46,100,277,True,2024-04-16 23:06:55.879000,R_3DTfCcKYyGQuLlb,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,s01,7.660668,7.663519,7.717839,7.66179,7.944158,Sì,"Prodotti alimentari,Automobili,Cosmetica e profumeria",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3,4,5,4,1,1.0
4,2024-04-16 23:07:01,2024-04-16 23:07:21,IP Address,2.233.164.46,100,20,True,2024-04-16 23:07:22.143000,R_8qpx9l737tJXHza,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,s12,7.660668,7.663519,7.717839,7.66179,7.944158,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,2024-06-03 19:05:14,2024-06-03 19:12:19,IP Address,93.51.247.100,100,425,True,2024-06-03 19:12:20.274000,R_2Rz7P8eB6ZoCEao,NaN,NaN,NaN,NaN,45.4722,9.

In [ ]:
#QUELLO DEL FIL
# # Standardize columns
# scaler = StandardScaler()
# df_post_standardized = scaler.fit_transform(df_post[['Q1_1', 'Q1_2', 'Q1_3', 'Q1_4', 'Q1_5']])

# # Perform PCA
# pca = PCA(n_components=1)
# first_principal_component = pca.fit_transform(df_post_standardized)

# # Ensure the loadings are positive
# if pca.components_[0].mean() < 0:
#     pca.components_ *= -1
#     first_principal_component *= -1

# # Print percentage explained by the first component
# print("Percentage Explained by the First Component:", pca.explained_variance_ratio_[0])

# # Add the first principal component to the DataFrame
# df_post['Content Appreciation'] = first_principal_component

# # Get the weights (loadings) of the PCA
# pca_loadings = pca.components_[0]
# pca_loadings_df = pd.DataFrame(pca_loadings, index=['Q1_1', 'Q1_2', 'Q1_3', 'Q1_4', 'Q1_5'], columns=['PCA Loading'])

# print("\nPCA Loadings:")
# print(pca_loadings_df)

In [ ]:
#New content appreciation, computed as avg of the first 4 questions

df_post['Content Appreciation'] = df_post[['Q1_1', 'Q1_2', 'Q1_3', 'Q1_4']].mean(axis=1) #escludiamo la Q1_5 perchè meno rappresentativa di quanto il contenuto sia piaciuto (è quella sullo stage)

## Salvo come excel cosi non devo runnare tutto

In [ ]:
sensors_df_path = 'sensors_df.csv'
df_post_path = 'df_post.csv'

# Save DataFrames to CSV files
sensors_df.to_csv(sensors_df_path, index=False)
df_post.to_csv(df_post_path, index=False)

print(f"sensors_df saved to {sensors_df_path}")
print(f"df_post saved to {df_post_path}")

sensors_df saved to sensors_df.csv
df_post saved to df_post.csv


In [ ]:
vision2 = pd.read_csv('Vision2.csv')
sensors_df = pd.read_csv('sensors_df.csv')
df_post = pd.read_csv('df_post.csv')

<ipython-input-70-6d8b5bbb21ff>:2: DtypeWarning: Columns (112,123) have mixed types. Specify dtype option on import or set low_memory=False.
  sensors_df = pd.read_csv('sensors_df.csv')


Su df_post voglio mettere le colonne di Vision2.csv per fare poi il merge con sensors_df.


In [ ]:
# Add Content Appreciation in sensors_df
sensors_df = sensors_df.merge(df_post[['ID', 'Vision', 'Content Appreciation']],
                              on = ['ID', 'Vision'])

In [ ]:
len(sensors_df['ID'].unique())

563

### Clean the Contents names and add the category

In [ ]:
# Clean names
sensors_df['Content'] = sensors_df['Content'].str.replace('_A', '').str.replace('_B', '')

sensors_df['Content'] = sensors_df['Content'].replace({
    'grande fratello': 'Grande fratello',
    'In viaggio con barbero': 'In viaggio con Barbero',
    'Alberto Angela - Stanotte': 'Stanotte a Parigi',
    'Splendida_cornice': 'Splendida Cornice',
    'Master and commander': 'Master and Commander',
    'Sky TG24 Wow': 'Sky TG24 wow',
    'Lolita lobosco': 'Lolita Lobosco',
    'Il cavallo e la torre': 'Il cavallo e la Torre',
    'Splendida cornice': 'Splendida Cornice',
    'Quarta repubblica': 'Quarta Repubblica',
    'The Equalizer': 'The equalizer',
    'Terra amara': 'Terra Amara',
    'Sky Calcio Club': 'Sky calcio club',
    'dritto e rovescio': 'Diritto e rovescio',
    'Sky TG24 Numeri': 'Sky TG24 numeri',
    'Natura meraviglia senza tempo': 'natura meraviglia senza tempo'
})
sensors_df['Content'] = sensors_df['Content'].str.replace('_A', '').str.replace('_B', '')
sensors_df

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Asymmetry Delta,Asymmetry Theta,Mean Frontal Asymmetry Alpha,Asymmetry Beta,Asymmetry Gamma,Device,Protocol,Vision,Position,Type,Engagement,Engagement(1/alpha),FAA,Pleasantness Index,Pleasantness,Attention,AW Index,Vamv Valence,Kirk Valence,Ram12 Valence,Ram15 Valence,Choice Index (Gamma),Choice Index (Beta),Effort Index 1,Effort Index 2,Workload_mean,Workload_mean2,SASI,Engagement standardized,Pleasantness standardized,Attention standardized,FAA standardized,Engagement(1/alpha) standardized,Asymmetry Delta standardized,Asymmetry Theta standardized,Asymmetry Beta standardized,Asymmetry Gamma standardized,Beta standardized,Alpha standardized,Gamma standardized,Delta standardized,AW Index standardized,Vamv Valence standardized,Kirk Valence standardized,Ram12 Valence standardized,Ram15 Valence standardized,Choice Index (Gamma) standardized,Choice Index (Beta) standardized,Effort Index 1 standardized,Effort Index 2 standardized,Workload_mean standardized,Workload_mean2 standardized,SASI standardized,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,DOB,Gender,Unaided Recall,Aided Recall,Content Appreciation
0,s10,406001.0,500,ORASI,Subdivision 1,0.0,0.0,NaN,51.274331,4.421809,0.656750,0.554243,0.052481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.404432,1.180864,0.352452,0.162106,0.077571,NaN,NaN,NaN,NaN,NaN,60.841512,2.801560,0.504545,0.358179,0.065028,0.317,-1.320,-0.622,-1.229,0.391,Laptop,Linear_120,2.0,2.0,ad,0.108339,2.971,-2.703,-3.4938,-0.622,4.474,-0.301523,0.383979,NaN,-0.201791,0.989253,NaN,NaN,-0.578464,NaN,0.801928,0.838789,-0.773286,-0.451573,-0.500692,-0.114722,0.288997,1.002783,0.414855,-1.615264,-2.316575,0.375997,-0.802676,-1.002783,-1.151474,0.715618,0.205021,0.936126,NaN,-0.316300,0.569383,NaN,NaN,-0.512294,NaN,-0.190858,0.150995,-0.582985,2024-04-17 15:50:34,2024-04-17 15:57:46,IP Address,93.51.247.100,100.0,431.0,True,2024-04-17 15:57:46.450000,R_5dMsXsUPvJePDAo,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,1974,Femmina,1,1,7.675954
1,s10,406501.0,500,ORASI,Subdivision 2,0.0,0.0,NaN,NaN,NaN,3.033891,0.291541,0.029242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.202454,4.680584,0.410677,0.217270,0.054413,NaN,NaN,NaN,NaN,NaN,135.737548,NaN,1.722265,0.254390,0.041831,-1.092,NaN,NaN,-0.294,0.621,Laptop,Linear_120,2.0,2.0,ad,0.006328,-2.361,-8.685,-6.4626,NaN,15.852,-0.761551,-0.432959,NaN,0.119135,NaN,NaN,NaN,-0.878333,NaN,0.383641,0.171995,-0.986864,-1.400210,NaN,2

In [ ]:
# Import file with categories
categories_df = pd.read_excel('Categorie contenuti.xlsx').drop_duplicates()
sensors_df = sensors_df.merge(categories_df, on = 'Content', how = 'left')

In [ ]:
# Add the category of the content also for the ads
categories = {}
for idx, row in sensors_df.iterrows():
    if row['Type'] == 'content_A':
        categories[(row['ID'], row['Vision'])] = row['Category']

# Update the 'Category' for rows where Type is 'ad'
for idx, row in sensors_df.iterrows():
    if row['Type'] == 'ad':
        sensors_df.at[idx, 'Category'] = categories.get((row['ID'], row['Vision']))
sensors_df

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Asymmetry Delta,Asymmetry Theta,Mean Frontal Asymmetry Alpha,Asymmetry Beta,Asymmetry Gamma,Device,Protocol,Vision,Position,Type,Engagement,Engagement(1/alpha),FAA,Pleasantness Index,Pleasantness,Attention,AW Index,Vamv Valence,Kirk Valence,Ram12 Valence,Ram15 Valence,Choice Index (Gamma),Choice Index (Beta),Effort Index 1,Effort Index 2,Workload_mean,Workload_mean2,SASI,Engagement standardized,Pleasantness standardized,Attention standardized,FAA standardized,Engagement(1/alpha) standardized,Asymmetry Delta standardized,Asymmetry Theta standardized,Asymmetry Beta standardized,Asymmetry Gamma standardized,Beta standardized,Alpha standardized,Gamma standardized,Delta standardized,AW Index standardized,Vamv Valence standardized,Kirk Valence standardized,Ram12 Valence standardized,Ram15 Valence standardized,Choice Index (Gamma) standardized,Choice Index (Beta) standardized,Effort Index 1 standardized,Effort Index 2 standardized,Workload_mean standardized,Workload_mean2 standardized,SASI standardized,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,DOB,Gender,Unaided Recall,Aided Recall,Content Appreciation,Category
0,s10,406001.0,500,ORASI,Subdivision 1,0.0,0.0,NaN,51.274331,4.421809,0.656750,0.554243,0.052481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.404432,1.180864,0.352452,0.162106,0.077571,NaN,NaN,NaN,NaN,NaN,60.841512,2.801560,0.504545,0.358179,0.065028,0.317,-1.320,-0.622,-1.229,0.391,Laptop,Linear_120,2.0,2.0,ad,0.108339,2.971,-2.703,-3.4938,-0.622,4.474,-0.301523,0.383979,NaN,-0.201791,0.989253,NaN,NaN,-0.578464,NaN,0.801928,0.838789,-0.773286,-0.451573,-0.500692,-0.114722,0.288997,1.002783,0.414855,-1.615264,-2.316575,0.375997,-0.802676,-1.002783,-1.151474,0.715618,0.205021,0.936126,NaN,-0.316300,0.569383,NaN,NaN,-0.512294,NaN,-0.190858,0.150995,-0.582985,2024-04-17 15:50:34,2024-04-17 15:57:46,IP Address,93.51.247.100,100.0,431.0,True,2024-04-17 15:57:46.450000,R_5dMsXsUPvJePDAo,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,1974,Femmina,1,1,7.675954,None
1,s10,406501.0,500,ORASI,Subdivision 2,0.0,0.0,NaN,NaN,NaN,3.033891,0.291541,0.029242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.202454,4.680584,0.410677,0.217270,0.054413,NaN,NaN,NaN,NaN,NaN,135.737548,NaN,1.722265,0.254390,0.041831,-1.092,NaN,NaN,-0.294,0.621,Laptop,Linear_120,2.0,2.0,ad,0.006328,-2.361,-8.685,-6.4626,NaN,15.852,-0.761551,-0.432959,NaN,0.119135,NaN,NaN,NaN,-0.878333,NaN,0.383641,0.171995,-0.986864,-

In [ ]:
len(sensors_df['ID'].unique())

563

In [ ]:
sensors_df.loc[sensors_df['ID'] == 's186', 'Gender'] = 'Femmina'
sensors_df.loc[sensors_df['ID'] == 's222', 'Gender'] = 'Femmina'

## DOB Cleaning

In [ ]:
# Ensure the column is of type string
sensors_df['DOB'] = sensors_df['DOB'].astype(str)
sensors_df['DOB'].unique()

array(['1974', '1978', '1970', '1967', '1962', '1958', '1960', '1956',
       '1986', '2001', '1976', '1959', '1964', '2000', '1954', '1965',
       '2005', '1979', '1973', '1977', '1950', '1957', '1961', '1997',
       '1993', '1989', '57', '1999', '1983', '1963', '1969', '1952',
       '1980', '1953', '1985', '1987', '1984', '4/6/58', '1992', '1972',
       '1975', '1982', '1966', '30/09/1996', '1994', '2002', '1988',
       '1955', '1998', '1991', '28.08.1965', '1990', '27 05 1958',
       '14/06/1973', '2003', '11/03/1995', '31\\12\\73', '02/05/1957',
       '1951', '2004', '26/01/1986', '10/11/2002', '1944', '14/06/1990',
       '27.01.1988', '1968', '1-08-1973', '8/1/1960', '1971',
       '15.05.1960', '2006', '33', 's268', '1948', '1984*', '2/4/85',
       '1995', '1996', '1981', '02/05/1970', '07/12/1970', '11-03-1951',
       '11/08/1953', '02/02/1996', '090964', '90964', '25/06/1965',
       '011 04 1960', '28  10  60', '1317', '1946'], dtype=object)

In [ ]:

# Replace values using the mapping dictionary
mapping_dict = {
    '28.08.1965': '1965',
    '30/09/1996': '1996',
    '10/11/2002': '2002',
    '14/06/1990': '1990',
    '27.01.1988': '1988',
    '1-08-1973': '1973',
    '15.05.1960': '1960',
    '2004.0': '2004',
    '1964.0': '1964',
    '2002.0': '2002',
    '1984.0': '1984',
    '1969.0': '1969',
    '1967.0': '1967',
    '1968.0': '1968',
    '1978.0': '1978',
    '1971.0': '1971',
    '1983.0': '1983',
    '1965.0': '1965',
    '2/4/85': '1985',
    '02/05/1970': '1970',
    '07/12/1970': '1970',
    '11-03-1951': '1951',
    '11/08/1953': '1953',
    '090964': '1964',
    '25/06/1965': '1965',
    '011 04 1960': '1960',
    '28  10  60': '1960',
    '57': '1957',
    '11/03/1995': '1995',
    '33':'1977',
    's268':'1977',
    '31\\12\\73':'1973',
    '27 05 1958':'1958',
    '14/06/1973':'1973',
    '02/05/1957':'1957',
    '26/01/1986':'1986',
    '8/1/1960':'1960',
    '1984*':'1984',
    '02/02/1996':'1996',
    '4/6/58':'1958'
}


# Apply the replacement
sensors_df['DOB'] = sensors_df['DOB'].replace(mapping_dict)

sensors_df.loc[sensors_df['ID'] == 's105', 'DOB'] = '2000'
sensors_df.loc[sensors_df['ID'] == 's318', 'DOB'] = '1960'

# Verify the replacement
print(sensors_df['DOB'].unique())

['1974' '1978' '1970' '1967' '1962' '1958' '1960' '1956' '1986' '2001'
 '1976' '1959' '1964' '2000' '1954' '1965' '2005' '1979' '1973' '1977'
 '1950' '1957' '1961' '1997' '1993' '1989' '1999' '1983' '1963' '1969'
 '1952' '1980' '1953' '1985' '1987' '1984' '1992' '1972' '1975' '1982'
 '1966' '1996' '1994' '2002' '1988' '1955' '1998' '1991' '1990' '2003'
 '1995' '1951' '2004' '1944' '1968' '1971' '2006' '1948' '1981' '90964'
 '1317' '1946']


In [ ]:
sensors_df.loc[sensors_df['DOB'] == '1317', 'DOB'] = '1977'

In [ ]:
print(sorted(sensors_df['DOB'].unique()))


['1944', '1946', '1948', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '90964']


In [ ]:
sensors_df.to_csv('data_cleaned_19122024.csv', index=False)

Note
1. le categorie dei contenuti per i VOD le hanno stabilite loro, per gli altri contenuti le ho decise io
2. per qualche ragione, nella visione di s237 numero 1, viene visto prima il pezzo B e poi il pezzo A della trasmissione
3. secondo lo scheduling dovrebbe esserci lo studio 1 del partecipante s302, ma è mancante
4. il 1285 ha le visioni Smarthpone, ma sullo scheduling è segnato come TV. Anche 1291, 1294, 1297. Invece 1091, 1284, 1287, 1293, 1299 sono segnati sullo scheduling come Smartphone e invece sono TV (ATTENZIONE, QUESTI POTREBBERO DOVER ESSERE RICLASSIFICATI, PERCHè MI RICORDO CHE ALCUNI LI AVEVAMO FATTI SU DISPOSITIVI DIVERSI DA QUELLI PROGRAMMATI)
5. s1222 VOD, s1225 YT, s1256 YT, s1294 YT  manca la registrazione (c'è solo una riga)
6. 1160 VOD, 1200 VOD e YT, 1203 VOD e YT, 1256 VOD manca un marker
7. da s1247 in avanti mancano i dati EEG


# ⬇⬇⬇ RI RUNNA DA QUI ⬇⬇⬇:


## Leggo le features

In [1]:
# Install only the third-party libraries
!pip install pandas numpy openpyxl

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os

# Percorso alla cartella desiderata
# Cambia la directory alla cartella desiderata
os.chdir('/content/drive/My Drive/Thesis/ANALISI_TESI')  # Modifica il percorso con il nome corretto


# Conferma che la directory è stata impostata
print("Directory corrente:", os.getcwd())

Mounted at /content/drive
Directory corrente: /content/drive/.shortcut-targets-by-id/1HL6A6j5Pa-pX7NaJjXlcoTsKKaU_jPJ0/Thesis/ANALISI_TESI


In [3]:
import pandas as pd
import os
import numpy as np
import re
pd.set_option('display.max_columns', None)

In [4]:
# Read the Excel file of the features
feature_df = pd.read_excel("Dynamic Features Dataset.xlsx", sheet_name='Sheet1')
feature_df.head()

,Frame,KEY,Frame_Num,ID,Ad name,VOLTI,TONALITÀ DOMINANTE CALDA,BRAND,PERSON SHOWN,CELEBRITY PRESENT,CHILDREN,GROUP OF PEOPLE,ANIMAL,HAPPY,SAD,ANGRY,CONFUSED,DISGUSTED,SURPRISED,CALM,FEAR,STORYTELLING,TONALITà DOMINANTE CALDA,DYNAMISM,NUTRITION,PRINCIPAL CHARACTER IS FEMALE,CELEBRITY PRESENT.1,CHILDREN.1,ANIMAL.1,CREATIVITY,FAMIGLIA E AMICI,ANIMATED
0,frame_1.jpg,ALFA ROMEO_1,1,ALFA TONALE_frame_1.jpg,ALFA ROMEO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0
1,frame_2.jpg,ALFA ROMEO_2,2,ALFA TONALE_frame_2.jpg,ALFA ROMEO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0
2,frame_3.jpg,ALFA ROMEO_3,3,ALFA TONALE_frame_3.jpg,ALFA ROMEO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0
3,frame_4.jpg,ALFA ROMEO_4,4,ALFA TONALE_frame_4.jpg,ALFA ROMEO,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0
4,frame_5.jpg,ALFA ROMEO_5,5,ALFA TONALE_frame_5.jpg,ALFA ROMEO,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,3.0,0,0


In [5]:
feature_df.drop(columns=['Frame','ID'], inplace=True)
feature_df

,KEY,Frame_Num,Ad name,VOLTI,TONALITÀ DOMINANTE CALDA,BRAND,PERSON SHOWN,CELEBRITY PRESENT,CHILDREN,GROUP OF PEOPLE,ANIMAL,HAPPY,SAD,ANGRY,CONFUSED,DISGUSTED,SURPRISED,CALM,FEAR,STORYTELLING,TONALITà DOMINANTE CALDA,DYNAMISM,NUTRITION,PRINCIPAL CHARACTER IS FEMALE,CELEBRITY PRESENT.1,CHILDREN.1,ANIMAL.1,CREATIVITY,FAMIGLIA E AMICI,ANIMATED
0,ALFA ROMEO_1,1,ALFA ROMEO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0
1,ALFA ROMEO_2,2,ALFA ROMEO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0
2,ALFA ROMEO_3,3,ALFA ROMEO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0
3,ALFA ROMEO_4,4,ALFA ROMEO,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0
4,ALFA ROMEO_5,5,ALFA ROMEO,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,3.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,ORASI_27,27,ORASI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0
518,ORASI_28,28,ORASI,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0
519,ORASI_29,29,ORASI,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0
520,ORASI_30,30,ORASI,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0


In [6]:
feature_df['Ad name'].unique()

array(['ALFA ROMEO', 'AMAZON', 'BARILLA', 'CHANEL', 'COCA COLA', 'COOP',
       'FASTWEB', 'IKEA', 'LETE', 'MULTICENTRUM', 'MUTTI', 'ORASI'],
      dtype=object)

## Leggo nuove features

In [7]:
# Percorso del file Excel
file_path = "final_features2.csv"


# Leggere il foglio specifico
try:
    final_features2 = pd.read_csv(file_path)
    # Mostrare i primi dati per conferma
    final_features2.head()
except Exception as e:
    e

In [8]:
# Mapping of video names to Ad names
video_to_ad_name_mapping = {
    'COCA COLA': 'COCA COLA',
    'ALFA TONALE': 'ALFA ROMEO',
    'BARILLA': 'BARILLA',
    'FASTWEB': 'FASTWEB',
    'Amazon': 'AMAZON',
    'COOP': 'COOP',
    'MULTICENTRUM': 'MULTICENTRUM',
    'LETE': 'LETE',
    'CHANEL': 'CHANEL',
    'IKEA ITALIA': 'IKEA',
    'MUTTI': 'MUTTI',
    'ORASI': 'ORASI'
}

# Assuming `final_features2` is the DataFrame where you want to apply the mapping
final_features2['video'] = final_features2['video'].map(video_to_ad_name_mapping)

# Print the unique values to confirm the mapping
print(final_features2['video'].unique())


['COCA COLA' 'ALFA ROMEO' 'BARILLA' 'FASTWEB' 'AMAZON' 'COOP'
 'MULTICENTRUM' 'LETE' 'CHANEL' 'IKEA' 'MUTTI' 'ORASI']


In [9]:
final_features2['video'].unique()

array(['COCA COLA', 'ALFA ROMEO', 'BARILLA', 'FASTWEB', 'AMAZON', 'COOP',
       'MULTICENTRUM', 'LETE', 'CHANEL', 'IKEA', 'MUTTI', 'ORASI'],
      dtype=object)

In [10]:
feature_df = pd.merge(feature_df, final_features2, left_on=['Ad name','Frame_Num'],right_on=['video','frame'], how='left')
feature_df

,KEY,Frame_Num,Ad name,VOLTI,TONALITÀ DOMINANTE CALDA,BRAND,PERSON SHOWN,CELEBRITY PRESENT,CHILDREN,GROUP OF PEOPLE,ANIMAL,HAPPY,SAD,ANGRY,CONFUSED,DISGUSTED,SURPRISED,CALM,FEAR,STORYTELLING,TONALITà DOMINANTE CALDA,DYNAMISM,NUTRITION,PRINCIPAL CHARACTER IS FEMALE,CELEBRITY PRESENT.1,CHILDREN.1,ANIMAL.1,CREATIVITY,FAMIGLIA E AMICI,ANIMATED,frame,luminance,dominant_color,saturation,hue,contrast,num_edges,symmetry,feature_complexity,entropy,colorfulness,hog_features,texture_contrast,video,motion_per_frame,motion_video
0,ALFA ROMEO_1,1,ALFA ROMEO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0,1.0,0.000000,"[0.0, 0.0, 0.0]",0.000000,0.000000,0.0,0.0,1.000000,0.382089,-1.442695e-07,0.000000,0.000000,0.000000,ALFA ROMEO,0.000000,0.483105
1,ALFA ROMEO_2,2,ALFA ROMEO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0,2.0,27.826085,"[33.93011474609375, 27.073606491088867, 26.786...",136.972729,105.322095,253.0,12179.0,0.108141,76.290509,6.068215e+00,56.397508,0.117005,736.799658,ALFA ROMEO,0.368335,0.483105
2,ALFA ROMEO_3,3,ALFA ROMEO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0,3.0,35.419471,"[41.48649978637695, 35.03296661376953, 33.5271...",123.303932,104.095530,248.0,14886.0,0.108880,77.729986,6.442992e+00,61.278380,0.116734,872.969047,ALFA ROMEO,0.106628,0.483105
3,ALFA ROMEO_4,4,ALFA ROMEO,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0,4.0,35.994936,"[41.988121032714844, 35.62789535522461, 34.033...",125.465059,104.574024,254.0,14926.0,0.106930,78.164448,6.472624e+00,62.668927,0.116814,917.997983,ALFA ROMEO,1.043367,0.483105
4,ALFA ROMEO_5,5,ALFA ROMEO,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,3.0,0,0,5.0,38.939497,"[37.08623504638672, 26.79613494873047, 63.5233...",152.697574,154.762075,147.0,0.0,0.035623,60.718557,5.948974e+00,47.668746,0.111430,21.943449,ALFA ROMEO,0.299338,0.483105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,ORASI_27,27,ORASI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0,27.0,127.859730,"[89.53961181640625, 125.43181610107422, 147.46...",118.096161,20.933797,255.0,222163.0,0.007868,163.967785,7.498583e+00,97.151937,0.126051,9243.214423,ORASI,1.035613,0.637366
518,ORASI_28,28,ORASI,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0,28.0,92.908422,"[45.60554122924805, 102.16219329833984, 92.318...",159.776903,35.889633,227.0,2187.0,0.009591,62.239873,6.872880e+00,81.850282,0.108701,569.298462,ORASI,0.761687,0.637366
519,ORASI_29,29,ORASI,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0,29.0,126.039970,"[79.53384399414062, 140.45567321777344, 116.33...",122.414392,49.474485,254.0,215918.0,0.004768,166.399257,7.610824e+00,85.505197,0.126253,13217.401721,ORASI,0.230305,0.637366
520,ORASI_30,30,ORASI,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0,30.0,125.237246,"[78.94857788085938, 139.61073303222656, 115.49...",122.418936,49.448645,255.0,215589.0,0.004599,166.723331,7.613904e+00,85.123897,0.127122,13204.401410,ORASI,0.617312,0.637366


Aggiungiamo una colonna che dica se il brand sia nei primi 3 sec di video

In [11]:
def assign_section(group):
    # Trova il primo Frame_Num in cui Brand diventa 1
    brand_1_frame = group.loc[group["BRAND"] == 1, "Frame_Num"].min()
    if pd.isna(brand_1_frame):
        raise ValueError(f"Nessun valore '1' trovato nella colonna 'Brand' per il contenuto '{group['Ad name'].iloc[0]}'")

    # Assegna la sezione
    if brand_1_frame <= 6:
        section = 1
    else:
        section = 0
    return pd.Series([section] * len(group), index=group.index)

# Applica la funzione al dataframe raggruppato per 'Content'
feature_df["First_Brand"] = feature_df.groupby("Ad name").apply(assign_section).reset_index(level=0, drop=True)

feature_df

<ipython-input-11-fc63f5bb6e59>:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  feature_df["First_Brand"] = feature_df.groupby("Ad name").apply(assign_section).reset_index(level=0, drop=True)


,KEY,Frame_Num,Ad name,VOLTI,TONALITÀ DOMINANTE CALDA,BRAND,PERSON SHOWN,CELEBRITY PRESENT,CHILDREN,GROUP OF PEOPLE,ANIMAL,HAPPY,SAD,ANGRY,CONFUSED,DISGUSTED,SURPRISED,CALM,FEAR,STORYTELLING,TONALITà DOMINANTE CALDA,DYNAMISM,NUTRITION,PRINCIPAL CHARACTER IS FEMALE,CELEBRITY PRESENT.1,CHILDREN.1,ANIMAL.1,CREATIVITY,FAMIGLIA E AMICI,ANIMATED,frame,luminance,dominant_color,saturation,hue,contrast,num_edges,symmetry,feature_complexity,entropy,colorfulness,hog_features,texture_contrast,video,motion_per_frame,motion_video,First_Brand
0,ALFA ROMEO_1,1,ALFA ROMEO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0,1.0,0.000000,"[0.0, 0.0, 0.0]",0.000000,0.000000,0.0,0.0,1.000000,0.382089,-1.442695e-07,0.000000,0.000000,0.000000,ALFA ROMEO,0.000000,0.483105,1
1,ALFA ROMEO_2,2,ALFA ROMEO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0,2.0,27.826085,"[33.93011474609375, 27.073606491088867, 26.786...",136.972729,105.322095,253.0,12179.0,0.108141,76.290509,6.068215e+00,56.397508,0.117005,736.799658,ALFA ROMEO,0.368335,0.483105,1
2,ALFA ROMEO_3,3,ALFA ROMEO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0,3.0,35.419471,"[41.48649978637695, 35.03296661376953, 33.5271...",123.303932,104.095530,248.0,14886.0,0.108880,77.729986,6.442992e+00,61.278380,0.116734,872.969047,ALFA ROMEO,0.106628,0.483105,1
3,ALFA ROMEO_4,4,ALFA ROMEO,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0,4.0,35.994936,"[41.988121032714844, 35.62789535522461, 34.033...",125.465059,104.574024,254.0,14926.0,0.106930,78.164448,6.472624e+00,62.668927,0.116814,917.997983,ALFA ROMEO,1.043367,0.483105,1
4,ALFA ROMEO_5,5,ALFA ROMEO,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,3.0,0,0,5.0,38.939497,"[37.08623504638672, 26.79613494873047, 63.5233...",152.697574,154.762075,147.0,0.0,0.035623,60.718557,5.948974e+00,47.668746,0.111430,21.943449,ALFA ROMEO,0.299338,0.483105,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,ORASI_27,27,ORASI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0,27.0,127.859730,"[89.53961181640625, 125.43181610107422, 147.46...",118.096161,20.933797,255.0,222163.0,0.007868,163.967785,7.498583e+00,97.151937,0.126051,9243.214423,ORASI,1.035613,0.637366,1
518,ORASI_28,28,ORASI,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0,28.0,92.908422,"[45.60554122924805, 102.16219329833984, 92.318...",159.776903,35.889633,227.0,2187.0,0.009591,62.239873,6.872880e+00,81.850282,0.108701,569.298462,ORASI,0.761687,0.637366,1
519,ORASI_29,29,ORASI,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0,29.0,126.039970,"[79.53384399414062, 140.45567321777344, 116.33...",122.414392,49.474485,254.0,215918.0,0.004768,166.399257,7.610824e+00,85.505197,0.126253,13217.401721,ORASI,0.230305,0.637366,1
520,ORASI_30,30,ORASI,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0,30.0,125.237246,"[78.94857788085938, 139.61073303222656, 115.49...",122.418936,49.448645,255.0,215589.0,0.004599,166.723331,7.613904e+00,85.123897,0.127122,13204.401410,ORASI,0.617312,0.637366,1


In [12]:
feature_df.groupby('Ad name')["First_Brand"].first()

,First_Brand
Ad name,
ALFA ROMEO,1
AMAZON,0
BARILLA,1
CHANEL,0
COCA COLA,1
COOP,0
FASTWEB,0
IKEA,1
LETE,0


# Audio Features Nuove

In [13]:
import numpy as np
import pandas as pd
import librosa
import os
from tqdm import tqdm  # For progress bars

def compute_auditory_loudness(y, sr, n_fft=2048):
    """
    Compute mean and standard deviation of loudness for a given audio frame.

    Parameters:
    - y (np.ndarray): Audio time series.
    - sr (int): Sampling rate.
    - n_fft (int): Number of FFT components.

    Returns:
    - mean_loudness (float)
    - std_loudness (float)
    """
    # Compute Short-Time Fourier Transform (STFT)
    stft = np.abs(librosa.stft(y, n_fft=n_fft))

    # Compute frequency bins
    freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)

    # Apply A-weighting to mimic human auditory perception
    auditory_weight = librosa.A_weighting(freqs)

    # Reshape for broadcasting: (freq_bins, 1)
    auditory_weight = auditory_weight[:, np.newaxis]

    # Apply weighting
    weighted_stft = stft * (10 ** (auditory_weight / 10.0))

    # Compute loudness: log10 of the sum across frequency bins
    loudness = np.log10(np.sum(weighted_stft, axis=0) + 1e-10)  # Avoid log(0)

    # Summary statistics
    mean_loudness = np.mean(loudness)
    std_loudness = np.std(loudness)

    return mean_loudness, std_loudness

def compute_timbre_centroid(y, sr, n_fft=2048):
    """
    Compute mean and standard deviation of spectral centroid for a given audio frame.

    Parameters:
    - y (np.ndarray): Audio time series.
    - sr (int): Sampling rate.
    - n_fft (int): Number of FFT components.

    Returns:
    - mean_centroid (float)
    - std_centroid (float)
    """
    # Compute spectral centroid
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=n_fft)

    # Flatten to 1D array
    spectral_centroid = spectral_centroid.flatten()

    # Summary statistics
    mean_centroid = np.mean(spectral_centroid)
    std_centroid = np.std(spectral_centroid)

    return mean_centroid, std_centroid

def compute_onset_density(onset_times, frame_start, frame_end):
    """
    Compute the number of onsets within a frame.

    Parameters:
    - onset_times (np.ndarray): Array of onset times in seconds.
    - frame_start (float): Start time of the frame in seconds.
    - frame_end (float): End time of the frame in seconds.

    Returns:
    - onset_density (float): Number of onsets per second within the frame.
    """
    # Count onsets within the frame
    count = np.sum((onset_times >= frame_start) & (onset_times < frame_end))

    # Calculate density: onsets per second
    duration = frame_end - frame_start
    onset_density = count / duration if duration > 0 else 0.0  # Avoid division by zero

    return onset_density

def process_audio_file(file_path, frame_duration_sec=0.5, n_fft=2048):
    """
    Process a single audio file and extract features per frame.

    Parameters:
    - file_path (str): Path to the .mp3 file.
    - frame_duration_sec (float): Duration of each frame in seconds.
    - n_fft (int): Number of FFT components.

    Returns:
    - List of dictionaries containing features for each frame.
    """
    results = []
    file_name = os.path.basename(file_path)

    try:
        # Load the audio file
        y, sr = librosa.load(file_path, sr=None)
        duration = len(y) / sr

        # Detect onset frames and convert to times
        onset_frames = librosa.onset.onset_detect(y=y, sr=sr, backtrack=False)
        onset_times = librosa.frames_to_time(onset_frames, sr=sr)

        # Define frame parameters
        frame_length = int(frame_duration_sec * sr)
        hop_length = frame_length  # Non-overlapping frames
        total_frames = int(np.floor(len(y) / hop_length))

        for frame_idx in range(total_frames):
            start_sample = frame_idx * hop_length
            end_sample = start_sample + frame_length
            frame_start_time = frame_idx * frame_duration_sec
            frame_end_time = frame_start_time + frame_duration_sec

            # Extract frame audio
            frame_y = y[start_sample:end_sample]

            # Compute features
            mean_loudness, std_loudness = compute_auditory_loudness(frame_y, sr, n_fft=n_fft)
            mean_centroid, std_centroid = compute_timbre_centroid(frame_y, sr, n_fft=n_fft)
            onset_density = compute_onset_density(onset_times, frame_start_time, frame_end_time)

            # Append results
            results.append({
                "file": file_name,
                "frame_index": frame_idx + 1,
                "frame_start_time_sec": frame_start_time,
                "frame_end_time_sec": frame_end_time,
                "mean_loudness": mean_loudness,
                "std_loudness": std_loudness,
                "onset_density": onset_density,
                "mean_timbre_centroid": mean_centroid,
                "std_timbre_centroid": std_centroid
            })

        # Handle the last frame if it's shorter than frame_duration_sec
        remaining_samples = len(y) - total_frames * hop_length
        if remaining_samples > 0:
            frame_y = y[total_frames * hop_length:]
            frame_start_time = total_frames * frame_duration_sec
            frame_end_time = duration

            # Optionally, pad the frame with zeros to maintain consistency
            if len(frame_y) < frame_length:
                padding = frame_length - len(frame_y)
                frame_y = np.pad(frame_y, (0, padding), mode='constant')

            # Compute features
            mean_loudness, std_loudness = compute_auditory_loudness(frame_y, sr, n_fft=n_fft)
            mean_centroid, std_centroid = compute_timbre_centroid(frame_y, sr, n_fft=n_fft)
            onset_density = compute_onset_density(onset_times, frame_start_time, frame_end_time)

            # Append results
            results.append({
                "file": file_name,
                "frame_index": total_frames+ 1,
                "frame_start_time_sec": frame_start_time,
                "frame_end_time_sec": frame_end_time,
                "mean_loudness": mean_loudness,
                "std_loudness": std_loudness,
                "onset_density": onset_density,
                "mean_timbre_centroid": mean_centroid,
                "std_timbre_centroid": std_centroid
            })

    except Exception as e:
        print(f"Error processing {file_name}: {e}")

    return results
def process_audio_files(directory_path, frame_duration_sec=0.5, n_fft=2048):
    """
    Process all .mp3 files in the specified directory and extract features per frame.

    Parameters:
    - directory_path (str): Path to the directory containing .mp3 files.
    - frame_duration_sec (float): Duration of each frame in seconds.
    - n_fft (int): Number of FFT components.

    Returns:
    - pandas.DataFrame containing the extracted features.
    """
    all_results = []

    # Define mapping from filenames to desired names
    file_mapping = {
        'audio_LETE.mp3': 'LETE',
        'audio_COOP.mp3': 'COOP',
        'audio_MUTTI.mp3': 'MUTTI',
        'audio_ORASI.mp3': 'ORASI',
        'audio_BARILLA.mp3': 'BARILLA',
        'audio_Amazon.mp3': 'AMAZON',
        'audio_FASTWEB.mp3': 'FASTWEB',
        'audio_MULTICENTRUM.mp3': 'MULTICENTRUM',
        'audio_ALFA TONALE.mp3': 'ALFA ROMEO',
        'audio_COCA COLA.mp3': 'COCA COLA',
        'audio_Spot Fregola vol alto .mp3': 'FREGOLA',
        'S80302-MP4-IKEA-ITALIA.mp3': 'IKEA',
        'CHANEL.mp3' : 'CHANEL'
        # Add any additional mappings if necessary
    }

    # Get list of .mp3 files
    files = [file for file in os.listdir(directory_path) if file.lower().endswith('.mp3')]

    if not files:
        print("No .mp3 files found in the specified directory.")
        return pd.DataFrame()  # Return empty DataFrame

    # Process each file with a progress bar
    for file in tqdm(files, desc="Processing audio files"):
        file_path = os.path.join(directory_path, file)
        file_results = process_audio_file(file_path, frame_duration_sec, n_fft)

        # Replace the 'file' field with the mapped name
        for result in file_results:
            result['file'] = file_mapping.get(file, file)  # Use mapped name, or default to original

        all_results.extend(file_results)

    # Convert to pandas DataFrame
    df = pd.DataFrame(all_results)

    return df



def main():
    """
    Main function to execute the audio processing pipeline.

    Returns:
    - pandas.DataFrame containing all extracted features.
    """
    # Specify the directory containing your .mp3 files
    # Replace the path below with your actual directory path
    directory = '/content/drive/My Drive/Thesis/DATI E ALTRO/Pubblicità file/'

    # Check if the directory exists
    if not os.path.isdir(directory):
        print(f"The directory {directory} does not exist. Please check the path.")
        return pd.DataFrame()  # Return empty DataFrame

    # Process the audio files and extract features
    feature_df = process_audio_files(directory_path=directory, frame_duration_sec=0.5, n_fft=2048)

    if feature_df.empty:
        print("No features extracted. Exiting.")
        return feature_df

    # Save the DataFrame to a CSV file (optional)
    output_csv = 'audio_features_per_frame.csv'
    feature_df.to_csv(output_csv, index=False)
    print(f"\nFeature extraction complete. Results saved to {output_csv}")

    # Display the first few rows of the DataFrame
    print("\nSample of extracted features:")
    print(feature_df.head())

    return feature_df  # Return the DataFrame

if __name__ == "__main__":
    # Execute the main function and capture the returned DataFrame
    audio_df = main()




audio_df.drop(columns=['frame_start_time_sec','frame_end_time_sec'], inplace=True)

Processing audio files:   0%|          | 0/13 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/librosa/core/convert.py:1870: RuntimeWarning: divide by zero encountered in log10
  + 2 * np.log10(f_sq)
Processing audio files: 100%|██████████| 13/13 [00:38<00:00,  3.00s/it]



Feature extraction complete. Results saved to audio_features_per_frame.csv

Sample of extracted features:
   file  frame_index  frame_start_time_sec  frame_end_time_sec  mean_loudness  \
0  LETE            1                   0.0                 0.5      -2.118448   
1  LETE            2                   0.5                 1.0       2.372021   
2  LETE            3                   1.0                 1.5       2.430249   
3  LETE            4                   1.5                 2.0       2.437812   
4  LETE            5                   2.0                 2.5       2.503323   

   std_loudness  onset_density  mean_timbre_centroid  std_timbre_centroid  
0      5.741819            2.0           1812.075597          1363.603096  
1      0.152812            2.0           3786.552392          1636.314341  
2      0.100014            4.0           4247.297879           378.669551  
3      0.083155            4.0           4444.889779          1097.825356  
4      0.093359           

In [14]:
audio_df

,file,frame_index,mean_loudness,std_loudness,onset_density,mean_timbre_centroid,std_timbre_centroid
0,LETE,1,-2.118448,5.741819,2.0,1812.075597,1363.603096
1,LETE,2,2.372021,0.152812,2.0,3786.552392,1636.314341
2,LETE,3,2.430249,0.100014,4.0,4247.297879,378.669551
3,LETE,4,2.437812,0.083155,4.0,4444.889779,1097.825356
4,LETE,5,2.503323,0.093359,2.0,6035.973009,1506.715694
...,...,...,...,...,...,...,...
580,CHANEL,57,2.289074,0.401611,6.0,3915.518878,2431.569112
581,CHANEL,58,1.693004,0.175065,0.0,1384.036578,624.447474
582,CHANEL,59,2.136818,0.523288,6.0,3146.952919,1791.372196
583,CHANEL,60,0.999363,0.521763,2.0,1643.411764,696.214694


In [15]:
feature_df = pd.merge(feature_df, audio_df, left_on=['Ad name','Frame_Num'],right_on=['file','frame_index'], how='left')
feature_df

,KEY,Frame_Num,Ad name,VOLTI,TONALITÀ DOMINANTE CALDA,BRAND,PERSON SHOWN,CELEBRITY PRESENT,CHILDREN,GROUP OF PEOPLE,ANIMAL,HAPPY,SAD,ANGRY,CONFUSED,DISGUSTED,SURPRISED,CALM,FEAR,STORYTELLING,TONALITà DOMINANTE CALDA,DYNAMISM,NUTRITION,PRINCIPAL CHARACTER IS FEMALE,CELEBRITY PRESENT.1,CHILDREN.1,ANIMAL.1,CREATIVITY,FAMIGLIA E AMICI,ANIMATED,frame,luminance,dominant_color,saturation,hue,contrast,num_edges,symmetry,feature_complexity,entropy,colorfulness,hog_features,texture_contrast,video,motion_per_frame,motion_video,First_Brand,file,frame_index,mean_loudness,std_loudness,onset_density,mean_timbre_centroid,std_timbre_centroid
0,ALFA ROMEO_1,1,ALFA ROMEO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0,1.0,0.000000,"[0.0, 0.0, 0.0]",0.000000,0.000000,0.0,0.0,1.000000,0.382089,-1.442695e-07,0.000000,0.000000,0.000000,ALFA ROMEO,0.000000,0.483105,1,ALFA ROMEO,1,-2.246584,5.674266,4.0,4010.187303,4064.864260
1,ALFA ROMEO_2,2,ALFA ROMEO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0,2.0,27.826085,"[33.93011474609375, 27.073606491088867, 26.786...",136.972729,105.322095,253.0,12179.0,0.108141,76.290509,6.068215e+00,56.397508,0.117005,736.799658,ALFA ROMEO,0.368335,0.483105,1,ALFA ROMEO,2,2.387223,0.115719,4.0,3961.772386,456.284951
2,ALFA ROMEO_3,3,ALFA ROMEO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0,3.0,35.419471,"[41.48649978637695, 35.03296661376953, 33.5271...",123.303932,104.095530,248.0,14886.0,0.108880,77.729986,6.442992e+00,61.278380,0.116734,872.969047,ALFA ROMEO,0.106628,0.483105,1,ALFA ROMEO,3,2.263476,0.109519,4.0,3491.815595,945.658139
3,ALFA ROMEO_4,4,ALFA ROMEO,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,3.0,0,0,4.0,35.994936,"[41.988121032714844, 35.62789535522461, 34.033...",125.465059,104.574024,254.0,14926.0,0.106930,78.164448,6.472624e+00,62.668927,0.116814,917.997983,ALFA ROMEO,1.043367,0.483105,1,ALFA ROMEO,4,2.307894,0.137379,4.0,3482.200652,877.435431
4,ALFA ROMEO_5,5,ALFA ROMEO,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,3.0,0,0,5.0,38.939497,"[37.08623504638672, 26.79613494873047, 63.5233...",152.697574,154.762075,147.0,0.0,0.035623,60.718557,5.948974e+00,47.668746,0.111430,21.943449,ALFA ROMEO,0.299338,0.483105,1,ALFA ROMEO,5,2.329419,0.075222,8.0,2973.925616,803.576210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,ORASI_27,27,ORASI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0,27.0,127.859730,"[89.53961181640625, 125.43181610107422, 147.46...",118.096161,20.933797,255.0,222163.0,0.007868,163.967785,7.498583e+00,97.151937,0.126051,9243.214423,ORASI,1.035613,0.637366,1,ORASI,27,2.298222,0.350913,4.0,3846.632583,1289.680755
518,ORASI_28,28,ORASI,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0,28.0,92.908422,"[45.60554122924805, 102.16219329833984, 92.318...",159.776903,35.889633,227.0,2187.0,0.009591,62.239873,6.872880e+00,81.850282,0.108701,569.298462,ORASI,0.761687,0.637366,1,ORASI,28,2.109342,0.335195,8.0,4353.688378,2035.642230
519,ORASI_29,29,ORASI,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0,29.0,126.039970,"[79.53384399414062, 140.45567321777344, 116.33...",122.414392,49.474485,254.0,215918.0,0.004768,166.399257,7.610824e+00,85.505197,0.126253,13217.401721,ORASI,0.230305,0.637366,1,ORASI,29,2.301330,0.303654,8.0,3604.240986,1636.339756
520,ORASI_30,30,ORASI,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2.0,1,0,30.0,125.237246,"[78.94857788085938, 139.61073303222656, 115.49...",122.418936,49.448645,255.0,215589.0,0.004599,166.723331,7.613904e+00,85.123897,0.127122,13204.401410,ORASI,0.617312,0.637366,1,ORASI,30,1.686596,0.608608,6.0,4617.553430,2571.095288


## Leggo sensors_df

In [16]:

# Percorso del file Excel
file_path = "data_cleaned_19122024.csv"


# Leggere il foglio specifico
try:
    df = pd.read_csv(file_path)
    # Mostrare i primi dati per conferma
    df.head()
except Exception as e:
    e

<ipython-input-16-ff4f5a9b29b3>:7: DtypeWarning: Columns (112) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [17]:
df[~(df['Type']=='ad') ]

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Asymmetry Delta,Asymmetry Theta,Mean Frontal Asymmetry Alpha,Asymmetry Beta,Asymmetry Gamma,Device,Protocol,Vision,Position,Type,Engagement,Engagement(1/alpha),FAA,Pleasantness Index,Pleasantness,Attention,AW Index,Vamv Valence,Kirk Valence,Ram12 Valence,Ram15 Valence,Choice Index (Gamma),Choice Index (Beta),Effort Index 1,Effort Index 2,Workload_mean,Workload_mean2,SASI,Engagement standardized,Pleasantness standardized,Attention standardized,FAA standardized,Engagement(1/alpha) standardized,Asymmetry Delta standardized,Asymmetry Theta standardized,Asymmetry Beta standardized,Asymmetry Gamma standardized,Beta standardized,Alpha standardized,Gamma standardized,Delta standardized,AW Index standardized,Vamv Valence standardized,Kirk Valence standardized,Ram12 Valence standardized,Ram15 Valence standardized,Choice Index (Gamma) standardized,Choice Index (Beta) standardized,Effort Index 1 standardized,Effort Index 2 standardized,Workload_mean standardized,Workload_mean2 standardized,SASI standardized,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,DOB,Gender,Unaided Recall,Aided Recall,Content Appreciation,Category


In [18]:
df_adv = df[df['Type']=='ad']
display(df_adv)

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Asymmetry Delta,Asymmetry Theta,Mean Frontal Asymmetry Alpha,Asymmetry Beta,Asymmetry Gamma,Device,Protocol,Vision,Position,Type,Engagement,Engagement(1/alpha),FAA,Pleasantness Index,Pleasantness,Attention,AW Index,Vamv Valence,Kirk Valence,Ram12 Valence,Ram15 Valence,Choice Index (Gamma),Choice Index (Beta),Effort Index 1,Effort Index 2,Workload_mean,Workload_mean2,SASI,Engagement standardized,Pleasantness standardized,Attention standardized,FAA standardized,Engagement(1/alpha) standardized,Asymmetry Delta standardized,Asymmetry Theta standardized,Asymmetry Beta standardized,Asymmetry Gamma standardized,Beta standardized,Alpha standardized,Gamma standardized,Delta standardized,AW Index standardized,Vamv Valence standardized,Kirk Valence standardized,Ram12 Valence standardized,Ram15 Valence standardized,Choice Index (Gamma) standardized,Choice Index (Beta) standardized,Effort Index 1 standardized,Effort Index 2 standardized,Workload_mean standardized,Workload_mean2 standardized,SASI standardized,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,DOB,Gender,Unaided Recall,Aided Recall,Content Appreciation,Category
0,s10,406001.0,500,ORASI,Subdivision 1,0.0,0.0,NaN,51.274331,4.421809,0.656750,0.554243,0.052481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.404432,1.180864,0.352452,0.162106,0.077571,NaN,NaN,NaN,NaN,NaN,60.841512,2.801560,0.504545,0.358179,0.065028,0.317,-1.320,-0.622,-1.229,0.391,Laptop,Linear_120,2.0,2.0,ad,0.108339,2.971,-2.703,-3.4938,-0.622,4.474,-0.301523,0.383979,NaN,-0.201791,0.989253,NaN,NaN,-0.578464,NaN,0.801928,0.838789,-0.773286,-0.451573,-0.500692,-0.114722,0.288997,1.002783,0.414855,-1.615264,-2.316575,0.375997,-0.802676,-1.002783,-1.151474,0.715618,0.205021,0.936126,NaN,-0.316300,0.569383,NaN,NaN,-0.512294,NaN,-0.190858,0.150995,-0.582985,2024-04-17 15:50:34,2024-04-17 15:57:46,IP Address,93.51.247.100,100.0,431.0,True,2024-04-17 15:57:46.450000,R_5dMsXsUPvJePDAo,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,1974,Femmina,1,1,7.675954,NaN
1,s10,406501.0,500,ORASI,Subdivision 2,0.0,0.0,NaN,NaN,NaN,3.033891,0.291541,0.029242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.202454,4.680584,0.410677,0.217270,0.054413,NaN,NaN,NaN,NaN,NaN,135.737548,NaN,1.722265,0.254390,0.041831,-1.092,NaN,NaN,-0.294,0.621,Laptop,Linear_120,2.0,2.0,ad,0.006328,-2.361,-8.685,-6.4626,NaN,15.852,-0.761551,-0.432959,NaN,0.119135,NaN,NaN,NaN,-0.878333,NaN,0.383641,0.171995,-0.986864,-1

### eliminate participant s264-TV (because it has been done twice)

In [19]:
df_adv=df_adv[~((df_adv['ID']=='s264')&(df_adv['Device']=='TV'))]
df_adv

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Asymmetry Delta,Asymmetry Theta,Mean Frontal Asymmetry Alpha,Asymmetry Beta,Asymmetry Gamma,Device,Protocol,Vision,Position,Type,Engagement,Engagement(1/alpha),FAA,Pleasantness Index,Pleasantness,Attention,AW Index,Vamv Valence,Kirk Valence,Ram12 Valence,Ram15 Valence,Choice Index (Gamma),Choice Index (Beta),Effort Index 1,Effort Index 2,Workload_mean,Workload_mean2,SASI,Engagement standardized,Pleasantness standardized,Attention standardized,FAA standardized,Engagement(1/alpha) standardized,Asymmetry Delta standardized,Asymmetry Theta standardized,Asymmetry Beta standardized,Asymmetry Gamma standardized,Beta standardized,Alpha standardized,Gamma standardized,Delta standardized,AW Index standardized,Vamv Valence standardized,Kirk Valence standardized,Ram12 Valence standardized,Ram15 Valence standardized,Choice Index (Gamma) standardized,Choice Index (Beta) standardized,Effort Index 1 standardized,Effort Index 2 standardized,Workload_mean standardized,Workload_mean2 standardized,SASI standardized,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,DOB,Gender,Unaided Recall,Aided Recall,Content Appreciation,Category
0,s10,406001.0,500,ORASI,Subdivision 1,0.0,0.0,NaN,51.274331,4.421809,0.656750,0.554243,0.052481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.404432,1.180864,0.352452,0.162106,0.077571,NaN,NaN,NaN,NaN,NaN,60.841512,2.801560,0.504545,0.358179,0.065028,0.317,-1.320,-0.622,-1.229,0.391,Laptop,Linear_120,2.0,2.0,ad,0.108339,2.971,-2.703,-3.4938,-0.622,4.474,-0.301523,0.383979,NaN,-0.201791,0.989253,NaN,NaN,-0.578464,NaN,0.801928,0.838789,-0.773286,-0.451573,-0.500692,-0.114722,0.288997,1.002783,0.414855,-1.615264,-2.316575,0.375997,-0.802676,-1.002783,-1.151474,0.715618,0.205021,0.936126,NaN,-0.316300,0.569383,NaN,NaN,-0.512294,NaN,-0.190858,0.150995,-0.582985,2024-04-17 15:50:34,2024-04-17 15:57:46,IP Address,93.51.247.100,100.0,431.0,True,2024-04-17 15:57:46.450000,R_5dMsXsUPvJePDAo,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,1974,Femmina,1,1,7.675954,NaN
1,s10,406501.0,500,ORASI,Subdivision 2,0.0,0.0,NaN,NaN,NaN,3.033891,0.291541,0.029242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.202454,4.680584,0.410677,0.217270,0.054413,NaN,NaN,NaN,NaN,NaN,135.737548,NaN,1.722265,0.254390,0.041831,-1.092,NaN,NaN,-0.294,0.621,Laptop,Linear_120,2.0,2.0,ad,0.006328,-2.361,-8.685,-6.4626,NaN,15.852,-0.761551,-0.432959,NaN,0.119135,NaN,NaN,NaN,-0.878333,NaN,0.383641,0.171995,-0.986864,-1

In [20]:
# Sort the DataFrame by ID, Vision, and Start (ms)
df_adv = df_adv.sort_values(by=['ID', 'Vision', 'Start (ms)'])
# df_adv = df_adv[df_adv['Content'] == 'BARILLA'].drop_duplicates(subset='ID', keep='first')


In [21]:

# Drop duplicates in 'Start (ms)' for each ID, keeping the first occurrence
df_adv = df_adv.drop_duplicates(subset=['ID','Vision', 'Start (ms)'], keep='first')
df_adv

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Asymmetry Delta,Asymmetry Theta,Mean Frontal Asymmetry Alpha,Asymmetry Beta,Asymmetry Gamma,Device,Protocol,Vision,Position,Type,Engagement,Engagement(1/alpha),FAA,Pleasantness Index,Pleasantness,Attention,AW Index,Vamv Valence,Kirk Valence,Ram12 Valence,Ram15 Valence,Choice Index (Gamma),Choice Index (Beta),Effort Index 1,Effort Index 2,Workload_mean,Workload_mean2,SASI,Engagement standardized,Pleasantness standardized,Attention standardized,FAA standardized,Engagement(1/alpha) standardized,Asymmetry Delta standardized,Asymmetry Theta standardized,Asymmetry Beta standardized,Asymmetry Gamma standardized,Beta standardized,Alpha standardized,Gamma standardized,Delta standardized,AW Index standardized,Vamv Valence standardized,Kirk Valence standardized,Ram12 Valence standardized,Ram15 Valence standardized,Choice Index (Gamma) standardized,Choice Index (Beta) standardized,Effort Index 1 standardized,Effort Index 2 standardized,Workload_mean standardized,Workload_mean2 standardized,SASI standardized,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,DOB,Gender,Unaided Recall,Aided Recall,Content Appreciation,Category
244,s10,381008.0,500,COOP,Subdivision 1,0.0,0.0,NaN,NaN,6.154602,0.648485,0.586273,0.092619,NaN,6.660395,1.393157,0.956533,0.136804,28.183829,4.016983,2.163715,1.027070,0.176970,6.677286,3.317416,1.820958,0.873373,0.087498,32.033176,5.977230,2.031421,1.226874,0.390841,NaN,8.945405,2.678551,0.776784,0.591698,26.736220,1.365841,1.143405,0.174743,0.076015,NaN,NaN,NaN,NaN,NaN,78.523563,5.205950,1.697071,0.803156,0.221769,-0.356,-0.033,0.331,-0.165,0.957,Laptop,Linear_180,1.0,2.0,ad,0.116349,-2.297,2.463,0.1470,0.331,7.165,0.276200,0.751239,-0.627008,0.557132,NaN,NaN,-0.700775,-0.636766,-2.503752,0.634626,0.603937,-0.732687,-0.121507,0.823346,0.283143,1.601816,-0.520473,-0.327133,0.134988,-0.428935,1.490422,0.790804,0.520473,0.698664,0.748758,1.794236,2.069098,-1.297678,1.387694,NaN,NaN,-1.759758,-0.613609,-0.458606,-0.393228,-0.193970,-0.168616,2024-04-17 15:50:34,2024-04-17 15:57:46,IP Address,93.51.247.100,100.0,431.0,True,2024-04-17 15:57:46.450000,R_5dMsXsUPvJePDAo,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,1974,Femmina,1,1,7.675954,NaN
245,s10,381508.0,500,COOP,Subdivision 2,0.0,0.0,NaN,79.213645,4.489521,0.320922,0.456667,0.156207,76.966192,5.571857,0.671583,0.770016,0.273842,33.273616,3.335032,0.581433,1.135795,0.238287,9.977001,2.034230,0.265644,1.057548,0.104641,7.717915,1.570001,0.415145,1.172195,0.330598,32.938210,3.262121,1.034427,1.087928,0.189409,5.166542,0.564937,0.313617

### Create Frame_Num column in order to count the number of frame for each ad

In [22]:
# Reset the count to 1 if ID, Vision, or Content changes
columns_to_group = ['Vision', 'Content', 'ID']
df_adv['Frame_Num'] = df_adv.groupby(columns_to_group).cumcount() + 1

### Normalize each content to get each ad of the same type with the same num of frames

In [23]:
len(df_adv['ID'].unique())

563

In [24]:
import pandas as pd

# Calcola il massimo per ciascun gruppo di 'ID' e 'Content'
group_max = df_adv.groupby(['ID', 'Content','Vision'])['Frame_Num'].transform('max')

# Filtra i gruppi validi
filtered_df = df_adv[((group_max >= 27) & (group_max <= 33)) | ((group_max >=57) & (group_max <= 62))]

# Lista pub30
pub30 = ['IKEA', 'MULTICENTRUM', 'BARILLA', 'FASTWEB', 'MUTTI','ORASI','LETE']

# --- Step 1: Per i gruppi in pub30 ---
filtered_pub30 = filtered_df[filtered_df['Content'].isin(pub30)]
max_frame_pub30 = filtered_pub30.groupby(['ID', 'Content','Vision'])['Frame_Num'].max().reset_index()

# Trova i gruppi con massimo tra 28 e 30
valid_pub30 = max_frame_pub30[(max_frame_pub30['Frame_Num'] >= 27) & (max_frame_pub30['Frame_Num'] <= 30)]

# Sostituisci i frame per i gruppi validi
for _, row in valid_pub30.iterrows():
    id_value, content_value, vision = row['ID'], row['Content'], row['Vision']
    # Filtra il gruppo specifico
    group_rows = filtered_df[(filtered_df['ID'] == id_value) & (filtered_df['Content'] == content_value)& (filtered_df['Vision'] == vision)]
    # Trova il massimo corrente
    max_frame = group_rows['Frame_Num'].max()
    # Crea nuove righe per arrivare a 31
    for i in range(max_frame + 1, 32):
        new_row = group_rows.iloc[-1].copy()
        new_row['Frame_Num'] = i
        filtered_df = pd.concat([filtered_df, pd.DataFrame([new_row])], ignore_index=True)



In [25]:


# --- Step 2: Per gli altri gruppi (non in pub30) ---
filtered_other_content = filtered_df[~filtered_df['Content'].isin(pub30)]
max_frame_other_content = filtered_other_content.groupby(['ID', 'Content','Vision'])['Frame_Num'].max().reset_index()

# Trova i gruppi con massimo tra 58 e 60
valid_other_content = max_frame_other_content[(max_frame_other_content['Frame_Num'] >= 57) & (max_frame_other_content['Frame_Num'] <= 60)]

# Sostituisci i frame per i gruppi validi
for _, row in valid_other_content.iterrows():
    id_value, content_value, vision = row['ID'], row['Content'], row['Vision']
    # Filtra il gruppo specifico
    group_rows = filtered_df[(filtered_df['ID'] == id_value) & (filtered_df['Content'] == content_value)& (filtered_df['Vision'] == vision)]
    # Trova il massimo corrente
    max_frame = group_rows['Frame_Num'].max()
    # Crea nuove righe per arrivare a 61
    for i in range(max_frame + 1, 62):
        new_row = group_rows.iloc[-1].copy()
        new_row['Frame_Num'] = i
        filtered_df = pd.concat([filtered_df, pd.DataFrame([new_row])], ignore_index=True)

# Risultato finale
filtered_df = filtered_df.sort_values(['ID', 'Content','Vision', 'Frame_Num']).reset_index(drop=True)

Creation of a new column for the duration of the AD

In [26]:
# Lista dei brand
pub30 = ['IKEA', 'MULTICENTRUM', 'BARILLA', 'FASTWEB', 'MUTTI', 'ORASI', 'LETE']

# Creazione della nuova colonna
filtered_df['Duration AD'] = filtered_df['Content'].apply(lambda x: 15 if x in pub30 else 30)

In [27]:
merged_feat_bio = pd.merge(filtered_df, feature_df, left_on=['Content','Frame_Num'],right_on=['Ad name','Frame_Num'], how='left', indicator=True)
merged_feat_bio

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Asymmetry Delta,Asymmetry Theta,Mean Frontal Asymmetry Alpha,Asymmetry Beta,Asymmetry Gamma,Device,Protocol,Vision,Position,Type,Engagement,Engagement(1/alpha),FAA,Pleasantness Index,Pleasantness,Attention,AW Index,Vamv Valence,Kirk Valence,Ram12 Valence,Ram15 Valence,Choice Index (Gamma),Choice Index (Beta),Effort Index 1,Effort Index 2,Workload_mean,Workload_mean2,SASI,Engagement standardized,Pleasantness standardized,Attention standardized,FAA standardized,Engagement(1/alpha) standardized,Asymmetry Delta standardized,Asymmetry Theta standardized,Asymmetry Beta standardized,Asymmetry Gamma standardized,Beta standardized,Alpha standardized,Gamma standardized,Delta standardized,AW Index standardized,Vamv Valence standardized,Kirk Valence standardized,Ram12 Valence standardized,Ram15 Valence standardized,Choice Index (Gamma) standardized,Choice Index (Beta) standardized,Effort Index 1 standardized,Effort Index 2 standardized,Workload_mean standardized,Workload_mean2 standardized,SASI standardized,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,DOB,Gender,Unaided Recall,Aided Recall,Content Appreciation,Category,Frame_Num,Duration AD,KEY,Ad name,VOLTI,TONALITÀ DOMINANTE CALDA,BRAND,PERSON SHOWN,CELEBRITY PRESENT,CHILDREN,GROUP OF PEOPLE,ANIMAL,HAPPY,SAD,ANGRY,CONFUSED,DISGUSTED,SURPRISED,CALM,FEAR,STORYTELLING,TONALITà DOMINANTE CALDA,DYNAMISM,NUTRITION,PRINCIPAL CHARACTER IS FEMALE,CELEBRITY PRESENT.1,CHILDREN.1,ANIMAL.1,CREATIVITY,FAMIGLIA E AMICI,ANIMATED,frame,luminance,dominant_color,saturation,hue,contrast,num_edges,symmetry,feature_complexity,entropy,colorfulness,hog_features,texture_contrast,video,motion_per_frame,motion_video,First_Brand,file,frame_index,mean_loudness,std_loudness,onset_density,mean_timbre_centroid,std_timbre_centroid,_merge
0,s10,505536.0,500,AMAZON,Subdivision 1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_180,1.0,8.0,ad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-17 15:50:34,2024-04-17 15:57:46,IP Address,93.51.247.100,100.0,431.0,True,2024-04-17 15:57:46.450000,R_5dMsXsUPvJePDAo,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,1974,Femmina,1,1,7.675954,NaN,1,30,AMAZON_1,AMAZON,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,

In [28]:
len(merged_feat_bio[['ID', 'Content']].drop_duplicates())


3711

## Leggo ISC

In [29]:

# Percorso del file Excel
file_path_isc = "isc_results_optimized.csv"


# Leggere il foglio specifico
try:
    df_isc = pd.read_csv(file_path_isc)
    # Mostrare i primi dati per conferma
    df_isc.head()
except Exception as e:
    e



merged_feat_bio = merged_feat_bio.merge(df_isc, on=['ID','Content','Vision'], how='left')
merged_feat_bio

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Asymmetry Delta,Asymmetry Theta,Mean Frontal Asymmetry Alpha,Asymmetry Beta,Asymmetry Gamma,Device,Protocol,Vision,Position,Type,Engagement,Engagement(1/alpha),FAA,Pleasantness Index,Pleasantness,Attention,AW Index,Vamv Valence,Kirk Valence,Ram12 Valence,Ram15 Valence,Choice Index (Gamma),Choice Index (Beta),Effort Index 1,Effort Index 2,Workload_mean,Workload_mean2,SASI,Engagement standardized,Pleasantness standardized,Attention standardized,FAA standardized,Engagement(1/alpha) standardized,Asymmetry Delta standardized,Asymmetry Theta standardized,Asymmetry Beta standardized,Asymmetry Gamma standardized,Beta standardized,Alpha standardized,Gamma standardized,Delta standardized,AW Index standardized,Vamv Valence standardized,Kirk Valence standardized,Ram12 Valence standardized,Ram15 Valence standardized,Choice Index (Gamma) standardized,Choice Index (Beta) standardized,Effort Index 1 standardized,Effort Index 2 standardized,Workload_mean standardized,Workload_mean2 standardized,SASI standardized,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,DOB,Gender,Unaided Recall,Aided Recall,Content Appreciation,Category,Frame_Num,Duration AD,KEY,Ad name,VOLTI,TONALITÀ DOMINANTE CALDA,BRAND,PERSON SHOWN,CELEBRITY PRESENT,CHILDREN,GROUP OF PEOPLE,ANIMAL,HAPPY,SAD,ANGRY,CONFUSED,DISGUSTED,SURPRISED,CALM,FEAR,STORYTELLING,TONALITà DOMINANTE CALDA,DYNAMISM,NUTRITION,PRINCIPAL CHARACTER IS FEMALE,CELEBRITY PRESENT.1,CHILDREN.1,ANIMAL.1,CREATIVITY,FAMIGLIA E AMICI,ANIMATED,frame,luminance,dominant_color,saturation,hue,contrast,num_edges,symmetry,feature_complexity,entropy,colorfulness,hog_features,texture_contrast,video,motion_per_frame,motion_video,First_Brand,file,frame_index,mean_loudness,std_loudness,onset_density,mean_timbre_centroid,std_timbre_centroid,_merge,Delta,Theta,Alpha,Beta,Gamma,Mean ISC
0,s10,505536.0,500,AMAZON,Subdivision 1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_180,1.0,8.0,ad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-17 15:50:34,2024-04-17 15:57:46,IP Address,93.51.247.100,100.0,431.0,True,2024-04-17 15:57:46.450000,R_5dMsXsUPvJePDAo,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,1974,Femmina,1,1,7.675954,NaN,1,30,AMAZON_1,AMAZON,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

## Leggo Volume

In [30]:
volume=pd.read_excel("Volume.xlsx")
volume.columns

Index(['Content', 'volume_dbfs', 'diff_percent', 'Frame_Num'], dtype='object')

In [31]:
merged_feat_bio = pd.merge(merged_feat_bio, volume, left_on=['Content','Frame_Num'],right_on=['Content','Frame_Num'], how='left')
merged_feat_bio

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Asymmetry Delta,Asymmetry Theta,Mean Frontal Asymmetry Alpha,Asymmetry Beta,Asymmetry Gamma,Device,Protocol,Vision,Position,Type,Engagement,Engagement(1/alpha),FAA,Pleasantness Index,Pleasantness,Attention,AW Index,Vamv Valence,Kirk Valence,Ram12 Valence,Ram15 Valence,Choice Index (Gamma),Choice Index (Beta),Effort Index 1,Effort Index 2,Workload_mean,Workload_mean2,SASI,Engagement standardized,Pleasantness standardized,Attention standardized,FAA standardized,Engagement(1/alpha) standardized,Asymmetry Delta standardized,Asymmetry Theta standardized,Asymmetry Beta standardized,Asymmetry Gamma standardized,Beta standardized,Alpha standardized,Gamma standardized,Delta standardized,AW Index standardized,Vamv Valence standardized,Kirk Valence standardized,Ram12 Valence standardized,Ram15 Valence standardized,Choice Index (Gamma) standardized,Choice Index (Beta) standardized,Effort Index 1 standardized,Effort Index 2 standardized,Workload_mean standardized,Workload_mean2 standardized,SASI standardized,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,DOB,Gender,Unaided Recall,Aided Recall,Content Appreciation,Category,Frame_Num,Duration AD,KEY,Ad name,VOLTI,TONALITÀ DOMINANTE CALDA,BRAND,PERSON SHOWN,CELEBRITY PRESENT,CHILDREN,GROUP OF PEOPLE,ANIMAL,HAPPY,SAD,ANGRY,CONFUSED,DISGUSTED,SURPRISED,CALM,FEAR,STORYTELLING,TONALITà DOMINANTE CALDA,DYNAMISM,NUTRITION,PRINCIPAL CHARACTER IS FEMALE,CELEBRITY PRESENT.1,CHILDREN.1,ANIMAL.1,CREATIVITY,FAMIGLIA E AMICI,ANIMATED,frame,luminance,dominant_color,saturation,hue,contrast,num_edges,symmetry,feature_complexity,entropy,colorfulness,hog_features,texture_contrast,video,motion_per_frame,motion_video,First_Brand,file,frame_index,mean_loudness,std_loudness,onset_density,mean_timbre_centroid,std_timbre_centroid,_merge,Delta,Theta,Alpha,Beta,Gamma,Mean ISC,volume_dbfs,diff_percent
0,s10,505536.0,500,AMAZON,Subdivision 1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_180,1.0,8.0,ad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-17 15:50:34,2024-04-17 15:57:46,IP Address,93.51.247.100,100.0,431.0,True,2024-04-17 15:57:46.450000,R_5dMsXsUPvJePDAo,NaN,NaN,NaN,NaN,45.4722,9.1922,anonymous,EN,1974,Femmina,1,1,7.675954,NaN,1,30,AMAZON_1,AMAZON,0.0,1.0,0.0,0.0,0

### Eliminiamo tutti i frame in più <ovvero quelli che non hanno avuto una corrispondenza nel merge> e quindi anche la FREGOLA

In [32]:
merged_feat_bio=merged_feat_bio[~(merged_feat_bio['_merge']== 'left_only')]

In [33]:
merged_feat_bio[(merged_feat_bio['ID']=='s99')&(merged_feat_bio['Content']=='MULTICENTRUM')&(merged_feat_bio['Vision']==1)]

,ID,Start (ms),Duration (ms),Content,Comment,Peak Count,Peaks Per Minute,Average Peak Amplitude,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Ch8 Delta (dB),Mean PSD Ch8 Theta (dB),Mean PSD Ch8 Alpha (dB),Mean PSD Ch8 Beta (dB),Mean PSD Ch8 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Asymmetry Delta,Asymmetry Theta,Mean Frontal Asymmetry Alpha,Asymmetry Beta,Asymmetry Gamma,Device,Protocol,Vision,Position,Type,Engagement,Engagement(1/alpha),FAA,Pleasantness Index,Pleasantness,Attention,AW Index,Vamv Valence,Kirk Valence,Ram12 Valence,Ram15 Valence,Choice Index (Gamma),Choice Index (Beta),Effort Index 1,Effort Index 2,Workload_mean,Workload_mean2,SASI,Engagement standardized,Pleasantness standardized,Attention standardized,FAA standardized,Engagement(1/alpha) standardized,Asymmetry Delta standardized,Asymmetry Theta standardized,Asymmetry Beta standardized,Asymmetry Gamma standardized,Beta standardized,Alpha standardized,Gamma standardized,Delta standardized,AW Index standardized,Vamv Valence standardized,Kirk Valence standardized,Ram12 Valence standardized,Ram15 Valence standardized,Choice Index (Gamma) standardized,Choice Index (Beta) standardized,Effort Index 1 standardized,Effort Index 2 standardized,Workload_mean standardized,Workload_mean2 standardized,SASI standardized,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,DOB,Gender,Unaided Recall,Aided Recall,Content Appreciation,Category,Frame_Num,Duration AD,KEY,Ad name,VOLTI,TONALITÀ DOMINANTE CALDA,BRAND,PERSON SHOWN,CELEBRITY PRESENT,CHILDREN,GROUP OF PEOPLE,ANIMAL,HAPPY,SAD,ANGRY,CONFUSED,DISGUSTED,SURPRISED,CALM,FEAR,STORYTELLING,TONALITà DOMINANTE CALDA,DYNAMISM,NUTRITION,PRINCIPAL CHARACTER IS FEMALE,CELEBRITY PRESENT.1,CHILDREN.1,ANIMAL.1,CREATIVITY,FAMIGLIA E AMICI,ANIMATED,frame,luminance,dominant_color,saturation,hue,contrast,num_edges,symmetry,feature_complexity,entropy,colorfulness,hog_features,texture_contrast,video,motion_per_frame,motion_video,First_Brand,file,frame_index,mean_loudness,std_loudness,onset_density,mean_timbre_centroid,std_timbre_centroid,_merge,Delta,Theta,Alpha,Beta,Gamma,Mean ISC,volume_dbfs,diff_percent
205345,s99,541324.0,500,MULTICENTRUM,Subdivision 1,0.0,0.0,NaN,110.509598,4.884274,2.279817,0.752142,0.221666,9.620552,2.782275,1.891037,0.879630,NaN,4.268743,2.147336,2.291395,0.744732,NaN,6.756161,3.997606,2.934945,0.765068,0.410204,11.909677,2.844461,1.875426,1.039202,0.543000,14.184042,3.052109,1.948947,1.901516,0.686752,8.063062,2.958012,1.057061,0.388687,0.114762,NaN,NaN,NaN,NaN,NaN,23.615655,3.238173,2.039858,0.924485,0.576899,-1.293,-0.103,-0.281,0.337,-0.530,Laptop,Linear_180,1.0,9.0,ad,0.175157,-3.096,-3.338,-3.1967,-0.281,5.103,-0.366437,-0.037792,0.327508,0.304919,-0.311530,NaN,NaN,-0.245625,-1.656427,0.753767,0.736024,-0.555820,0.859585,0.481499,-0.569776,-0.002336,-0.58

Queste sono le roghe che non hanno trovato una corrispondenza nel merge


In [34]:
no_match_rows = merged_feat_bio[merged_feat_bio['_merge'] == 'left_only']
no_match_rows[no_match_rows['Content'] == 'AMAZON']['ID'].value_counts()
#no_match_rows['Content'].unique()


,count
ID,


In [35]:
merged_feat_bio.columns

Index(['ID', 'Start (ms)', 'Duration (ms)', 'Content', 'Comment', 'Peak Count',
       'Peaks Per Minute', 'Average Peak Amplitude', 'Mean PSD Ch1 Delta (dB)',
       'Mean PSD Ch1 Theta (dB)',
       ...
       'std_timbre_centroid', '_merge', 'Delta', 'Theta', 'Alpha', 'Beta',
       'Gamma', 'Mean ISC', 'volume_dbfs', 'diff_percent'],
      dtype='object', length=193)

In [36]:
merged_feat_bio.drop(columns=['Duration (ms)', 'Comment',  'StartDate',
       'EndDate', 'Status', 'IPAddress', 'Progress', 'Duration (in seconds)',
       'Finished', 'RecordedDate', 'ResponseId', 'RecipientLastName',
       'RecipientFirstName', 'RecipientEmail', 'ExternalReference',
       'LocationLatitude', 'LocationLongitude', 'DistributionChannel',
       'UserLanguage' , 'Ad name', 'Average Peak Amplitude' ], inplace=True)

<ipython-input-36-218d15a01372>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_feat_bio.drop(columns=['Duration (ms)', 'Comment',  'StartDate',


## Leggo GSR

In [37]:

gsr = pd.read_csv("df_eda_compressed_ALL(1).csv", encoding="latin-1")

gsr

,Study Name,Respondent Name,SourceStimuliName,startTimestamp.CAL,GSR.Resistance.CAL,GSR.Conductance.CAL,MINstartTimestamp.CAL,Start (ms),Average.GSR.Conductance.CAL,StdDevp.GSR.Conductance.CAL
0,FCP_IG_smartphone,S1138,SceneRecording-1,2604125.00,280.282093,3.567926,2604125.00,0.0000,3.623373,0.121283
1,FCP_IG_smartphone,S1138,SceneRecording-1,2605125.00,279.793521,3.574073,2604125.00,1000.0000,3.623373,0.121283
2,FCP_IG_smartphone,S1138,SceneRecording-1,2606125.00,277.787075,3.599886,2604125.00,2000.0000,3.623373,0.121283
3,FCP_IG_smartphone,S1138,SceneRecording-1,2607125.00,276.284417,3.619483,2604125.00,3000.0000,3.623373,0.121283
4,FCP_IG_smartphone,S1138,SceneRecording-1,2608125.00,273.620318,3.654716,2604125.00,4000.0000,3.623373,0.121283
...,...,...,...,...,...,...,...,...,...,...
834541,FCP_lineare_60_TV,s253,Stanotte a Parigi_B,12239843.75,202.464425,4.939152,11675882.81,563960.9375,4.557762,0.247991
834542,FCP_lineare_60_TV,s253,Stanotte a Parigi_B,12240843.75,201.886828,4.953271,11675882.81,564960.9375,4.557762,0.247991
834543,FCP_lineare_60_TV,s253,Stanotte a Parigi_B,12241843.75,201.938894,4.951995,11675882.81,565960.9375,4.557762,0.247991
834544,FCP_lineare_60_TV,s253,Stanotte a Parigi_B,12242843.75,202.372444,4.941386,11675882.81,566960.9375,4.557762,0.247991


In [38]:

# Funzione per normalizzare i valori
def normalize_value(value):
    # Estrai i numeri usando una regex
    match = re.search(r'\d+', str(value))  # Converti in stringa per sicurezza
    if match:
        # Ritorna il valore normalizzato
        return f's{match.group(0)}'
    return None  # Ignora valori che non hanno numeri

# Applica la funzione a ogni valore nella colonna 'Respondent Name'
gsr['Respondent Name'] = gsr['Respondent Name'].apply(normalize_value)

# Rimuovi eventuali duplicati dalla nuova colonna
unique_normalized_values = gsr['Respondent Name'].drop_duplicates().dropna().tolist()
# Ordina i valori (opzionale)
unique_normalized_values.sort(key=lambda x: int(x[1:]))  # Ordina per numero


#merged_feat_bio = merged_feat_bio[(merged_feat_bio['ID'] == 's250') & (merged_feat_bio['Protocol'] == 'Linear_120')].sort_values(by='Start (ms)')
merged_feat_bio['Start (s)'] = (merged_feat_bio['Start (ms)']/1000).round()

gsr=gsr.sort_values(by=['Respondent Name','Start (ms)'])

gsr['Start (s)'] = (gsr['Start (ms)']/1000).round()

# Create columns Device, Protocol and Length
def transform_study_name(df):
    # Define functions to extract the new column values based on conditions
    def get_device(study_name):
        if 'TV' in study_name:
            return 'TV'
        elif 'Smartphone' in study_name or 'smartphone' in study_name:
            return 'Smartphone'
        elif 'Laptop' in study_name or 'laptop' in study_name:
            return 'Laptop'
        else:
            return np.nan

    def get_protocol(study_name):
        if 'lineare' in study_name:
            return 'Linear_' + str(get_length(study_name))
        elif 'YouTube' in study_name or 'YT' in study_name or 'Youtube' in study_name:
            return 'YouTube'
        elif 'VOD' in study_name:
            return 'VOD'
        else:
            return np.nan

    def get_length(study_name):
        if '60' in study_name:
            return 60
        elif '120' in study_name:
            return 120
        elif '180' in study_name:
            return 180
        else:
            return np.nan

    # Apply the functions to the 'Study Name' column to create new columns
    gsr['Device'] = gsr['Study Name'].apply(get_device)
    gsr['Protocol'] = gsr['Study Name'].apply(get_protocol)

    return df

# Transform the DataFrame
gsr = transform_study_name(gsr)

gsr[(gsr['Respondent Name']=='s1030') & (gsr['Start (s)']>=400)].head(10)

<ipython-input-38-f54aa116c483>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_feat_bio['Start (s)'] = (merged_feat_bio['Start (ms)']/1000).round()


,Study Name,Respondent Name,SourceStimuliName,startTimestamp.CAL,GSR.Resistance.CAL,GSR.Conductance.CAL,MINstartTimestamp.CAL,Start (ms),Average.GSR.Conductance.CAL,StdDevp.GSR.Conductance.CAL,Start (s),Device,Protocol
544585,FCP_YouTube_TV,s1030,YouTube,6684367.188,4135.948957,0.241783,6284656.250,399710.9375,0.244815,0.006825,400.0,TV,YouTube
473929,FCP_VOD_TV,s1030,Screen recording 1,5712312.500,4354.872956,0.229628,5312046.875,400265.6250,0.228298,0.009820,400.0,TV,VOD
544586,FCP_YouTube_TV,s1030,YouTube,6685367.188,4135.356559,0.241817,6284656.250,400710.9375,0.244815,0.006825,401.0,TV,YouTube
473930,FCP_VOD_TV,s1030,Screen recording 1,5713312.500,4352.898047,0.229732,5312046.875,401265.6250,0.228298,0.009820,401.0,TV,VOD
544587,FCP_YouTube_TV,s1030,YouTube,6686367.188,4131.387489,0.242050,6284656.250,401710.9375,0.244815,0.006825,402.0,TV,YouTube
473931,FCP_VOD_TV,s1030,Screen recording 1,5714312.500,4352.305574,0.229763,5312046.875,402265.6250,0.228298,0.009820,402.0,TV,VOD
544588,FCP_YouTube_TV,s1030,YouTube,6687367.188,4130.024972,0.242129,6284656.250,402710.9375,0.244815,0.006825,403.0,TV,YouTube
473932,FCP_VOD_TV,s1030,Screen recording 1,5715312.500,4352.108337,0.229774,5312046.875,403265.6250,0.228298,0.009820,403.0,TV,VOD
544589,FCP_YouTube_TV,s1030,YouTube,6688367.188,4132.454242,0.241987,6284656.250,403710.9375,0.244815,0.006825,404.0,TV,YouTube
473933,FCP_VOD_TV,s1030,Screen recording 1,5716312.500,4350.862385,0.229840,5312046.875,404265.6250,0.228298,0.009820,404.0,TV,VOD


In [39]:
gsr[(gsr['Respondent Name']=='s1013')&(gsr['Protocol']=='YouTube')].head(10)

,Study Name,Respondent Name,SourceStimuliName,startTimestamp.CAL,GSR.Resistance.CAL,GSR.Conductance.CAL,MINstartTimestamp.CAL,Start (ms),Average.GSR.Conductance.CAL,StdDevp.GSR.Conductance.CAL,Start (s),Device,Protocol
684961,FCP_Youtube_Laptop_NEW,s1013,Website 1,1809085.938,928.746441,1.076724,1809085.938,0.000,0.79531,0.181422,0.0,Laptop,YouTube
684962,FCP_Youtube_Laptop_NEW,s1013,Website 1,1810007.813,933.689245,1.071023,1809085.938,921.875,0.79531,0.181422,1.0,Laptop,YouTube
684963,FCP_Youtube_Laptop_NEW,s1013,Website 1,1811007.813,938.628821,1.065386,1809085.938,1921.875,0.79531,0.181422,2.0,Laptop,YouTube
684964,FCP_Youtube_Laptop_NEW,s1013,Website 1,1812007.813,943.186338,1.060238,1809085.938,2921.875,0.79531,0.181422,3.0,Laptop,YouTube
684965,FCP_Youtube_Laptop_NEW,s1013,Website 1,1813007.813,948.318043,1.054501,1809085.938,3921.875,0.79531,0.181422,4.0,Laptop,YouTube
684966,FCP_Youtube_Laptop_NEW,s1013,Website 1,1814007.813,953.316548,1.048972,1809085.938,4921.875,0.79531,0.181422,5.0,Laptop,YouTube
684967,FCP_Youtube_Laptop_NEW,s1013,Website 1,1815007.813,957.256056,1.044653,1809085.938,5921.875,0.79531,0.181422,6.0,Laptop,YouTube
684968,FCP_Youtube_Laptop_NEW,s1013,Website 1,1816007.813,951.581985,1.050897,1809085.938,6921.875,0.79531,0.181422,7.0,Laptop,YouTube
684969,FCP_Youtube_Laptop_NEW,s1013,Website 1,1817007.813,943.607371,1.059763,1809085.938,7921.875,0.79531,0.181422,8.0,Laptop,YouTube
684970,FCP_Youtube_Laptop_NEW,s1013,Website 1,1818007.813,946.394466,1.056645,1809085.938,8921.875,0.79531,0.181422,9.0,Laptop,YouTube


### Standardizziamo il GSR
Standardizziamo due volte il GSR, una rispetto rispetto alla visione completa


Se si vuole fare rispetto al content_A fare come nelle parti commentate delle prossime celle (ma trovare il modo di risolvere il caso dei VOD e YT, in quanto il lineare va bene)

In [40]:
# gsr2 = pd.merge(gsr, df_adv.groupby(['ID', 'Protocol'])['Start (ms)'].min().reset_index(),
#                 left_on=['Respondent Name', 'Protocol'], right_on=['ID', 'Protocol'])
# gsr2 = gsr2.rename(columns={'Start (ms)_x': 'Start (ms)', 'Start (ms)_y': 'Ad_Start'})
# gsr2

In [41]:
def standardize_both(group):
    # Standardizzazione per content_A
    # content_a = group[group['Start (ms)'] < group['Ad_Start']]
    # mean_a = content_a['GSR.Conductance.CAL'].mean()
    # std_a = content_a['GSR.Conductance.CAL'].std()
    # group['GSR_standardized_content_A'] = (group['GSR.Conductance.CAL'] - mean_a) / std_a if std_a != 0 else 0

    # Standardizzazione per tutte le osservazioni
    mean_all = group['GSR.Conductance.CAL'].mean()
    std_all = group['GSR.Conductance.CAL'].std()
    group['GSR_standardized_all'] = (group['GSR.Conductance.CAL'] - mean_all) / std_all if std_all != 0 else 0

    return group

# Applicazione della funzione# Applicazione della funzione
gsr = gsr.groupby(['Respondent Name', 'Protocol']).apply(standardize_both).reset_index(drop = True)
#gsr2.drop(columns=['Ad_Start','ID'], inplace=True)


<ipython-input-41-ac356e43601f>:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gsr = gsr.groupby(['Respondent Name', 'Protocol']).apply(standardize_both).reset_index(drop = True)


In [42]:
gsr.head()

,Study Name,Respondent Name,SourceStimuliName,startTimestamp.CAL,GSR.Resistance.CAL,GSR.Conductance.CAL,MINstartTimestamp.CAL,Start (ms),Average.GSR.Conductance.CAL,StdDevp.GSR.Conductance.CAL,Start (s),Device,Protocol,GSR_standardized_all
0,FCP_lineare_180_Laptop,s01,4 Ristoranti_A,3195445.313,2432.189227,0.411152,3195445.313,0.0,0.247622,0.152065,0.0,Laptop,Linear_180,1.074624
1,FCP_lineare_180_Laptop,s01,4 Ristoranti_A,3195882.813,2442.078530,0.409490,3195445.313,437.5,0.247622,0.152065,0.0,Laptop,Linear_180,1.063699
2,FCP_lineare_180_Laptop,s01,4 Ristoranti_A,3196882.813,2461.907143,0.406191,3195445.313,1437.5,0.247622,0.152065,1.0,Laptop,Linear_180,1.042021
3,FCP_lineare_180_Laptop,s01,4 Ristoranti_A,3197882.813,2478.690007,0.403440,3195445.313,2437.5,0.247622,0.152065,2.0,Laptop,Linear_180,1.023945
4,FCP_lineare_180_Laptop,s01,4 Ristoranti_A,3198882.813,2493.416172,0.401057,3195445.313,3437.5,0.247622,0.152065,3.0,Laptop,Linear_180,1.008285


# DATASET 0.5 SEC

In [43]:
merged = pd.merge(
    merged_feat_bio,
    gsr,
    left_on=['ID','Protocol', 'Start (s)','Device'],
    right_on=['Respondent Name', 'Protocol','Start (s)','Device'],
    how='left'
)


### elimino quelli che non hanno DOB e gender (sono i partecipanti 105 e 318)
#merged=merged[~((merged['Device'].isna()==False) &(merged['DOB'].isna()==True)) ]
### eliminiamo tutte le righe che non hanno valori per il gsr (sono circa 1900 righe MA NON HO VOGLIA DI CAPIRE PERCHè NON CI SONO I VALORI DEL GSR, QUINDI PER ORA ELIMINIAMOLE)
# merged=merged[~((merged['ID'].isna()==False) &(merged['Respondent Name'].isna()==True)) ]
# ### pleasantness missing values removed
# merged=merged[~((merged['Pleasantness standardized'].isna()==True)) ]

#merged.drop(['Peak Count','Peaks Per Minute','GSR.Resistance.CAL','SourceStimuliName','Respondent Name','Study Name','SourceStimuliName',	'startTimestamp.CAL'	], axis=1,inplace=True)


In [44]:
merged.shape

(160841, 184)

In [45]:
merged.groupby(['ID','Content','Vision'])['Frame_Num'].max()

ID   Content       Vision
s10  AMAZON        1.0       61
     CHANEL        1.0       61
     COOP          1.0       61
     FASTWEB       2.0       31
     IKEA          1.0       31
                             ..
s99  IKEA          1.0       31
     LETE          1.0       31
     MULTICENTRUM  1.0       31
                   2.0       31
     MUTTI         1.0       31
Name: Frame_Num, Length: 3631, dtype: int64

In [46]:
# Colonne su cui basare l'eliminazione dei duplicati
columns_to_check = ['ID', 'Content', 'Vision', 'Frame_Num']

# Rimuovi i duplicati basandoti sulle colonne specificate
merged = merged.drop_duplicates(subset=columns_to_check)

In [47]:
merged.drop_duplicates(inplace=True)
merged.groupby(['ID','Content','Vision'])['ID'].count()

ID   Content       Vision
s10  AMAZON        1.0       61
     CHANEL        1.0       61
     COOP          1.0       61
     FASTWEB       2.0       31
     IKEA          1.0       31
                             ..
s99  IKEA          1.0       31
     LETE          1.0       31
     MULTICENTRUM  1.0       31
                   2.0       31
     MUTTI         1.0       31
Name: ID, Length: 3631, dtype: int64

In [48]:
columns_to_drop = [
    'CONFUSED', 'DISGUSTED', 'Study Name',	'Respondent Name',	'SourceStimuliName',	'startTimestamp.CAL',
    'Start (ms)_x', 'Start (s)', 'MINstartTimestamp.CAL',
    'Average.GSR.Conductance.CAL', 'StdDevp.GSR.Conductance.CAL',
    'Peak Count', 'Peaks Per Minute',
 'Category',	'_merge'	,'KEY','Mean PSD Ch8 Delta (dB)',	'Mean PSD Ch8 Theta (dB)','Mean PSD Ch8 Alpha (dB)',	'Mean PSD Ch8 Beta (dB)',	'Mean PSD Ch8 Gamma (dB)'
]


df_cleaned_drop = merged.drop(columns=columns_to_drop)

# FILL NA ATTENZIONE
CONTROLLANDO SE TUTTE LE COLONNE DEL GRUPPO HANNO PIù DEL 30% DI NA. SE Sì ELIMINIAMO IL GRUPPO, ALTRIMENTI INTERPOLIAMO IL VALORE CON QUELLI ADIACENTI

In [63]:
def remove_groups_with_too_many_nans(df, columns_to_check, nan_threshold=0.5):
    """
    Rimuove i gruppi se anche solo una colonna ha più del nan_threshold di NaN.
    """
    # Funzione da applicare per ogni gruppo
    def group_has_excessive_nans(group):
        group_nan_ratios = group[columns_to_check].isna().mean()
        return (group_nan_ratios > nan_threshold).any()  # True se almeno una colonna supera il threshold

    # Filtro i gruppi
    return df.groupby(['ID', 'Content', 'Vision']).filter(lambda group: not group_has_excessive_nans(group))


def interpolate_missing_values_in_groups(df, columns_to_interpolate):
    """
    Interpola i valori mancanti all'interno di ciascun gruppo per le colonne specificate.
    """
    def interpolate_group(group):
        group[columns_to_interpolate] = group[columns_to_interpolate].interpolate(method='linear', limit_direction='both')
        return group

    return df.groupby(['ID', 'Content', 'Vision']).apply(interpolate_group)


# Colonne da controllare e interpolare
columns_to_check = [
   # 'Engagement standardized',
    'Pleasantness standardized',
    'Attention standardized', # 'Engagement(1/alpha) standardized', 'FAA standardized',
    'Delta', 'Theta', 'Alpha', 'Beta', 'Gamma',
    'GSR_standardized_all', 'Beta standardized', 'Gamma standardized',
    'Delta standardized', 'Alpha standardized',
    'Asymmetry Delta standardized',
    'Asymmetry Theta standardized',
    'Asymmetry Beta standardized',
    'Asymmetry Gamma standardized',
    # 'AW Index standardized', 'Vamv Valence standardized',
    # 'Kirk Valence standardized', 'Ram12 Valence standardized',
    # 'Ram15 Valence standardized', 'Choice Index (Gamma) standardized',
    # 'Choice Index (Beta) standardized', 'Effort Index 1 standardized',
    # 'Effort Index 2 standardized',"Workload_mean standardized",'Workload_mean2 standardized','SASI standardized'
]

# Controlla i gruppi e rimuove quelli con troppe colonne NaN
df_filtered = remove_groups_with_too_many_nans(df_cleaned_drop, columns_to_check)

# # Interpola i valori mancanti nei gruppi rimanenti
df_filled3 = interpolate_missing_values_in_groups(df_filtered, columns_to_check)

# df_filled3=df_filtered.copy()   # rimuovi se devi fillare i NA

# Reset dell'indice per chiarezza
df_filled3 = df_filled3.reset_index(drop=True)



<ipython-input-63-d25e2c32f929>:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(['ID', 'Content', 'Vision']).apply(interpolate_group)


In [64]:
len(df_filled3[['ID', 'Content']].drop_duplicates())


1665

In [65]:
num_groups = df_filled3[df_filled3['Vision']==1].groupby(['ID', 'Content', 'Vision'])['ID'].nunique().sum()
print(f"Number of groups: {num_groups}")

Number of groups: 964


In [66]:
df_filled3.shape

(79712, 163)

In [67]:
len(df_filled3['ID'].unique())

387

In [68]:
# Define the combinations of (Content, ID, Vision) to remove
to_remove = [
    ('AMAZON', 's1005', 1),
    ('CHANEL', 's1100', 1),
    ('COOP', 's1110', 1)
]

# Filter out these combinations from the dataframe
filtered_df = df_filled3[~df_filled3[['Content', 'ID', 'Vision']].apply(tuple, axis=1).isin(to_remove)]


In [69]:
filtered_df.drop(columns=['frame'], inplace=True)

In [70]:
filtered_df.columns

Index(['ID', 'Content', 'Mean PSD Ch1 Delta (dB)', 'Mean PSD Ch1 Theta (dB)',
       'Mean PSD Ch1 Alpha (dB)', 'Mean PSD Ch1 Beta (dB)',
       'Mean PSD Ch1 Gamma (dB)', 'Mean PSD Ch2 Delta (dB)',
       'Mean PSD Ch2 Theta (dB)', 'Mean PSD Ch2 Alpha (dB)',
       ...
       'Alpha', 'Beta', 'Gamma', 'Mean ISC', 'volume_dbfs', 'diff_percent',
       'GSR.Resistance.CAL', 'GSR.Conductance.CAL', 'Start (ms)_y',
       'GSR_standardized_all'],
      dtype='object', length=162)

#Creaiamo colonne Age e Age_Group

In [71]:
filtered_df['Age'] = 2024 - filtered_df['DOB']

# Categorizza l'età in fasce: 18-34, 35-54, 55+
filtered_df['Age Group'] = pd.cut(
    filtered_df['Age'],
    bins=[17, 34, 54, 120],  # Intervalli per età
    labels=['18-34', '35-54', '55+'],  # Etichette per le categorie
    right=True
)

filtered_df.drop(columns=['DOB'], inplace=True)

# Creo la variabile segno per i due indicatori di piacevolezza

In [72]:
filtered_df['FAA_sign'] = filtered_df['FAA'].apply(lambda x: 1 if x > 0 else -1)
filtered_df['Pleasantness_sign'] = filtered_df['Pleasantness'].apply(lambda x: 1 if x > 0 else -1)

In [73]:
filtered_df = filtered_df.rename(columns={
    'Delta':'Delta_ISC',
    'Theta':'Theta_ISC',
    'Alpha':'Alpha_ISC',
    'Beta':'Beta_ISC',
    'Gamma':'Gamma_ISC',
    })

In [74]:
filtered_df

,ID,Content,Mean PSD Ch1 Delta (dB),Mean PSD Ch1 Theta (dB),Mean PSD Ch1 Alpha (dB),Mean PSD Ch1 Beta (dB),Mean PSD Ch1 Gamma (dB),Mean PSD Ch2 Delta (dB),Mean PSD Ch2 Theta (dB),Mean PSD Ch2 Alpha (dB),Mean PSD Ch2 Beta (dB),Mean PSD Ch2 Gamma (dB),Mean PSD Ch3 Delta (dB),Mean PSD Ch3 Theta (dB),Mean PSD Ch3 Alpha (dB),Mean PSD Ch3 Beta (dB),Mean PSD Ch3 Gamma (dB),Mean PSD Ch4 Delta (dB),Mean PSD Ch4 Theta (dB),Mean PSD Ch4 Alpha (dB),Mean PSD Ch4 Beta (dB),Mean PSD Ch4 Gamma (dB),Mean PSD Ch5 Delta (dB),Mean PSD Ch5 Theta (dB),Mean PSD Ch5 Alpha (dB),Mean PSD Ch5 Beta (dB),Mean PSD Ch5 Gamma (dB),Mean PSD Ch6 Delta (dB),Mean PSD Ch6 Theta (dB),Mean PSD Ch6 Alpha (dB),Mean PSD Ch6 Beta (dB),Mean PSD Ch6 Gamma (dB),Mean PSD Ch7 Delta (dB),Mean PSD Ch7 Theta (dB),Mean PSD Ch7 Alpha (dB),Mean PSD Ch7 Beta (dB),Mean PSD Ch7 Gamma (dB),Mean PSD Electrode Cluster Average Delta (dB),Mean PSD Electrode Cluster Average Theta (dB),Mean PSD Electrode Cluster Average Alpha (dB),Mean PSD Electrode Cluster Average Beta (dB),Mean PSD Electrode Cluster Average Gamma (dB),Asymmetry Delta,Asymmetry Theta,Mean Frontal Asymmetry Alpha,Asymmetry Beta,Asymmetry Gamma,Device,Protocol,Vision,Position,Type,Engagement,Engagement(1/alpha),FAA,Pleasantness Index,Pleasantness,Attention,AW Index,Vamv Valence,Kirk Valence,Ram12 Valence,Ram15 Valence,Choice Index (Gamma),Choice Index (Beta),Effort Index 1,Effort Index 2,Workload_mean,Workload_mean2,SASI,Engagement standardized,Pleasantness standardized,Attention standardized,FAA standardized,Engagement(1/alpha) standardized,Asymmetry Delta standardized,Asymmetry Theta standardized,Asymmetry Beta standardized,Asymmetry Gamma standardized,Beta standardized,Alpha standardized,Gamma standardized,Delta standardized,AW Index standardized,Vamv Valence standardized,Kirk Valence standardized,Ram12 Valence standardized,Ram15 Valence standardized,Choice Index (Gamma) standardized,Choice Index (Beta) standardized,Effort Index 1 standardized,Effort Index 2 standardized,Workload_mean standardized,Workload_mean2 standardized,SASI standardized,Gender,Unaided Recall,Aided Recall,Content Appreciation,Frame_Num,Duration AD,VOLTI,TONALITÀ DOMINANTE CALDA,BRAND,PERSON SHOWN,CELEBRITY PRESENT,CHILDREN,GROUP OF PEOPLE,ANIMAL,HAPPY,SAD,ANGRY,SURPRISED,CALM,FEAR,STORYTELLING,TONALITà DOMINANTE CALDA,DYNAMISM,NUTRITION,PRINCIPAL CHARACTER IS FEMALE,CELEBRITY PRESENT.1,CHILDREN.1,ANIMAL.1,CREATIVITY,FAMIGLIA E AMICI,ANIMATED,luminance,dominant_color,saturation,hue,contrast,num_edges,symmetry,feature_complexity,entropy,colorfulness,hog_features,texture_contrast,video,motion_per_frame,motion_video,First_Brand,file,frame_index,mean_loudness,std_loudness,onset_density,mean_timbre_centroid,std_timbre_centroid,Delta_ISC,Theta_ISC,Alpha_ISC,Beta_ISC,Gamma_ISC,Mean ISC,volume_dbfs,diff_percent,GSR.Resistance.CAL,GSR.Conductance.CAL,Start (ms)_y,GSR_standardized_all,Age,Age Group,FAA_sign,Pleasantness_sign
0,s10,CHANEL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,Linear_180,1.0,9.0,ad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.982567,-1.181374,NaN,NaN,0.455105,-0.252058,2.588862,1.676495,-0.786279,-1.231586,0.360276,1.721515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Femmina,1,1,7.675954,1,30,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,0.0,1.0,16.000000,"[16.0, 16.0, 16.0]",0.000000,0.000000,0.0,0.0,1.000000,0.696373,-1.442695e-07,0.000000,0.000000,0.000000,CHANEL,0.000000,0.555981,0.0,CHANEL,1.0,-3.467661,5.036628,2.000000,1124.703990,989.435663,-0.000239,-0.037402,0.140077,0.704353,0.916049,0.344568,-35.115642,24.305064,886.002671,1.128673,536445.3125,-0.860123,50,35-54,-1,-1
1,s10,CHANEL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.

In [75]:
filtered_df.to_csv('merged_feat_bio.csv', index=False)

# Check of Cleanliness
Nota che i Na vengono rimossi prima di eseguire i modelli, dopo aver eliminato le colonne non necessarie


In [76]:
import pandas as pd
import numpy as np

def verify_dataset_cleanliness(dataset):
    """
    Verifies the cleanliness of a dataset by checking for common issues.
    Args:
        dataset (pd.DataFrame): The input dataset.
    Returns:
        dict: A summary of issues found in the dataset.
    """
    issues = {}

    # Missing Values
    missing_values = dataset.isnull().sum()
    issues['missing_values'] = missing_values[missing_values > 0].to_dict()

    # Duplicates
    duplicates_count = dataset.duplicated().sum()
    issues['duplicates'] = duplicates_count

    # Outliers (using the IQR method for numerical columns)
    outlier_info = {}
    for col in dataset.select_dtypes(include=[np.number]).columns:
        Q1 = dataset[col].quantile(0.25)
        Q3 = dataset[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = dataset[(dataset[col] < (Q1 - 1.5 * IQR)) | (dataset[col] > (Q3 + 1.5 * IQR))]
        if not outliers.empty:
            outlier_info[col] = len(outliers)
    issues['outliers'] = outlier_info

    # Data Type Issues
    data_types = dataset.dtypes.to_dict()
    issues['data_types'] = data_types

    # Consistency Checks (e.g., unique values for categorical columns)
    consistency_info = {}
    for col in dataset.select_dtypes(include=['object']).columns:
        unique_values = dataset[col].nunique()
        consistency_info[col] = unique_values
    issues['consistency'] = consistency_info

    return issues


cleanliness_report = verify_dataset_cleanliness(filtered_df)

# Print a summary of issues
for issue, details in cleanliness_report.items():
    print(f"\n{issue.upper()}:\n{details}")



MISSING_VALUES:
{'Mean PSD Ch1 Delta (dB)': 13943, 'Mean PSD Ch1 Theta (dB)': 14718, 'Mean PSD Ch1 Alpha (dB)': 12523, 'Mean PSD Ch1 Beta (dB)': 11213, 'Mean PSD Ch1 Gamma (dB)': 13592, 'Mean PSD Ch2 Delta (dB)': 21549, 'Mean PSD Ch2 Theta (dB)': 21691, 'Mean PSD Ch2 Alpha (dB)': 19779, 'Mean PSD Ch2 Beta (dB)': 18766, 'Mean PSD Ch2 Gamma (dB)': 19986, 'Mean PSD Ch3 Delta (dB)': 22402, 'Mean PSD Ch3 Theta (dB)': 22587, 'Mean PSD Ch3 Alpha (dB)': 20491, 'Mean PSD Ch3 Beta (dB)': 18957, 'Mean PSD Ch3 Gamma (dB)': 20274, 'Mean PSD Ch4 Delta (dB)': 21871, 'Mean PSD Ch4 Theta (dB)': 22115, 'Mean PSD Ch4 Alpha (dB)': 19917, 'Mean PSD Ch4 Beta (dB)': 17980, 'Mean PSD Ch4 Gamma (dB)': 19374, 'Mean PSD Ch5 Delta (dB)': 22184, 'Mean PSD Ch5 Theta (dB)': 22591, 'Mean PSD Ch5 Alpha (dB)': 20294, 'Mean PSD Ch5 Beta (dB)': 19069, 'Mean PSD Ch5 Gamma (dB)': 20083, 'Mean PSD Ch6 Delta (dB)': 20771, 'Mean PSD Ch6 Theta (dB)': 20910, 'Mean PSD Ch6 Alpha (dB)': 18383, 'Mean PSD Ch6 Beta (dB)': 18105, 'M

-----------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
